In [2]:
import polars as pl
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt
import os
import re
import matplotlib.pyplot as plt
from shapely.geometry import box
import pickle
import duckdb
import pathlib
import json




In [3]:
config = {"MAX_DATE": '2024-07-20', "EXISTING_PIDS": []}

with open("sample.json", "w") as outfile: 
    json.dump(config, outfile)

In [4]:
trips_actual_DIR = "cta-stop-watch/cta-stop-etl/out/trips/*.parquet"
# combine with duckdb
cmd_pings = f"""
SELECT count(1), count(distinct unique_trip_vehicle_day)
FROM read_parquet('{trips_actual_DIR}')
"""
pings = duckdb.execute(cmd_pings).df()
pings

,count(1),count(DISTINCT unique_trip_vehicle_day)
0,603362389,9881650


In [5]:
# orig trips

trips_DIR = "cta-stop-watch/cta-stop-etl/out/cta_bus_full_day_data_v2.parquet"

trips_actual_DIR = "cta-stop-watch/cta-stop-etl/out/trips/*.parquet"
# combine with duckdb
cmd_pings = f"""
SELECT count(1), count(distinct unique_trip_vehicle_day)
FROM read_parquet('{trips_DIR}')
"""


cmd_trips_actual = f"""
SELECT count(distinct unique_trip_vehicle_day)
FROM read_parquet('{trips_actual_DIR}')
"""

pings = duckdb.execute(cmd_pings).df()


trips = duckdb.execute(cmd_trips_actual).df()


print(f"There are {pings['count(1)'][0]:,} bus pings in the original dataset from {pings['count(DISTINCT unique_trip_vehicle_day)'][0]:,} unique trips")

print(f"We processed {trips['count(DISTINCT unique_trip_vehicle_day)'][0]:,} unique trips")

There are 110,594,667 bus pings in the original dataset from 10,235,984 unique trips
We processed 9,881,650 unique trips


In [3]:
#

df = pl.read_parquet('/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/out/trips/trips_2922_full.parquet')

In [11]:
def all_pids(DIR, type):
    """
    Find list of all pids currently available.
    """

    if type == "trip_data":
        PID_DIR = f"{DIR}/pids"
        pids = []
        for pid_file in os.listdir(PID_DIR):
            # Skip auxiliary files in folder, only process parquets
            if not pid_file.endswith(".parquet"):
                continue
            numbers = re.findall(r"\d+", pid_file)
            pid = numbers[0]
            pids.append(pid)
    elif type == "processed_patterns":
        pattern_current_DIR = f"{DIR}/patterns_current"
        pattern_historic_DIR = f"{DIR}/patterns_historic"

        pids_c = []
        for pid_file in os.listdir(pattern_current_DIR):

            if not pid_file.endswith(".parquet"):
                continue

            numbers = re.findall(r"\d+", pid_file)
            pid = numbers[0]
            pids_c.append(pid)

        pids_h = []
        for pid_file in os.listdir(pattern_historic_DIR):
            if not pid_file.endswith(".parquet"):
                continue

            numbers = re.findall(r"\d+", pid_file)
            pid = numbers[0]
            pids_h.append(pid)

        pids = set(pids_c + pids_h)

    return pids

In [4]:
test_trips = pd.read_parquet('/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/out/trips_test/trips_6411_full.parquet')

In [52]:

trips_DIR = "cta-stop-watch/cta-stop-etl/out/cta_bus_full_day_data_v2.parquet"
# combine with duckdb
cmd_number = f"""
SELECT cast(pid as int) as pid
FROM read_parquet('{trips_DIR}')
group by pid
"""

trip_pids = duckdb.execute(cmd_number).df()

In [23]:
pattern_pids = all_pids("cta-stop-watch/cta-stop-etl/out", "processed_patterns") 

In [24]:
pattern_pids_df  = pd.DataFrame(list(pattern_pids), columns=["pid"])

In [40]:
pattern_pids_df['pid_pattern'] = pattern_pids_df['pid'].astype(str)
trip_pids['pid'] = trip_pids['pid'].astype(str)

In [43]:
join_df = trip_pids.merge(pattern_pids_df, on='pid', how='left')

In [50]:
join_df[join_df['pid'] == '17470']

,pid,pid_pattern
106,17470,NaN


In [93]:
trips_DIR = "cta-stop-watch/cta-stop-etl/out/cta_bus_full_day_data_v2.parquet"
# combine with duckdb
cmd_number = f"""
SELECT rt, cast(pid as int) pid, count(distinct unique_trip_vehicle_day)
FROM read_parquet('{trips_DIR}')
group by 1,2
"""

test_pid = duckdb.execute(cmd_number).df()

In [97]:
test_pid

,rt,pid,count(DISTINCT unique_trip_vehicle_day)
0,151,6414,39751
1,3,5342,5956
2,6,14134,52978
3,35,7205,5307
4,72,12432,9317
...,...,...,...
943,146,21482,78
944,8,8957,51
945,152,8460,27
946,4,20422,57


In [94]:
missing = join_df[join_df['pid_pattern'].isna()]['pid']

In [95]:
missing.tolist()

['17466',
 '7733',
 '16544',
 '5071',
 '4602',
 '8098',
 '5897',
 '14123',
 '5434',
 '4347',
 '8463',
 '17470',
 '23989',
 '6427',
 '19370',
 '441',
 '5432',
 '5348',
 '16546',
 '7123',
 '8466',
 '23988',
 '4247',
 '7122',
 '8212',
 '11674',
 '6493',
 '5429',
 '7126',
 '14101',
 '6660',
 '5430',
 '7448',
 '16543',
 '19364',
 '7907',
 '4219',
 '19387',
 '3226',
 '20423',
 '21482',
 '19368',
 '8131',
 '11677',
 '7446',
 '1466',
 '5433',
 '8424',
 '8688',
 '8460',
 '19375',
 '4529',
 '19389',
 '20425',
 '17473',
 '19362',
 '8133',
 '8132',
 '6596',
 '5568',
 '10908',
 '6672',
 '8386',
 '6979',
 '7906',
 '5431',
 '13252',
 '19386',
 '19385',
 '20424',
 '7121',
 '5342',
 '8136',
 '8385',
 '2645',
 '11675',
 '8182',
 '8464',
 '8730',
 '8139',
 '7466',
 '19391',
 '8957',
 '13242',
 '6597',
 '4244',
 '1458',
 '13246',
 '1478',
 '6659',
 '2268',
 '1730',
 '8462',
 '3761',
 '7861',
 '5343',
 '432',
 '6787',
 '8135',
 '2290',
 '10942',
 '2271',
 '7386',
 '1043',
 '19399',
 '11680']

In [99]:
test_pid['pid'] =  test_pid['pid'].astype(str)

test_pid[test_pid['pid'].isin(missing.tolist())].sort_values('count(DISTINCT unique_trip_vehicle_day)',ascending=False).head(10)

,rt,pid,count(DISTINCT unique_trip_vehicle_day)
389,124,17466,9839
253,3,5343,6675
1,3,5342,5956
770,63,4219,4352
895,67,11674,4051
534,4,432,3401
274,4,441,3280
127,4,8131,3273
656,4,8136,3122
624,55,20424,1915


In [18]:

tt_DIR = "cta-stop-watch/cta-stop-etl/out/clean_timetables/*.parquet"
# combine with duckdb
cmd_number = f"""
SELECT route_id,pid,stop_id
FROM read_parquet('{tt_DIR}')
group by route_id,pid,stop_id
"""

df = duckdb.execute(cmd_number).pl()

df.write_parquet('rt_pid_stop.parquet')

In [10]:
df.filter(pl.col("route_id") == "85A")

route_id,pid,stop_id
str,str,str
"""85A""","""05676""","""11684"""
"""85A""","""05676""","""11516"""
"""85A""","""05675""","""11663"""
"""85A""","""05675""","""11666"""
"""85A""","""05676""","""11673"""
…,…,…
"""85A""","""05675""","""11646"""
"""85A""","""05675""","""11655"""
"""85A""","""05675""","""11665"""


In [15]:
xwalk = df.with_columns(pl.col("pid").cast(pl.Int32).cast(pl.String))

xwalk.select(pl.col(["rt", "pid", 'stpid'])).unique(["rt", "pid", 'stpid']).sink_parquet(
    "rt_pid_stop.parquet"
)


ColumnNotFoundError: stpid

This error occurred with the following context stack:
	[1] 'select' failed
	[2] 'unique' input failed to resolve
	[3] 'sink' input failed to resolve


In [13]:
test = pd.read_parquet('/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/out/trips/trips_2922_full.parquet')
test

,seg_combined,typ,stop_sequence,bus_stop_time,speed_mph,unique_trip_vehicle_day,stpid,p_stp_id,vid,rt,pid
0,0.0,S,1,2024-03-30 05:28:22.422872653,14.566251,782922.01000198610182024-03-30,16109,2922-16109,1018,78,2922
1,2.0,S,2,2024-03-30 05:28:48.319594671,14.566251,782922.01000198610182024-03-30,11317,2922-11317,1018,78,2922
2,4.0,S,3,2024-03-30 05:29:19.489215540,14.566251,782922.01000198610182024-03-30,11318,2922-11318,1018,78,2922
3,6.0,S,4,2024-03-30 05:29:51.016620576,14.566251,782922.01000198610182024-03-30,11319,2922-11319,1018,78,2922
4,8.0,S,5,2024-03-30 05:30:22.568124900,14.566251,782922.01000198610182024-03-30,11320,2922-11320,1018,78,2922
...,...,...,...,...,...,...,...,...,...,...,...
87460,122.0,S,47,2023-07-13 08:40:06.938010310,5.768452,782922.08835723784652023-07-13,4915,2922-4915,8465,78,2922
87461,124.0,S,48,2023-07-13 08:41:03.374860878,5.768452,782922.08835723784652023-07-13,4916,2922-4916,8465,78,2922
87462,126.0,S,49,2023-07-13 08:41:47.902111765,5.768452,782922.08835723784652023-07-13,4917,2922-4917,8465,78,2922
87463,128.0,S,50,2023-07-13 08:42:32.961052007,3.913872,782922.08835723784652023-07-13,11362,2922-11362,8465,78,2922


In [14]:
test.groupby('pid').count()

,seg_combined,typ,stop_sequence,bus_stop_time,speed_mph,unique_trip_vehicle_day,stpid,p_stp_id,vid,rt
pid,,,,,,,,,,
2922,87465,87465,87465,87465,87465,87465,87465,87465,87465,87465


In [10]:
test = pd.read_parquet('/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/out/patterns_raw/pid_95_raw.parquet')

In [11]:
test

,seq,lat,lon,typ,stpid,stpnm,pdist
0,1,41.911304,-87.631222,S,923,North Ave & Clark Terminal,0.0
1,2,41.911241,-87.631290,W,None,None,0.0
2,3,41.911237,-87.631391,W,None,None,0.0
3,4,41.911239,-87.631995,W,None,None,0.0
4,5,41.911235,-87.632092,S,924,North Ave & Clark,277.0
...,...,...,...,...,...,...,...
432,433,41.910720,-87.804430,W,None,None,0.0
433,434,41.910730,-87.804000,W,None,None,0.0
434,435,41.909400,-87.803950,W,None,None,0.0
435,436,41.909284,-87.804012,W,None,None,0.0


## Missing Patterns

In [107]:
df_full = pd.read_parquet("cta-stop-watch/cta-stop-etl/out/2023_cta_bus_full_day_data_v2.parquet")

In [130]:
df_full.head()
trips_by_pattern = df_full.groupby('pid')['unique_trip_vehicle_day'].nunique().sort_values().reset_index()
trips_by_pattern.rename(columns={'unique_trip_vehicle_day':'trip_count'}, inplace=True)

In [134]:
trips_by_pattern[trips_by_pattern['pid'] == 5908]

,pid,trip_count
501,5908.0,1815


In [123]:
df_5908 = pd.read_parquet("cta-stop-watch/cta-stop-etl/out/pids/5908.parquet")

In [125]:
df_5908['unique_trip_vehicle_day'].nunique()

1815

In [ ]:
patterns_in_data = df_full['pid'].unique()


PID_DIR = "cta-stop-watch/cta-stop-etl/out/patterns"
pids = []
for pid_file in os.listdir(PID_DIR):

    numbers = re.findall(r'\d+', pid_file)
    pid = numbers[0]
    pids.append(pid)

current_patterns = {int(item) for item in pids}
current_patterns_count = len(current_patterns)

patterns_in_data
patterns_in_data_count = len(patterns_in_data)

print(current_patterns_count, patterns_in_data_count, current_patterns_count/patterns_in_data_count)

missing_patterns_in_current_data = set(patterns_in_data).symmetric_difference(set(current_patterns))
missing_patterns_in_current_data_count = len(missing_patterns_in_current_data)
missing_patterns_in_current_data


In [20]:
df = gpd.read_parquet("/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/test_full_pattern.parquet")

In [21]:
df

,seg_combined,typ,bus_stop_time,time_diff,time_diff_seconds,stop_dist,speed_mph,unique_trip_vehicle_day,stpid,p_stp_id,geometry,data_time,b_value,ping_time_diff
0,0.0,S,2023-01-03 15:50:54.601824268,NaT,31.755481,231.977430,16.341103,1523910.02353403138834755113982023-01-03,1163,3910-1163,POINT (357119.88826 586818.65610),NaT,NaN,NaT
1,6.0,S,2023-01-03 15:51:26.357305113,0 days 00:00:00,33.642695,245.763746,16.341103,1523910.02353403138834755113982023-01-03,4886,3910-4886,POINT (357073.35121 587027.55237),NaT,NaN,NaT
2,NaN,None,2023-01-03 15:52:00.000000000,NaT,NaN,NaN,NaN,None,None,None,None,2023-01-03 15:52:00,2.0,0 days 00:05:00
3,12.0,S,2023-01-03 15:53:01.372940685,19360 days 15:53:01.372940685,29.955544,218.828687,16.341103,1523910.02353403138834755113982023-01-03,1061,3910-1061,POINT (357061.61352 587012.68478),NaT,NaN,NaT
4,16.0,S,2023-01-03 15:53:36.121257870,0 days 00:00:34.748317185,34.740947,253.786604,16.341103,1523910.02353403138834755113982023-01-03,1062,3910-1062,POINT (357111.10486 586807.58883),NaT,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8261,366.0,S,2023-02-16 08:10:05.775211969,0 days 00:00:20.382374732,20.378051,197.905780,21.724519,1523910.02387034868835622682882023-02-16,12617,3910-12617,POINT (344364.51029 586189.08709),NaT,NaN,NaT
8262,373.0,S,2023-02-16 08:10:50.936641265,0 days 00:00:45.161429296,45.151851,206.960459,10.253359,1523910.02387034868835622682882023-02-16,12618,3910-12618,POINT (344166.77436 586183.41261),NaT,NaN,NaT
8263,375.0,S,2023-02-16 08:10:57.662068710,0 days 00:00:06.725427445,210.837075,966.404199,10.253359,1523910.02387034868835622682882023-02-16,12619,3910-12619,POINT (343959.92320 586176.68759),NaT,NaN,NaT
8264,NaN,None,2023-02-16 08:12:00.000000000,NaT,NaN,NaN,NaN,None,None,None,None,2023-02-16 08:12:00,18.0,NaT


In [16]:
df.to_crs(epsg=26971, inplace=True)

In [26]:
trips_df = pd.read_parquet("cta-stop-watch/cta-stop-etl/out/pids/95.parquet")

In [ ]:
#351977
#trips_df.groupby('unique_trip_vehicle_day').filter(lambda x: len(x) == 1)
#333

In [32]:
trips_df.groupby('unique_trip_vehicle_day').filter(lambda x: len(x) > 1)
# 351644

,vid,tmstmp,lat,lon,hdg,pid,rt,des,pdist,dly,tatripid,origtatripno,tablockid,zone,scrape_file,data_time,data_hour,data_date,unique_trip_vehicle_day
0,2029,20230101 05:57,41.911240,-87.632195,272,95.0,72,Harlem,277,False,1010048,235318404,72 -801,None,bus_data/2023-01-01/05:57:56.json,2023-01-01 05:57:00,5,2023-01-01,7295.0235318404101004820292023-01-01
1,2029,20230101 06:02,41.910974,-87.645285,272,95.0,72,Harlem,4027,False,1010048,235318404,72 -801,None,bus_data/2023-01-01/06:02:56.json,2023-01-01 06:02:00,6,2023-01-01,7295.0235318404101004820292023-01-01
2,2029,20230101 06:07,41.910569,-87.671994,264,95.0,72,Harlem,11737,False,1010048,235318404,72 -801,None,bus_data/2023-01-01/06:07:56.json,2023-01-01 06:07:00,6,2023-01-01,7295.0235318404101004820292023-01-01
3,2029,20230101 06:12,41.910391,-87.682323,267,95.0,72,Harlem,14854,False,1010048,235318404,72 -801,None,bus_data/2023-01-01/06:12:56.json,2023-01-01 06:12:00,6,2023-01-01,7295.0235318404101004820292023-01-01
4,2029,20230101 06:17,41.910099,-87.712382,267,95.0,72,Harlem,23531,False,1010048,235318404,72 -801,None,bus_data/2023-01-01/06:17:56.json,2023-01-01 06:17:00,6,2023-01-01,7295.0235318404101004820292023-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351972,8504,20231231 23:52,41.910702,-87.663037,265,95.0,72,Harlem,8650,False,1010112,252003911,72 -810,None,bus_data/2023-12-31/23:52:56.json,2023-12-31 23:52:00,23,2023-12-31,7295.0252003911101011285042023-12-31
351973,8160,20231231 23:52,41.909939,-87.726098,272,95.0,72,Harlem,25770,False,1010111,252003915,72 -852,None,bus_data/2023-12-31/23:52:56.json,2023-12-31 23:52:00,23,2023-12-31,7295.0252003915101011181602023-12-31
351974,8414,20231231 23:51,41.909602,-87.803963,180,95.0,72,Harlem,48963,True,1010109,252003908,72 -851,None,bus_data/2023-12-31/23:52:56.json,2023-12-31 23:51:00,23,2023-12-31,7295.0252003908101010984142023-12-31
351975,8504,20231231 23:57,41.910425,-87.680105,267,95.0,72,Harlem,13282,False,1010112,252003911,72 -810,None,bus_data/2023-12-31/23:57:56.json,2023-12-31 23:57:00,23,2023-12-31,7295.0252003911101011285042023-12-31


In [29]:
trips_df.groupby('unique_trip_vehicle_day').filter(lambda x: len(x) == 1)

,vid,tmstmp,lat,lon,hdg,pid,rt,des,pdist,dly,tatripid,origtatripno,tablockid,zone,scrape_file,data_time,data_hour,data_date,unique_trip_vehicle_day
1976,1980,20230102 23:57,41.911304,-87.631226,290,95.0,72,Harlem,0,False,1010113,235318416,72 -802,None,bus_data/2023-01-02/23:57:56.json,2023-01-02 23:57:00,23,2023-01-02,7295.0235318416101011319802023-01-02
2041,8185,20230103 05:32,41.911340,-87.631103,220,95.0,72,Harlem,0,False,1075368,235314408,72 -801,None,bus_data/2023-01-03/05:32:56.json,2023-01-03 05:32:00,5,2023-01-03,7295.0235314408107536881852023-01-03
4274,2002,20230104 20:22,41.911388,-87.631065,344,95.0,72,Harlem,0,False,88357459,235314183,72 -817,None,bus_data/2023-01-04/20:22:56.json,2023-01-04 20:22:00,20,2023-01-04,7295.02353141838835745920022023-01-04
5098,8111,20230105 14:47,41.910685,-87.804760,91,95.0,72,Harlem,51563,False,88357410,235314164,72 -818,None,bus_data/2023-01-05/14:47:56.json,2023-01-05 14:47:00,14,2023-01-05,7295.02353141648835741081112023-01-05
5459,1966,20230105 19:52,41.911304,-87.631226,238,95.0,72,Harlem,0,False,88357456,235314093,72 -858,None,bus_data/2023-01-05/19:52:56.json,2023-01-05 19:52:00,19,2023-01-05,7295.02353140938835745619662023-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343756,2002,20231224 23:57,41.911389,-87.631062,317,95.0,72,Harlem,0,False,1010113,252003909,72 -802,None,bus_data/2023-12-24/23:57:56.json,2023-12-24 23:57:00,23,2023-12-24,7295.0252003909101011320022023-12-24
346043,8150,20231227 00:02,41.910117,-87.803979,182,95.0,72,Harlem,48776,True,1075612,251999068,72 -801,None,bus_data/2023-12-27/00:02:56.json,2023-12-27 00:02:00,0,2023-12-27,7295.0251999068107561281502023-12-27
347111,8115,20231228 00:02,41.910697,-87.804428,82,95.0,72,Harlem,48437,False,1075612,251999068,72 -801,None,bus_data/2023-12-28/00:02:56.json,2023-12-28 00:02:00,0,2023-12-28,7295.0251999068107561281152023-12-28
348261,8430,20231228 23:57,41.910917,-87.648121,267,95.0,72,Harlem,4601,False,1075615,251999070,72 -802,None,bus_data/2023-12-28/23:58:02.json,2023-12-28 23:57:00,23,2023-12-28,7295.0251999070107561584302023-12-28


In [24]:
df.groupby('unique_trip_vehicle_day').filter(lambda x: len(x) == 1)

,seg_combined,typ,bus_stop_time,time_diff,time_diff_seconds,stop_dist,speed_mph,unique_trip_vehicle_day,stpid,p_stp_id,geometry,data_time,b_value,ping_time_diff


In [23]:
df
df.groupby('unique_trip_vehicle_day').filter(lambda x: len(x) > 1)

,seg_combined,typ,bus_stop_time,time_diff,time_diff_seconds,stop_dist,speed_mph,unique_trip_vehicle_day,stpid,p_stp_id,geometry,data_time,b_value,ping_time_diff
0,0.0,S,2023-01-03 15:50:54.601824268,NaT,31.755481,231.977430,16.341103,1523910.02353403138834755113982023-01-03,1163,3910-1163,POINT (357119.88826 586818.65610),NaT,NaN,NaT
1,6.0,S,2023-01-03 15:51:26.357305113,0 days 00:00:00,33.642695,245.763746,16.341103,1523910.02353403138834755113982023-01-03,4886,3910-4886,POINT (357073.35121 587027.55237),NaT,NaN,NaT
3,12.0,S,2023-01-03 15:53:01.372940685,19360 days 15:53:01.372940685,29.955544,218.828687,16.341103,1523910.02353403138834755113982023-01-03,1061,3910-1061,POINT (357061.61352 587012.68478),NaT,NaN,NaT
4,16.0,S,2023-01-03 15:53:36.121257870,0 days 00:00:34.748317185,34.740947,253.786604,16.341103,1523910.02353403138834755113982023-01-03,1062,3910-1062,POINT (357111.10486 586807.58883),NaT,NaN,NaT
5,20.0,S,2023-01-03 15:54:47.869404010,0 days 00:01:11.748146140,71.732929,161.456159,5.034895,1523910.02353403138834755113982023-01-03,12549,3910-12549,POINT (357167.22024 586624.65095),NaT,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8260,361.0,S,2023-02-16 08:09:45.392837237,0 days 00:00:27.621378935,27.615520,198.137036,16.049695,1523910.02387034868835622682882023-02-16,12616,3910-12616,POINT (344562.15888 586195.43298),NaT,NaN,NaT
8261,366.0,S,2023-02-16 08:10:05.775211969,0 days 00:00:20.382374732,20.378051,197.905780,21.724519,1523910.02387034868835622682882023-02-16,12617,3910-12617,POINT (344364.51029 586189.08709),NaT,NaN,NaT
8262,373.0,S,2023-02-16 08:10:50.936641265,0 days 00:00:45.161429296,45.151851,206.960459,10.253359,1523910.02387034868835622682882023-02-16,12618,3910-12618,POINT (344166.77436 586183.41261),NaT,NaN,NaT
8263,375.0,S,2023-02-16 08:10:57.662068710,0 days 00:00:06.725427445,210.837075,966.404199,10.253359,1523910.02387034868835622682882023-02-16,12619,3910-12619,POINT (343959.92320 586176.68759),NaT,NaN,NaT


In [17]:
df[df["unique_trip_vehicle_day"] == '1523910.02353403138834755113982023-01-03'].head(20)

,seg_combined,typ,bus_stop_time,time_diff,time_diff_seconds,stop_dist,speed_mph,unique_trip_vehicle_day,stpid,p_stp_id,geometry,data_time,b_value,ping_time_diff
0,0.0,S,2023-01-03 15:50:54.601824268,NaT,31.755481,231.977430,16.341103,1523910.02353403138834755113982023-01-03,1163,3910-1163,POINT (inf inf),NaT,NaN,NaT
1,6.0,S,2023-01-03 15:51:26.357305113,0 days 00:00:00,33.642695,245.763746,16.341103,1523910.02353403138834755113982023-01-03,4886,3910-4886,POINT (inf inf),NaT,NaN,NaT
3,12.0,S,2023-01-03 15:53:01.372940685,19360 days 15:53:01.372940685,29.955544,218.828687,16.341103,1523910.02353403138834755113982023-01-03,1061,3910-1061,POINT (inf inf),NaT,NaN,NaT
4,16.0,S,2023-01-03 15:53:36.121257870,0 days 00:00:34.748317185,34.740947,253.786604,16.341103,1523910.02353403138834755113982023-01-03,1062,3910-1062,POINT (inf inf),NaT,NaN,NaT
5,20.0,S,2023-01-03 15:54:47.869404010,0 days 00:01:11.748146140,71.732929,161.456159,5.034895,1523910.02353403138834755113982023-01-03,12549,3910-12549,POINT (inf inf),NaT,NaN,NaT
6,23.0,S,2023-01-03 15:55:31.183350780,0 days 00:00:43.313946770,43.304759,175.975010,9.090122,1523910.02353403138834755113982023-01-03,12550,3910-12550,POINT (inf inf),NaT,NaN,NaT
7,26.0,S,2023-01-03 15:56:20.498427426,0 days 00:00:49.315076646,49.304617,326.085070,14.794410,1523910.02353403138834755113982023-01-03,12551,3910-12551,POINT (inf inf),NaT,NaN,NaT
9,32.0,S,2023-01-03 15:57:38.587463273,0 days 00:01:18.089035847,78.072473,212.276485,6.082166,1523910.02353403138834755113982023-01-03,12552,3910-12552,POINT (inf inf),NaT,NaN,NaT
10,40.0,S,2023-01-03 15:58:24.305643510,0 days 00:00:45.718180237,45.708483,188.044578,9.202765,1523910.02353403138834755113982023-01-03,12553,3910-12553,POINT (inf inf),NaT,NaN,NaT
11,47.0,S,2023-01-03 15:59:04.057930824,0 days 00:00:39.752287314,39.743856,221.410207,12.461834,1523910.02353403138834755113982023-01-03,12554,3910-12554,POINT (inf inf),NaT,NaN,NaT


In [34]:
df = gpd.read_parquet("/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/out/trips/test_trips_95_small.parquet")

In [35]:
df

,seg_combined,typ,bus_stop_time,time_diff,time_diff_seconds,stop_dist,speed_mph,unique_trip_vehicle_day,stpid,p_stp_id,geometry,data_time,b_value,ping_time_diff
0,0.0,S,2023-01-04 23:36:45,NaT,15.900749,207.698618,29.219337,7295.0235314066107561419402023-01-04,923,95-923,POINT (358249.87291 582489.58505),NaT,NaN,NaT
1,NaN,None,2023-01-04 23:37:00,NaT,NaN,NaN,NaN,None,None,None,None,2023-01-04 23:37:00,2.0,0 days 00:05:00
2,NaN,None,NaT,NaT,NaN,NaN,NaN,None,None,None,None,2023-01-04 23:42:00,3.0,0 days 00:05:00
3,NaN,None,NaT,NaT,NaN,NaN,NaN,None,None,None,None,2023-01-04 23:47:00,4.0,-1 days +23:50:00
4,NaN,None,NaT,NaT,NaN,NaN,NaN,None,None,None,None,2023-01-04 23:37:00,5.0,0 days 00:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8992,NaN,None,NaT,NaT,NaN,NaN,NaN,None,None,None,None,2023-01-04 18:02:00,17.0,0 days 00:05:00
8993,NaN,None,2023-01-04 18:07:00,NaT,NaN,NaN,NaN,None,None,None,None,2023-01-04 18:07:00,18.0,NaT
8994,427.0,S,2023-01-04 18:07:25,0 days 00:16:08.237846636,25.791926,136.246434,11.816686,7295.02353141668835743584142023-01-04,1004,95-1004,POINT (343771.94900 582306.93701),NaT,NaN,NaT
8995,431.0,S,2023-01-04 18:08:06,0 days 00:00:00,41.990023,221.813248,11.816686,7295.02353141668835743584142023-01-04,1005,95-1005,POINT (343880.65030 582318.26952),NaT,NaN,NaT


In [104]:
df = pd.read_parquet("/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/out/pids/308.parquet")

trips_gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.GeoSeries.from_xy(
            x=df.loc[:, "lon"],
            y=df.loc[:, "lat"],
            crs="EPSG:4326",
        ),
    )
trips_gdf = trips_gdf.sort_values("data_time")
trips_gdf["bus_location_id"] = trips_gdf.index
trips_gdf = trips_gdf[
    ["bus_location_id", "unique_trip_vehicle_day", "vid", "data_time", "geometry"]
]

In [105]:
one_trip = trips_gdf[trips_gdf['unique_trip_vehicle_day']=='103308.02386643248834770510812023-02-23']

In [106]:
one_trip

,bus_location_id,unique_trip_vehicle_day,vid,data_time,geometry
37,37,103308.02386643248834770510812023-02-23,1081,2023-02-23 14:32:00,POINT (-87.62430 41.72084)
38,38,103308.02386643248834770510812023-02-23,1081,2023-02-23 14:37:00,POINT (-87.62481 41.72143)


In [93]:
one_trip.to_crs(epsg=26971, inplace=True)

/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/.venv/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [99]:
one_trip

,bus_location_id,unique_trip_vehicle_day,vid,data_time,geometry
26,26,54B19400.02387861528835805680352023-04-14,8035,2023-04-14 16:17:00,POINT (349437.986 568446.819)
27,27,54B19400.02387861528835805680352023-04-14,8035,2023-04-14 16:22:00,POINT (349437.986 568446.819)
28,28,54B19400.02387861528835805680352023-04-14,8035,2023-04-14 16:27:00,POINT (349437.986 568446.819)
29,29,54B19400.02387861528835805680352023-04-14,8035,2023-04-14 16:32:00,POINT (349437.986 568446.819)
31,31,54B19400.02387861528835805680352023-04-14,8035,2023-04-14 16:37:00,POINT (349446.848 568540.943)
41,41,54B19400.02387861528835805680352023-04-14,8035,2023-04-14 17:27:00,POINT (349458.056 568530.075)
42,42,54B19400.02387861528835805680352023-04-14,8035,2023-04-14 17:32:00,POINT (349458.056 568530.075)


In [100]:
bad_indexes = []
for trip_id, trip_gdf in one_trip.groupby('unique_trip_vehicle_day'):
    # if distance is super close

    for i in range(len(trip_gdf) - 1):
        print(trip_gdf.geometry.iloc[i].distance(trip_gdf.geometry.iloc[i+1]))
        if trip_gdf.geometry.iloc[i].distance(trip_gdf.geometry.iloc[i+1]) < 5:

            bad_indexes.append(trip_gdf.index[i])
        else:
            break

    filtered_trips_gdf = one_trip[~one_trip.index.isin(bad_indexes)] 


0.0
0.0
0.0
94.54065007082218


In [101]:
bad_indexes

[26, 27, 28]

In [102]:
filtered_trips_gdf

,bus_location_id,unique_trip_vehicle_day,vid,data_time,geometry
29,29,54B19400.02387861528835805680352023-04-14,8035,2023-04-14 16:32:00,POINT (349437.986 568446.819)
31,31,54B19400.02387861528835805680352023-04-14,8035,2023-04-14 16:37:00,POINT (349446.848 568540.943)
41,41,54B19400.02387861528835805680352023-04-14,8035,2023-04-14 17:27:00,POINT (349458.056 568530.075)
42,42,54B19400.02387861528835805680352023-04-14,8035,2023-04-14 17:32:00,POINT (349458.056 568530.075)


In [69]:
trips_gdf['unique_trip_vehicle_day'].nunique()

1704

In [66]:
test_trip = trips_gdf[trips_gdf['unique_trip_vehicle_day']=='1523910.02387035108834756782662023-02-16']
test_trip

,bus_location_id,unique_trip_vehicle_day,vid,data_time,geometry
2609,2609,1523910.02387035108834756782662023-02-16,8266,2023-02-16 16:07:00,POINT (-87.80706 41.94579)
2613,2613,1523910.02387035108834756782662023-02-16,8266,2023-02-16 16:12:00,POINT (-87.80706 41.94579)


In [67]:
test_trip.to_crs(epsg=26971, inplace=True)
#box(*test_trip.geometry.total_bounds).area

test_trip.groupby('unique_trip_vehicle_day').filter(lambda x: box(*x.geometry.total_bounds).area > 20)

/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/.venv/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,bus_location_id,unique_trip_vehicle_day,vid,data_time,geometry


In [62]:
test_trip

,bus_location_id,unique_trip_vehicle_day,vid,data_time,geometry
2609,2609,1523910.02387035108834756782662023-02-16,8266,2023-02-16 16:07:00,POINT (343638.090 586215.395)
2613,2613,1523910.02387035108834756782662023-02-16,8266,2023-02-16 16:12:00,POINT (343638.090 586215.395)


In [103]:
'95'.isnumeric()

True

In [57]:
trips_gdf.to_crs

In [58]:
one_trip = trips_gdf[trips_gdf['unique_trip_vehicle_day'] == '1523910.02353403308834754984082023-01-03']
box(*one_trip.geometry.total_bounds).area

8514810.461075222

In [50]:
trips_gdf.groupby('unique_trip_vehicle_day').apply(lambda x: box(*x.geometry.total_bounds).area < 20)

/var/folders/vk/_03tl7qs1sv80_pgkhd8bxch0000gn/T/ipykernel_26906/2936642998.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  trips_gdf.groupby('unique_trip_vehicle_day').apply(lambda x: x['geometry'].nunique() > 1)


unique_trip_vehicle_day
1523910.02353403138834755113982023-01-03    True
1523910.02353403138834755182862023-01-05    True
1523910.02353403138834755183952023-01-06    True
1523910.02353403308834754913942023-01-06    True
1523910.02353403308834754982942023-01-05    True
                                            ... 
1523910.02519923558835623115282023-12-22    True
1523910.02519923558835623116042023-12-20    True
1523910.02519923558835623116152023-12-27    True
1523910.02519923558835623116372023-12-20    True
1523910.02519923558835623183992023-12-19    True
Length: 1704, dtype: bool

In [3]:
df = gpd.read_parquet("/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/out/trips/trips_8124_full.parquet")

In [6]:
df = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:26971')
df.to_crs(epsg=4326, inplace=True)

In [13]:
stops = df[df['typ'] == 'S']
df[df['unique_trip_vehicle_day'] == '318124.02519806145779632023-12-19']

,seg_combined,typ,bus_stop_time,time_diff_seconds,stop_dist,speed_mph,unique_trip_vehicle_day,stpid,p_stp_id,geometry,data_time,time
2166,0.0,S,2023-12-19 19:21:03.108968807,56.891031,277.306190,10.903605,318124.02519806145779632023-12-19,18237,8124-18237,POINT (-87.61533 41.83366),NaT,2023-12-19 19:21:03.108968807
2167,0.5,B,2023-12-19 19:22:00.000000000,NaN,NaN,NaN,318124.02519806145779632023-12-19,None,None,POINT (-87.61533 41.83366),2023-12-19 19:22:00,2023-12-19 19:22:00.000000000
2168,7.0,S,2023-12-19 19:24:05.222674587,90.838364,442.777008,10.903605,318124.02519806145779632023-12-19,2169,8124-2169,POINT (-87.61696 41.83492),NaT,2023-12-19 19:24:05.222674587
2169,12.0,S,2023-12-19 19:24:55.967847758,30.747818,149.875298,10.903605,318124.02519806145779632023-12-19,7503,8124-7503,POINT (-87.61779 41.83846),NaT,2023-12-19 19:24:55.967847758
2170,14.0,S,2023-12-19 19:25:24.062932118,69.319826,337.888355,10.903605,318124.02519806145779632023-12-19,7504,8124-7504,POINT (-87.61959 41.83843),NaT,2023-12-19 19:25:24.062932118
2171,19.0,S,2023-12-19 19:26:17.836102266,58.414415,284.731679,10.903605,318124.02519806145779632023-12-19,18244,8124-18244,POINT (-87.62366 41.83840),NaT,2023-12-19 19:26:17.836102266
2172,22.5,B,2023-12-19 19:27:00.000000000,NaN,NaN,NaN,318124.02519806145779632023-12-19,None,None,POINT (-87.62660 41.83831),2023-12-19 19:27:00,2023-12-19 19:27:00.000000000
2173,25.0,S,2023-12-19 19:27:36.383654366,36.383654,177.346277,10.903605,318124.02519806145779632023-12-19,18239,8124-18239,POINT (-87.62681 41.83813),NaT,2023-12-19 19:27:36.383654366
2174,27.0,S,2023-12-19 19:28:16.219179602,39.835525,194.171867,10.903605,318124.02519806145779632023-12-19,2396,8124-2396,POINT (-87.62670 41.83654),NaT,2023-12-19 19:28:16.219179602
2175,29.0,S,2023-12-19 19:29:02.080145556,45.860966,223.541909,10.903605,318124.02519806145779632023-12-19,18489,8124-18489,POINT (-87.62667 41.83479),NaT,2023-12-19 19:29:02.080145556


In [18]:

with open("cta-stop-watch/cta-stop-etl/out/qc/bad_trips_4370.pickle", "rb") as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    issue_trips = pickle.load(f)

In [19]:
issue_trips

['974370.0238771531218722023-03-08', '974370.0248557437218692023-11-19']

In [21]:
sum([1,2,4])

7

In [41]:
text = """
Processing Patterns
Writing out/patterns/pid_1612_stop.parquet and out/patterns/pid_1612_segment.parquet
Processing stop time for all trips of pattern 1612
Originally 26892 trips
Trying to process 26130 trips for Pattern 1612 after filtering
Processed 25954 trips for Pattern 1612. There was 78 trip(s) with errors.
Time taken for stop times for 1612: 17.733772166570027 minutes
Writing out/patterns/pid_6581_stop.parquet and out/patterns/pid_6581_segment.parquet
Processing stop time for all trips of pattern 6581
Originally 1868 trips
Trying to process 1791 trips for Pattern 6581 after filtering
Processed 1737 trips for Pattern 6581. There was 24 trip(s) with errors.
Time taken for stop times for 6581: 1.071101180712382 minutes
Writing out/patterns/pid_7532_stop.parquet and out/patterns/pid_7532_segment.parquet
Processing stop time for all trips of pattern 7532
Originally 1574 trips
Trying to process 1565 trips for Pattern 7532 after filtering
Processed 1559 trips for Pattern 7532. There was 2 trip(s) with errors.
Time taken for stop times for 7532: 0.9002729296684265 minutes
Writing out/patterns/pid_8124_stop.parquet and out/patterns/pid_8124_segment.parquet
Processing stop time for all trips of pattern 8124
Originally 214 trips
Trying to process 159 trips for Pattern 8124 after filtering
Processed 154 trips for Pattern 8124. There was 0 trip(s) with errors.
Time taken for stop times for 8124: 0.05082105398178101 minutes
Writing out/patterns/pid_3082_stop.parquet and out/patterns/pid_3082_segment.parquet
Processing stop time for all trips of pattern 3082
Originally 1340 trips
Trying to process 1330 trips for Pattern 3082 after filtering
Processed 1327 trips for Pattern 3082. There was 1 trip(s) with errors.
Time taken for stop times for 3082: 0.6657376646995544 minutes
Writing out/patterns/pid_19393_stop.parquet and out/patterns/pid_19393_segment.parquet
Processing stop time for all trips of pattern 19393
Originally 166 trips
Trying to process 157 trips for Pattern 19393 after filtering
Processed 153 trips for Pattern 19393. There was 1 trip(s) with errors.
Time taken for stop times for 19393: 0.06289235353469849 minutes
Writing out/patterns/pid_3637_stop.parquet and out/patterns/pid_3637_segment.parquet
Processing stop time for all trips of pattern 3637
Originally 10322 trips
Trying to process 9517 trips for Pattern 3637 after filtering
Processed 9486 trips for Pattern 3637. There was 11 trip(s) with errors.
Time taken for stop times for 3637: 3.3057287693023683 minutes
Writing out/patterns/pid_9376_stop.parquet and out/patterns/pid_9376_segment.parquet
Processing stop time for all trips of pattern 9376
Originally 32210 trips
Trying to process 30764 trips for Pattern 9376 after filtering
Processed 30370 trips for Pattern 9376. There was 124 trip(s) with errors.
Time taken for stop times for 9376: 22.792467864354453 minutes
Writing out/patterns/pid_7107_stop.parquet and out/patterns/pid_7107_segment.parquet
Processing stop time for all trips of pattern 7107
Originally 31070 trips
Trying to process 30204 trips for Pattern 7107 after filtering
Processed 29946 trips for Pattern 7107. There was 97 trip(s) with errors.
Time taken for stop times for 7107: 19.182951851685843 minutes
Writing out/patterns/pid_6778_stop.parquet and out/patterns/pid_6778_segment.parquet
Processing stop time for all trips of pattern 6778
Originally 4108 trips
Trying to process 4084 trips for Pattern 6778 after filtering
Processed 4066 trips for Pattern 6778. There was 6 trip(s) with errors.
Time taken for stop times for 6778: 2.8853041489919025 minutes
Writing out/patterns/pid_23276_stop.parquet and out/patterns/pid_23276_segment.parquet
Processing stop time for all trips of pattern 23276
Originally 4 trips
Trying to process 4 trips for Pattern 23276 after filtering
Processed 4 trips for Pattern 23276. There was 0 trip(s) with errors.
Time taken for stop times for 23276: 0.0033604343732198077 minutes
Writing out/patterns/pid_7727_stop.parquet and out/patterns/pid_7727_segment.parquet
Processing stop time for all trips of pattern 7727
Originally 1560 trips
Trying to process 1551 trips for Pattern 7727 after filtering
Processed 1551 trips for Pattern 7727. There was 0 trip(s) with errors.
Time taken for stop times for 7727: 0.99133886496226 minutes
Writing out/patterns/pid_903_stop.parquet and out/patterns/pid_903_segment.parquet
Processing stop time for all trips of pattern 903
Originally 665 trips
Trying to process 661 trips for Pattern 903 after filtering
Processed 661 trips for Pattern 903. There was 0 trip(s) with errors.
Time taken for stop times for 903: 0.40039470195770266 minutes
Do not have pattern 5071 available currently
Writing out/patterns/pid_14129_stop.parquet and out/patterns/pid_14129_segment.parquet
Processing stop time for all trips of pattern 14129
Originally 4 trips
Trying to process 4 trips for Pattern 14129 after filtering
Processed 4 trips for Pattern 14129. There was 0 trip(s) with errors.
Time taken for stop times for 14129: 0.002143700917561849 minutes
Writing out/patterns/pid_1328_stop.parquet and out/patterns/pid_1328_segment.parquet
Processing stop time for all trips of pattern 1328
Originally 17793 trips
Trying to process 17694 trips for Pattern 1328 after filtering
Processed 17678 trips for Pattern 1328. There was 6 trip(s) with errors.
Time taken for stop times for 1328: 7.37198717991511 minutes
Writing out/patterns/pid_6136_stop.parquet and out/patterns/pid_6136_segment.parquet
Processing stop time for all trips of pattern 6136
Originally 10780 trips
Trying to process 10642 trips for Pattern 6136 after filtering
Processed 10581 trips for Pattern 6136. There was 26 trip(s) with errors.
Time taken for stop times for 6136: 5.129439918200175 minutes
Writing out/patterns/pid_6127_stop.parquet and out/patterns/pid_6127_segment.parquet
Processing stop time for all trips of pattern 6127
Originally 349 trips
Trying to process 344 trips for Pattern 6127 after filtering
Processed 340 trips for Pattern 6127. There was 2 trip(s) with errors.
Time taken for stop times for 6127: 0.20994642972946168 minutes
Do not have pattern 11675 available currently
Writing out/patterns/pid_8227_stop.parquet and out/patterns/pid_8227_segment.parquet
Processing stop time for all trips of pattern 8227
Originally 13942 trips
Trying to process 13773 trips for Pattern 8227 after filtering
Processed 13745 trips for Pattern 8227. There was 13 trip(s) with errors.
Time taken for stop times for 8227: 10.302305515607198 minutes
Writing out/patterns/pid_7905_stop.parquet and out/patterns/pid_7905_segment.parquet
Processing stop time for all trips of pattern 7905
Originally 1463 trips
Trying to process 1144 trips for Pattern 7905 after filtering
Processed 1140 trips for Pattern 7905. There was 1 trip(s) with errors.
Time taken for stop times for 7905: 0.3813040177027384 minutes
Writing out/patterns/pid_10932_stop.parquet and out/patterns/pid_10932_segment.parquet
Processing stop time for all trips of pattern 10932
Originally 691 trips
Trying to process 685 trips for Pattern 10932 after filtering
Processed 685 trips for Pattern 10932. There was 0 trip(s) with errors.
Time taken for stop times for 10932: 0.502897588411967 minutes
Writing out/patterns/pid_6400_stop.parquet and out/patterns/pid_6400_segment.parquet
Processing stop time for all trips of pattern 6400
Originally 694 trips
Trying to process 689 trips for Pattern 6400 after filtering
Processed 684 trips for Pattern 6400. There was 1 trip(s) with errors.
Time taken for stop times for 6400: 0.3576606829961141 minutes
Writing out/patterns/pid_8230_stop.parquet and out/patterns/pid_8230_segment.parquet
Processing stop time for all trips of pattern 8230
Originally 711 trips
Trying to process 709 trips for Pattern 8230 after filtering
Processed 708 trips for Pattern 8230. There was 0 trip(s) with errors.
Time taken for stop times for 8230: 0.38792335192362465 minutes
Do not have pattern 1730 available currently
Writing out/patterns/pid_4319_stop.parquet and out/patterns/pid_4319_segment.parquet
Processing stop time for all trips of pattern 4319
Originally 9941 trips
Trying to process 9798 trips for Pattern 4319 after filtering
Processed 9752 trips for Pattern 4319. There was 23 trip(s) with errors.
Time taken for stop times for 4319: 11.584111146132152 minutes
Writing out/patterns/pid_4490_stop.parquet and out/patterns/pid_4490_segment.parquet
Processing stop time for all trips of pattern 4490
Originally 15120 trips
Trying to process 14880 trips for Pattern 4490 after filtering
Processed 14831 trips for Pattern 4490. There was 21 trip(s) with errors.
Time taken for stop times for 4490: 5.244935901959737 minutes
Writing out/patterns/pid_10926_stop.parquet and out/patterns/pid_10926_segment.parquet
Processing stop time for all trips of pattern 10926
Originally 1963 trips
Trying to process 1953 trips for Pattern 10926 after filtering
Processed 1953 trips for Pattern 10926. There was 0 trip(s) with errors.
Time taken for stop times for 10926: 1.0301572362581888 minutes
Writing out/patterns/pid_1042_stop.parquet and out/patterns/pid_1042_segment.parquet
Processing stop time for all trips of pattern 1042
Originally 23966 trips
Trying to process 23377 trips for Pattern 1042 after filtering
Processed 23135 trips for Pattern 1042. There was 107 trip(s) with errors.
Time taken for stop times for 1042: 47.23109723329544 minutes
Writing out/patterns/pid_4665_stop.parquet and out/patterns/pid_4665_segment.parquet
Processing stop time for all trips of pattern 4665
Originally 4127 trips
Trying to process 4078 trips for Pattern 4665 after filtering
Processed 4054 trips for Pattern 4665. There was 11 trip(s) with errors.
Time taken for stop times for 4665: 10.331929528713227 minutes
Writing out/patterns/pid_5610_stop.parquet and out/patterns/pid_5610_segment.parquet
Processing stop time for all trips of pattern 5610
Originally 3908 trips
Trying to process 3889 trips for Pattern 5610 after filtering
Processed 3889 trips for Pattern 5610. There was 0 trip(s) with errors.
Time taken for stop times for 5610: 1.8669652024904886 minutes
Writing out/patterns/pid_7120_stop.parquet and out/patterns/pid_7120_segment.parquet
Processing stop time for all trips of pattern 7120
Originally 22128 trips
Trying to process 21413 trips for Pattern 7120 after filtering
Processed 21292 trips for Pattern 7120. There was 56 trip(s) with errors.
Time taken for stop times for 7120: 13.618516647815705 minutes
Writing out/patterns/pid_6602_stop.parquet and out/patterns/pid_6602_segment.parquet
Processing stop time for all trips of pattern 6602
Originally 988 trips
Trying to process 984 trips for Pattern 6602 after filtering
Processed 982 trips for Pattern 6602. There was 1 trip(s) with errors.
Time taken for stop times for 6602: 0.4849066654841105 minutes
Do not have pattern 20424 available currently
Writing out/patterns/pid_8117_stop.parquet and out/patterns/pid_8117_segment.parquet
Processing stop time for all trips of pattern 8117
Originally 471 trips
Trying to process 468 trips for Pattern 8117 after filtering
Processed 467 trips for Pattern 8117. There was 0 trip(s) with errors.
Time taken for stop times for 8117: 0.18924369812011718 minutes
Writing out/patterns/pid_7031_stop.parquet and out/patterns/pid_7031_segment.parquet
Processing stop time for all trips of pattern 7031
Originally 1038 trips
Trying to process 1025 trips for Pattern 7031 after filtering
Processed 1022 trips for Pattern 7031. There was 1 trip(s) with errors.
Time taken for stop times for 7031: 0.4295474648475647 minutes
Writing out/patterns/pid_4587_stop.parquet and out/patterns/pid_4587_segment.parquet
Processing stop time for all trips of pattern 4587
Originally 2339 trips
Trying to process 2302 trips for Pattern 4587 after filtering
Processed 2297 trips for Pattern 4587. There was 0 trip(s) with errors.
Time taken for stop times for 4587: 0.6794902205467224 minutes
Writing out/patterns/pid_1039_stop.parquet and out/patterns/pid_1039_segment.parquet
Processing stop time for all trips of pattern 1039
Originally 23040 trips
Trying to process 22376 trips for Pattern 1039 after filtering
Processed 22265 trips for Pattern 1039. There was 54 trip(s) with errors.
Time taken for stop times for 1039: 14.052078568935395 minutes
Writing out/patterns/pid_1234_stop.parquet and out/patterns/pid_1234_segment.parquet
Processing stop time for all trips of pattern 1234
Originally 2304 trips
Trying to process 2283 trips for Pattern 1234 after filtering
Processed 2251 trips for Pattern 1234. There was 16 trip(s) with errors.
Time taken for stop times for 1234: 1.0525458455085754 minutes
Writing out/patterns/pid_4596_stop.parquet and out/patterns/pid_4596_segment.parquet
Processing stop time for all trips of pattern 4596
Originally 2199 trips
Trying to process 2170 trips for Pattern 4596 after filtering
Processed 2149 trips for Pattern 4596. There was 10 trip(s) with errors.
Time taken for stop times for 4596: 0.9871208667755127 minutes
Writing out/patterns/pid_7741_stop.parquet and out/patterns/pid_7741_segment.parquet
Processing stop time for all trips of pattern 7741
Originally 1587 trips
Trying to process 189 trips for Pattern 7741 after filtering
Processed -82 trips for Pattern 7741. There was 85 trip(s) with errors.
Time taken for stop times for 7741: 0.03268078565597534 minutes
Writing out/patterns/pid_1333_stop.parquet and out/patterns/pid_1333_segment.parquet
Processing stop time for all trips of pattern 1333
Originally 542 trips
Trying to process 534 trips for Pattern 1333 after filtering
Processed 530 trips for Pattern 1333. There was 2 trip(s) with errors.
Time taken for stop times for 1333: 0.23713728189468383 minutes
Writing out/patterns/pid_12431_stop.parquet and out/patterns/pid_12431_segment.parquet
Processing stop time for all trips of pattern 12431
Originally 821 trips
Trying to process 811 trips for Pattern 12431 after filtering
Processed 811 trips for Pattern 12431. There was 0 trip(s) with errors.
Time taken for stop times for 12431: 0.4328617175420125 minutes
Do not have pattern 8464 available currently
Writing out/patterns/pid_8143_stop.parquet and out/patterns/pid_8143_segment.parquet
Processing stop time for all trips of pattern 8143
Originally 2681 trips
Trying to process 2367 trips for Pattern 8143 after filtering
Processed 2355 trips for Pattern 8143. There was 2 trip(s) with errors.
Time taken for stop times for 8143: 0.7749426007270813 minutes
Do not have pattern 7448 available currently
Do not have pattern 19389 available currently
Writing out/patterns/pid_8413_stop.parquet and out/patterns/pid_8413_segment.parquet
Processing stop time for all trips of pattern 8413
Originally 487 trips
Trying to process 482 trips for Pattern 8413 after filtering
Processed 482 trips for Pattern 8413. There was 0 trip(s) with errors.
Time taken for stop times for 8413: 0.1685550014177958 minutes
Writing out/patterns/pid_8147_stop.parquet and out/patterns/pid_8147_segment.parquet
Processing stop time for all trips of pattern 8147
Originally 23488 trips
Trying to process 23210 trips for Pattern 8147 after filtering
Processed 23143 trips for Pattern 8147. There was 23 trip(s) with errors.
Time taken for stop times for 8147: 11.25942664941152 minutes
Do not have pattern 8182 available currently
Writing out/patterns/pid_14125_stop.parquet and out/patterns/pid_14125_segment.parquet
Processing stop time for all trips of pattern 14125
Originally 176 trips
Trying to process 170 trips for Pattern 14125 after filtering
Processed 170 trips for Pattern 14125. There was 0 trip(s) with errors.
Time taken for stop times for 14125: 0.062140766779581705 minutes
Writing out/patterns/pid_8180_stop.parquet and out/patterns/pid_8180_segment.parquet
Processing stop time for all trips of pattern 8180
Originally 14640 trips
Trying to process 14565 trips for Pattern 8180 after filtering
Processed 14544 trips for Pattern 8180. There was 10 trip(s) with errors.
Time taken for stop times for 8180: 6.089177950223287 minutes
Writing out/patterns/pid_8382_stop.parquet and out/patterns/pid_8382_segment.parquet
Processing stop time for all trips of pattern 8382
Originally 1982 trips
Trying to process 1960 trips for Pattern 8382 after filtering
Processed 1948 trips for Pattern 8382. There was 6 trip(s) with errors.
Time taken for stop times for 8382: 0.9467861652374268 minutes
Writing out/patterns/pid_10901_stop.parquet and out/patterns/pid_10901_segment.parquet
Processing stop time for all trips of pattern 10901
Originally 2767 trips
Trying to process 2747 trips for Pattern 10901 after filtering
Processed 2745 trips for Pattern 10901. There was 1 trip(s) with errors.
Time taken for stop times for 10901: 1.145363716284434 minutes
Writing out/patterns/pid_6661_stop.parquet and out/patterns/pid_6661_segment.parquet
Processing stop time for all trips of pattern 6661
Originally 1409 trips
Trying to process 1400 trips for Pattern 6661 after filtering
Processed 1397 trips for Pattern 6661. There was 1 trip(s) with errors.
Time taken for stop times for 6661: 0.6068361679712931 minutes
Writing out/patterns/pid_6872_stop.parquet and out/patterns/pid_6872_segment.parquet
Processing stop time for all trips of pattern 6872
Originally 187 trips
Trying to process 186 trips for Pattern 6872 after filtering
Processed 186 trips for Pattern 6872. There was 0 trip(s) with errors.
Time taken for stop times for 6872: 0.07287180026372274 minutes
Writing out/patterns/pid_5728_stop.parquet and out/patterns/pid_5728_segment.parquet
Processing stop time for all trips of pattern 5728
Originally 277 trips
Trying to process 269 trips for Pattern 5728 after filtering
Processed 269 trips for Pattern 5728. There was 0 trip(s) with errors.
Time taken for stop times for 5728: 0.06318551699320475 minutes
Writing out/patterns/pid_1662_stop.parquet and out/patterns/pid_1662_segment.parquet
Processing stop time for all trips of pattern 1662
Originally 5133 trips
Trying to process 5104 trips for Pattern 1662 after filtering
Processed 5099 trips for Pattern 1662. There was 2 trip(s) with errors.
Time taken for stop times for 1662: 2.5945886333783466 minutes
Writing out/patterns/pid_8388_stop.parquet and out/patterns/pid_8388_segment.parquet
Processing stop time for all trips of pattern 8388
Originally 51 trips
Trying to process 49 trips for Pattern 8388 after filtering
Processed 49 trips for Pattern 8388. There was 0 trip(s) with errors.
Time taken for stop times for 8388: 0.01803004741668701 minutes
Writing out/patterns/pid_6409_stop.parquet and out/patterns/pid_6409_segment.parquet
Processing stop time for all trips of pattern 6409
Originally 1208 trips
Trying to process 1200 trips for Pattern 6409 after filtering
Processed 1195 trips for Pattern 6409. There was 2 trip(s) with errors.
Time taken for stop times for 6409: 0.31112556854883827 minutes
Writing out/patterns/pid_8090_stop.parquet and out/patterns/pid_8090_segment.parquet
Processing stop time for all trips of pattern 8090
Originally 1333 trips
Trying to process 1319 trips for Pattern 8090 after filtering
Processed 1307 trips for Pattern 8090. There was 5 trip(s) with errors.
Time taken for stop times for 8090: 0.8241421302159627 minutes
Do not have pattern 2290 available currently
Writing out/patterns/pid_13248_stop.parquet and out/patterns/pid_13248_segment.parquet
Processing stop time for all trips of pattern 13248
Originally 1316 trips
Trying to process 1308 trips for Pattern 13248 after filtering
Processed 1308 trips for Pattern 13248. There was 0 trip(s) with errors.
Time taken for stop times for 13248: 0.5402795473734537 minutes
Writing out/patterns/pid_4599_stop.parquet and out/patterns/pid_4599_segment.parquet
Processing stop time for all trips of pattern 4599
Originally 2204 trips
Trying to process 2177 trips for Pattern 4599 after filtering
Processed 2145 trips for Pattern 4599. There was 14 trip(s) with errors.
Time taken for stop times for 4599: 1.616065267721812 minutes
Writing out/patterns/pid_23277_stop.parquet and out/patterns/pid_23277_segment.parquet
Processing stop time for all trips of pattern 23277
Originally 4 trips
Trying to process 4 trips for Pattern 23277 after filtering
Processed 4 trips for Pattern 23277. There was 0 trip(s) with errors.
Time taken for stop times for 23277: 0.001902767022450765 minutes
Writing out/patterns/pid_6582_stop.parquet and out/patterns/pid_6582_segment.parquet
Processing stop time for all trips of pattern 6582
Originally 20558 trips
Trying to process 19883 trips for Pattern 6582 after filtering
Processed 19721 trips for Pattern 6582. There was 59 trip(s) with errors.       Time taken for stop times for 6582: 24.85095160007477 minutes
Writing out/patterns/pid_19400_stop.parquet and out/patterns/pid_19400_segment.parquet
Processing stop time for all trips of pattern 19400
Originally 26 trips
Trying to process 16 trips for Pattern 19400 after filtering
Processed 14 trips for Pattern 19400. There was 1 trip(s) with errors.
Time taken for stop times for 19400: 0.006943317254384359 minutes
Writing out/patterns/pid_4370_stop.parquet and out/patterns/pid_4370_segment.parquet
Processing stop time for all trips of pattern 4370
Originally 1498 trips
Trying to process 1493 trips for Pattern 4370 after filtering
Processed 1489 trips for Pattern 4370. There was 2 trip(s) with errors.
Time taken for stop times for 4370: 0.5423362016677856 minutes
Writing out/patterns/pid_2291_stop.parquet and out/patterns/pid_2291_segment.parquet
Processing stop time for all trips of pattern 2291
Originally 24755 trips
Trying to process 24471 trips for Pattern 2291 after filtering
Processed 24411 trips for Pattern 2291. There was 22 trip(s) with errors.
Time taken for stop times for 2291: 16.18464318116506 minutes
Writing out/patterns/pid_14126_stop.parquet and out/patterns/pid_14126_segment.parquet
Processing stop time for all trips of pattern 14126
Originally 301 trips
Trying to process 298 trips for Pattern 14126 after filtering
Processed 298 trips for Pattern 14126. There was 0 trip(s) with errors.
Time taken for stop times for 14126: 0.12454752922058106 minutes
Writing out/patterns/pid_4111_stop.parquet and out/patterns/pid_4111_segment.parquet
Processing stop time for all trips of pattern 4111
Originally 1762 trips
Trying to process 1748 trips for Pattern 4111 after filtering
Processed 1744 trips for Pattern 4111. There was 1 trip(s) with errors.
Time taken for stop times for 4111: 1.2727587699890137 minutes
Writing out/patterns/pid_8127_stop.parquet and out/patterns/pid_8127_segment.parquet
Processing stop time for all trips of pattern 8127
Originally 273 trips
Trying to process 266 trips for Pattern 8127 after filtering
Processed 265 trips for Pattern 8127. There was 0 trip(s) with errors.
Time taken for stop times for 8127: 0.09622518221537273 minutes
Writing out/patterns/pid_8467_stop.parquet and out/patterns/pid_8467_segment.parquet
Processing stop time for all trips of pattern 8467
Originally 882 trips
Trying to process 879 trips for Pattern 8467 after filtering
Processed 879 trips for Pattern 8467. There was 0 trip(s) with errors.
Time taken for stop times for 8467: 0.417806084950765 minutes
Writing out/patterns/pid_4278_stop.parquet and out/patterns/pid_4278_segment.parquet
Processing stop time for all trips of pattern 4278
Originally 8 trips
Trying to process 8 trips for Pattern 4278 after filtering
Processed 8 trips for Pattern 4278. There was 0 trip(s) with errors.
Time taken for stop times for 4278: 0.005394482612609863 minutes
Writing out/patterns/pid_14111_stop.parquet and out/patterns/pid_14111_segment.parquet
Processing stop time for all trips of pattern 14111
Originally 10731 trips
Trying to process 10397 trips for Pattern 14111 after filtering
Processed 10349 trips for Pattern 14111. There was 21 trip(s) with errors.
Time taken for stop times for 14111: 4.14764983256658 minutes
Writing out/patterns/pid_4662_stop.parquet and out/patterns/pid_4662_segment.parquet
Processing stop time for all trips of pattern 4662
Originally 9883 trips
Trying to process 9769 trips for Pattern 4662 after filtering
Processed 9675 trips for Pattern 4662. There was 43 trip(s) with errors.
Time taken for stop times for 4662: 4.547948451836904 minutes
Writing out/patterns/pid_6775_stop.parquet and out/patterns/pid_6775_segment.parquet
Processing stop time for all trips of pattern 6775
Originally 3613 trips
Trying to process 3583 trips for Pattern 6775 after filtering
Processed 3576 trips for Pattern 6775. There was 3 trip(s) with errors.
Time taken for stop times for 6775: 2.0013094623883565 minutes
Writing out/patterns/pid_8149_stop.parquet and out/patterns/pid_8149_segment.parquet
Processing stop time for all trips of pattern 8149
Originally 178 trips
Trying to process 170 trips for Pattern 8149 after filtering
Processed 163 trips for Pattern 8149. There was 1 trip(s) with errors.
Time taken for stop times for 8149: 0.052416332562764484 minutes
Writing out/patterns/pid_7025_stop.parquet and out/patterns/pid_7025_segment.parquet
Processing stop time for all trips of pattern 7025
Originally 13645 trips
Trying to process 13531 trips for Pattern 7025 after filtering
Processed 13503 trips for Pattern 7025. There was 13 trip(s) with errors.
Time taken for stop times for 7025: 8.587400531768798 minutes
Writing out/patterns/pid_19392_stop.parquet and out/patterns/pid_19392_segment.parquet
Processing stop time for all trips of pattern 19392
Originally 126 trips
Trying to process 124 trips for Pattern 19392 after filtering
Processed 122 trips for Pattern 19392. There was 1 trip(s) with errors.
Time taken for stop times for 19392: 0.07877498467763265 minutes
Writing out/patterns/pid_1618_stop.parquet and out/patterns/pid_1618_segment.parquet
Processing stop time for all trips of pattern 1618
Originally 843 trips
Trying to process 831 trips for Pattern 1618 after filtering
Processed 827 trips for Pattern 1618. There was 2 trip(s) with errors.
Time taken for stop times for 1618: 0.7475748340288798 minutes
Writing out/patterns/pid_839_stop.parquet and out/patterns/pid_839_segment.parquet
Processing stop time for all trips of pattern 839
Originally 2054 trips
Trying to process 2017 trips for Pattern 839 after filtering
Processed 1984 trips for Pattern 839. There was 15 trip(s) with errors.
Time taken for stop times for 839: 0.9240849177042644 minutes
Writing out/patterns/pid_3936_stop.parquet and out/patterns/pid_3936_segment.parquet
Processing stop time for all trips of pattern 3936
Originally 31582 trips
Trying to process 30623 trips for Pattern 3936 after filtering
Processed 30069 trips for Pattern 3936. There was 201 trip(s) with errors.
Time taken for stop times for 3936: 20.243141583601634 minutes
Writing out/patterns/pid_7006_stop.parquet and out/patterns/pid_7006_segment.parquet
Processing stop time for all trips of pattern 7006
Originally 1170 trips
Trying to process 1165 trips for Pattern 7006 after filtering
Processed 1101 trips for Pattern 7006. There was 30 trip(s) with errors.
Time taken for stop times for 7006: 0.49486271540323895 minutes
Writing out/patterns/pid_10882_stop.parquet and out/patterns/pid_10882_segment.parquet
Processing stop time for all trips of pattern 10882
Originally 115 trips
Trying to process 112 trips for Pattern 10882 after filtering
Processed 112 trips for Pattern 10882. There was 0 trip(s) with errors.
Time taken for stop times for 10882: 0.07078200181325277 minutes
Writing out/patterns/pid_5725_stop.parquet and out/patterns/pid_5725_segment.parquet
Processing stop time for all trips of pattern 5725
Originally 609 trips
Trying to process 592 trips for Pattern 5725 after filtering
Processed 583 trips for Pattern 5725. There was 4 trip(s) with errors.
Time taken for stop times for 5725: 0.3954617977142334 minutes
Writing out/patterns/pid_7533_stop.parquet and out/patterns/pid_7533_segment.parquet
Processing stop time for all trips of pattern 7533
Originally 3 trips
Trying to process 3 trips for Pattern 7533 after filtering
Processed 3 trips for Pattern 7533. There was 0 trip(s) with errors.
Time taken for stop times for 7533: 0.002207664648691813 minutes
Writing out/patterns/pid_4505_stop.parquet and out/patterns/pid_4505_segment.parquet
Processing stop time for all trips of pattern 4505
Originally 13606 trips
Trying to process 13477 trips for Pattern 4505 after filtering
Processed 13454 trips for Pattern 4505. There was 8 trip(s) with errors.
Time taken for stop times for 4505: 7.618599450588226 minutes
Writing out/patterns/pid_8102_stop.parquet and out/patterns/pid_8102_segment.parquet
Processing stop time for all trips of pattern 8102
Originally 25344 trips
Trying to process 24786 trips for Pattern 8102 after filtering
Processed 24657 trips for Pattern 8102. There was 58 trip(s) with errors.
Time taken for stop times for 8102: 18.150223902861278 minutes
Writing out/patterns/pid_894_stop.parquet and out/patterns/pid_894_segment.parquet
Processing stop time for all trips of pattern 894
Originally 1514 trips
Trying to process 1510 trips for Pattern 894 after filtering
Processed 1510 trips for Pattern 894. There was 0 trip(s) with errors.
Time taken for stop times for 894: 0.6078261017799378 minutes
Writing out/patterns/pid_7735_stop.parquet and out/patterns/pid_7735_segment.parquet
Processing stop time for all trips of pattern 7735
Originally 193 trips
Trying to process 192 trips for Pattern 7735 after filtering
Processed 192 trips for Pattern 7735. There was 0 trip(s) with errors.
Time taken for stop times for 7735: 0.07393603324890137 minutes
Do not have pattern 11674 available currently
Writing out/patterns/pid_3883_stop.parquet and out/patterns/pid_3883_segment.parquet
Processing stop time for all trips of pattern 3883
Originally 151 trips
Trying to process 151 trips for Pattern 3883 after filtering
Processed 149 trips for Pattern 3883. There was 1 trip(s) with errors.
Time taken for stop times for 3883: 0.051758249600728355 minutes
Writing out/patterns/pid_8695_stop.parquet and out/patterns/pid_8695_segment.parquet
Processing stop time for all trips of pattern 8695
Originally 1446 trips
Trying to process 755 trips for Pattern 8695 after filtering
Processed 736 trips for Pattern 8695. There was 5 trip(s) with errors.
Time taken for stop times for 8695: 0.17275030215581258 minutes
Writing out/patterns/pid_10811_stop.parquet and out/patterns/pid_10811_segment.parquet
Processing stop time for all trips of pattern 10811
Originally 18419 trips
Trying to process 18299 trips for Pattern 10811 after filtering
Processed 18285 trips for Pattern 10811. There was 5 trip(s) with errors.
Time taken for stop times for 10811: 26.20250969727834 minutes
Writing out/patterns/pid_16551_stop.parquet and out/patterns/pid_16551_segment.parquet
Processing stop time for all trips of pattern 16551
Originally 35037 trips
Trying to process 33388 trips for Pattern 16551 after filtering
/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/Interpolation.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trip_df.loc[:, "data_time"] = pd.to_datetime(trip_df.data_time)
Processed 32470 trips for Pattern 16551. There was 427 trip(s) with errors.
Time taken for stop times for 16551: 116.79713074763616 minutes
Do not have pattern 4602 available currently
Do not have pattern 21482 available currently
Writing out/patterns/pid_1600_stop.parquet and out/patterns/pid_1600_segment.parquet
Processing stop time for all trips of pattern 1600
Originally 22347 trips
Trying to process 22089 trips for Pattern 1600 after filtering
Processed 22063 trips for Pattern 1600. There was 11 trip(s) with errors.
Time taken for stop times for 1600: 11.687487383683523 minutes
Writing out/patterns/pid_7015_stop.parquet and out/patterns/pid_7015_segment.parquet
Processing stop time for all trips of pattern 7015
Originally 18017 trips
Trying to process 17936 trips for Pattern 7015 after filtering
Processed 17917 trips for Pattern 7015. There was 6 trip(s) with errors.
Time taken for stop times for 7015: 12.738005900382996 minutes
Writing out/patterns/pid_3935_stop.parquet and out/patterns/pid_3935_segment.parquet
Processing stop time for all trips of pattern 3935
Originally 117 trips
Trying to process 117 trips for Pattern 3935 after filtering
Processed 117 trips for Pattern 3935. There was 0 trip(s) with errors.
Time taken for stop times for 3935: 0.0518484632174174 minutes
Writing out/patterns/pid_7724_stop.parquet and out/patterns/pid_7724_segment.parquet
Processing stop time for all trips of pattern 7724
Originally 1 trips
Trying to process 1 trips for Pattern 7724 after filtering
Processed 1 trips for Pattern 7724. There was 0 trip(s) with errors.
Time taken for stop times for 7724: 0.000980222225189209 minutes
Writing out/patterns/pid_7557_stop.parquet and out/patterns/pid_7557_segment.parquet
Processing stop time for all trips of pattern 7557
Originally 415 trips
Trying to process 411 trips for Pattern 7557 after filtering
Processed 411 trips for Pattern 7557. There was 0 trip(s) with errors.
Time taken for stop times for 7557: 0.4223749995231628 minutes
Writing out/patterns/pid_746_stop.parquet and out/patterns/pid_746_segment.parquet
Processing stop time for all trips of pattern 746
Originally 141 trips
Trying to process 140 trips for Pattern 746 after filtering
Processed 140 trips for Pattern 746. There was 0 trip(s) with errors.
Time taken for stop times for 746: 0.03921275536219279 minutes
Writing out/patterns/pid_8689_stop.parquet and out/patterns/pid_8689_segment.parquet
Processing stop time for all trips of pattern 8689
Originally 173 trips
Trying to process 167 trips for Pattern 8689 after filtering
Processed 165 trips for Pattern 8689. There was 1 trip(s) with errors.
Time taken for stop times for 8689: 0.04676596721013387 minutes
Writing out/patterns/pid_4725_stop.parquet and out/patterns/pid_4725_segment.parquet
Processing stop time for all trips of pattern 4725
Originally 283 trips
Trying to process 280 trips for Pattern 4725 after filtering
Processed 278 trips for Pattern 4725. There was 1 trip(s) with errors.
Time taken for stop times for 4725: 0.07740868330001831 minutes
Do not have pattern 2271 available currently
Writing out/patterns/pid_7173_stop.parquet and out/patterns/pid_7173_segment.parquet
Processing stop time for all trips of pattern 7173
Originally 485 trips
Trying to process 482 trips for Pattern 7173 after filtering
Processed 480 trips for Pattern 7173. There was 1 trip(s) with errors.
Time taken for stop times for 7173: 0.19520228306452433 minutes
Writing out/patterns/pid_9345_stop.parquet and out/patterns/pid_9345_segment.parquet
Processing stop time for all trips of pattern 9345
Originally 1183 trips
Trying to process 1181 trips for Pattern 9345 after filtering
Processed 1179 trips for Pattern 9345. There was 1 trip(s) with errors.
Time taken for stop times for 9345: 0.5094561139742534 minutes
Writing out/patterns/pid_7018_stop.parquet and out/patterns/pid_7018_segment.parquet
Processing stop time for all trips of pattern 7018
Originally 245 trips
Trying to process 244 trips for Pattern 7018 after filtering
Processed 244 trips for Pattern 7018. There was 0 trip(s) with errors.
Time taken for stop times for 7018: 0.0747112512588501 minutes
Writing out/patterns/pid_7385_stop.parquet and out/patterns/pid_7385_segment.parquet
Processing stop time for all trips of pattern 7385
Originally 3902 trips
Trying to process 3888 trips for Pattern 7385 after filtering
Processed 3876 trips for Pattern 7385. There was 6 trip(s) with errors.
Time taken for stop times for 7385: 1.5527477343877156 minutes
Do not have pattern 3761 available currently
Do not have pattern 8688 available currently
Writing out/patterns/pid_7388_stop.parquet and out/patterns/pid_7388_segment.parquet
Processing stop time for all trips of pattern 7388
Originally 288 trips
Trying to process 286 trips for Pattern 7388 after filtering
Processed 286 trips for Pattern 7388. There was 0 trip(s) with errors.
Time taken for stop times for 7388: 0.12012472947438559 minutes
Writing out/patterns/pid_8122_stop.parquet and out/patterns/pid_8122_segment.parquet
Processing stop time for all trips of pattern 8122
Originally 1614 trips
Trying to process 1591 trips for Pattern 8122 after filtering
Processed 1537 trips for Pattern 8122. There was 26 trip(s) with errors.
Time taken for stop times for 8122: 0.8021863142649333 minutes
Writing out/patterns/pid_2062_stop.parquet and out/patterns/pid_2062_segment.parquet
Processing stop time for all trips of pattern 2062
Originally 1057 trips
Trying to process 1050 trips for Pattern 2062 after filtering
Processed 1049 trips for Pattern 2062. There was 0 trip(s) with errors.
Time taken for stop times for 2062: 0.36769094864527385 minutes
Writing out/patterns/pid_755_stop.parquet and out/patterns/pid_755_segment.parquet
Processing stop time for all trips of pattern 755
Originally 811 trips
Trying to process 789 trips for Pattern 755 after filtering
Processed 777 trips for Pattern 755. There was 5 trip(s) with errors.
Time taken for stop times for 755: 0.25070343414942425 minutes
Writing out/patterns/pid_1348_stop.parquet and out/patterns/pid_1348_segment.parquet
Processing stop time for all trips of pattern 1348
Originally 46 trips
Trying to process 46 trips for Pattern 1348 after filtering
Processed 46 trips for Pattern 1348. There was 0 trip(s) with errors.
Time taken for stop times for 1348: 0.019845998287200926 minutes
Writing out/patterns/pid_20422_stop.parquet and out/patterns/pid_20422_segment.parquet
Processing stop time for all trips of pattern 20422
Originally 34 trips
Trying to process 34 trips for Pattern 20422 after filtering
Processed 34 trips for Pattern 20422. There was 0 trip(s) with errors.
Time taken for stop times for 20422: 0.016867701212565103 minutes
Writing out/patterns/pid_5070_stop.parquet and out/patterns/pid_5070_segment.parquet
Processing stop time for all trips of pattern 5070
Originally 14870 trips
Trying to process 14724 trips for Pattern 5070 after filtering
Processed 14684 trips for Pattern 5070. There was 18 trip(s) with errors.
Time taken for stop times for 5070: 7.6762090682983395 minutes
Do not have pattern 6660 available currently
Writing out/patterns/pid_7023_stop.parquet and out/patterns/pid_7023_segment.parquet
Processing stop time for all trips of pattern 7023
Originally 14186 trips
Trying to process 14117 trips for Pattern 7023 after filtering
Processed 14113 trips for Pattern 7023. There was 2 trip(s) with errors.
Time taken for stop times for 7023: 8.010383101304372 minutes
Writing out/patterns/pid_4344_stop.parquet and out/patterns/pid_4344_segment.parquet
Processing stop time for all trips of pattern 4344
Originally 1395 trips
Trying to process 1389 trips for Pattern 4344 after filtering
Processed 1389 trips for Pattern 4344. There was 0 trip(s) with errors.
Time taken for stop times for 4344: 0.5787124514579773 minutes
Writing out/patterns/pid_1110_stop.parquet and out/patterns/pid_1110_segment.parquet
Processing stop time for all trips of pattern 1110
Originally 196 trips
Trying to process 189 trips for Pattern 1110 after filtering
Processed 189 trips for Pattern 1110. There was 0 trip(s) with errors.
Time taken for stop times for 1110: 0.06666537125905354 minutes
Writing out/patterns/pid_13251_stop.parquet and out/patterns/pid_13251_segment.parquet
Processing stop time for all trips of pattern 13251
Originally 154 trips
Trying to process 140 trips for Pattern 13251 after filtering
Processed 140 trips for Pattern 13251. There was 0 trip(s) with errors.
Time taken for stop times for 13251: 0.03858263492584228 minutes
Writing out/patterns/pid_19384_stop.parquet and out/patterns/pid_19384_segment.parquet
Processing stop time for all trips of pattern 19384
Originally 1802 trips
Trying to process 1793 trips for Pattern 19384 after filtering
Processed 1787 trips for Pattern 19384. There was 3 trip(s) with errors.
Time taken for stop times for 19384: 0.9879273811976115 minutes
Writing out/patterns/pid_7024_stop.parquet and out/patterns/pid_7024_segment.parquet
Processing stop time for all trips of pattern 7024
Originally 1289 trips
Trying to process 1266 trips for Pattern 7024 after filtering
Processed 1260 trips for Pattern 7024. There was 3 trip(s) with errors.
Time taken for stop times for 7024: 0.4299445509910583 minutes
Do not have pattern 1478 available currently
Writing out/patterns/pid_1627_stop.parquet and out/patterns/pid_1627_segment.parquet
Processing stop time for all trips of pattern 1627
Originally 3895 trips
Trying to process 3805 trips for Pattern 1627 after filtering
Processed 3800 trips for Pattern 1627. There was 2 trip(s) with errors.
Time taken for stop times for 1627: 1.0676488359769185 minutes
Writing out/patterns/pid_2292_stop.parquet and out/patterns/pid_2292_segment.parquet
Processing stop time for all trips of pattern 2292
Originally 1355 trips
Trying to process 881 trips for Pattern 2292 after filtering
Processed 863 trips for Pattern 2292. There was 8 trip(s) with errors.
Time taken for stop times for 2292: 0.2476232131322225 minutes
Writing out/patterns/pid_8095_stop.parquet and out/patterns/pid_8095_segment.parquet
Processing stop time for all trips of pattern 8095
Originally 2414 trips
Trying to process 2367 trips for Pattern 8095 after filtering
Processed 2353 trips for Pattern 8095. There was 7 trip(s) with errors.
Time taken for stop times for 8095: 0.9386185487111409 minutes
Writing out/patterns/pid_899_stop.parquet and out/patterns/pid_899_segment.parquet
Processing stop time for all trips of pattern 899
Originally 20330 trips
Trying to process 19992 trips for Pattern 899 after filtering
Processed 19926 trips for Pattern 899. There was 31 trip(s) with errors.
Time taken for stop times for 899: 8.93233288526535 minutes
Do not have pattern 20425 available currently
Writing out/patterns/pid_4275_stop.parquet and out/patterns/pid_4275_segment.parquet
Processing stop time for all trips of pattern 4275
Originally 23389 trips
Trying to process 22742 trips for Pattern 4275 after filtering
Processed 22481 trips for Pattern 4275. There was 99 trip(s) with errors.
Time taken for stop times for 4275: 58.11360244750976 minutes
Writing out/patterns/pid_8097_stop.parquet and out/patterns/pid_8097_segment.parquet
Processing stop time for all trips of pattern 8097
Originally 714 trips
Trying to process 711 trips for Pattern 8097 after filtering
Processed 711 trips for Pattern 8097. There was 0 trip(s) with errors.
Time taken for stop times for 8097: 0.247369380791982 minutes
Writing out/patterns/pid_4110_stop.parquet and out/patterns/pid_4110_segment.parquet
Processing stop time for all trips of pattern 4110
Originally 60 trips
Trying to process 60 trips for Pattern 4110 after filtering
Processed 60 trips for Pattern 4110. There was 0 trip(s) with errors.
Time taken for stop times for 4110: 0.025970200697580974 minutes
Writing out/patterns/pid_1499_stop.parquet and out/patterns/pid_1499_segment.parquet
Processing stop time for all trips of pattern 1499
Originally 3685 trips
Trying to process 2928 trips for Pattern 1499 after filtering
Processed 2926 trips for Pattern 1499. There was 1 trip(s) with errors.
Time taken for stop times for 1499: 17.525383309523264 minutes
Writing out/patterns/pid_5687_stop.parquet and out/patterns/pid_5687_segment.parquet
Processing stop time for all trips of pattern 5687
Originally 296 trips
Trying to process 291 trips for Pattern 5687 after filtering
Processed 291 trips for Pattern 5687. There was 0 trip(s) with errors.
Time taken for stop times for 5687: 0.11004033088684081 minutes
Writing out/patterns/pid_7030_stop.parquet and out/patterns/pid_7030_segment.parquet
Processing stop time for all trips of pattern 7030
Originally 2161 trips
Trying to process 2150 trips for Pattern 7030 after filtering
Processed 2146 trips for Pattern 7030. There was 2 trip(s) with errors.
Time taken for stop times for 7030: 1.0557064493497212 minutes
Writing out/patterns/pid_1654_stop.parquet and out/patterns/pid_1654_segment.parquet
Processing stop time for all trips of pattern 1654
Originally 1166 trips
Trying to process 1106 trips for Pattern 1654 after filtering
Processed 1102 trips for Pattern 1654. There was 1 trip(s) with errors.
Time taken for stop times for 1654: 0.31991268396377565 minutes
Writing out/patterns/pid_1728_stop.parquet and out/patterns/pid_1728_segment.parquet
Processing stop time for all trips of pattern 1728
Originally 14473 trips
Trying to process 14222 trips for Pattern 1728 after filtering
Processed 14167 trips for Pattern 1728. There was 19 trip(s) with errors.
Time taken for stop times for 1728: 34.59363136291504 minutes
Writing out/patterns/pid_95_stop.parquet and out/patterns/pid_95_segment.parquet
Processing stop time for all trips of pattern 95
Originally 24849 trips
Trying to process 24361 trips for Pattern 95 after filtering
Processed 24096 trips for Pattern 95. There was 120 trip(s) with errors.
Time taken for stop times for 95: 65.3607160170873 minutes
Writing out/patterns/pid_7561_stop.parquet and out/patterns/pid_7561_segment.parquet
Processing stop time for all trips of pattern 7561
Originally 3522 trips
Trying to process 3505 trips for Pattern 7561 after filtering
Processed 3502 trips for Pattern 7561. There was 1 trip(s) with errors.
Time taken for stop times for 7561: 2.0651553829511005 minutes
Writing out/patterns/pid_1298_stop.parquet and out/patterns/pid_1298_segment.parquet
Processing stop time for all trips of pattern 1298
Originally 445 trips
Trying to process 436 trips for Pattern 1298 after filtering
Processed 433 trips for Pattern 1298. There was 1 trip(s) with errors.
Time taken for stop times for 1298: 0.14140309890111288 minutes
Writing out/patterns/pid_4577_stop.parquet and out/patterns/pid_4577_segment.parquet
Processing stop time for all trips of pattern 4577
Originally 171 trips
Trying to process 77 trips for Pattern 4577 after filtering
Processed 76 trips for Pattern 4577. There was 0 trip(s) with errors.
Time taken for stop times for 4577: 0.02325664758682251 minutes
Writing out/patterns/pid_22568_stop.parquet and out/patterns/pid_22568_segment.parquet
Processing stop time for all trips of pattern 22568
Originally 213 trips
Trying to process 211 trips for Pattern 22568 after filtering
Processed 211 trips for Pattern 22568. There was 0 trip(s) with errors.
Time taken for stop times for 22568: 0.1036781350771586 minutes
Writing out/patterns/pid_9375_stop.parquet and out/patterns/pid_9375_segment.parquet
Processing stop time for all trips of pattern 9375
Originally 7406 trips
Trying to process 7337 trips for Pattern 9375 after filtering
Processed 7313 trips for Pattern 9375. There was 12 trip(s) with errors.
Time taken for stop times for 9375: 21.30554107030233 minutes
Writing out/patterns/pid_7452_stop.parquet and out/patterns/pid_7452_segment.parquet
Processing stop time for all trips of pattern 7452
Originally 881 trips
Trying to process 876 trips for Pattern 7452 after filtering
Processed 872 trips for Pattern 7452. There was 2 trip(s) with errors.
Time taken for stop times for 7452: 0.29110643863677976 minutes
Writing out/patterns/pid_9343_stop.parquet and out/patterns/pid_9343_segment.parquet
Processing stop time for all trips of pattern 9343
Originally 783 trips
Trying to process 778 trips for Pattern 9343 after filtering
Processed 776 trips for Pattern 9343. There was 1 trip(s) with errors.
Time taken for stop times for 9343: 0.26760676701863606 minutes
Writing out/patterns/pid_731_stop.parquet and out/patterns/pid_731_segment.parquet
Processing stop time for all trips of pattern 731
Originally 8631 trips
Trying to process 8518 trips for Pattern 731 after filtering
Processed 8459 trips for Pattern 731. There was 27 trip(s) with errors.
Time taken for stop times for 731: 2.8817176540692646 minutes
Writing out/patterns/pid_940_stop.parquet and out/patterns/pid_940_segment.parquet
Processing stop time for all trips of pattern 940
Originally 1860 trips
Trying to process 1649 trips for Pattern 940 after filtering
Processed 1648 trips for Pattern 940. There was 0 trip(s) with errors.
Time taken for stop times for 940: 0.4565221269925435 minutes
Writing out/patterns/pid_4080_stop.parquet and out/patterns/pid_4080_segment.parquet
Processing stop time for all trips of pattern 4080
Originally 2640 trips
Trying to process 2622 trips for Pattern 4080 after filtering
Processed 2616 trips for Pattern 4080. There was 2 trip(s) with errors.
Time taken for stop times for 4080: 6.106329115231832 minutes
Writing out/patterns/pid_19398_stop.parquet and out/patterns/pid_19398_segment.parquet
Processing stop time for all trips of pattern 19398
Originally 2408 trips
Trying to process 2397 trips for Pattern 19398 after filtering
Processed 2393 trips for Pattern 19398. There was 2 trip(s) with errors.
Time taken for stop times for 19398: 0.9358102202415466 minutes
Writing out/patterns/pid_8105_stop.parquet and out/patterns/pid_8105_segment.parquet
Processing stop time for all trips of pattern 8105
Originally 1083 trips
Trying to process 1078 trips for Pattern 8105 after filtering
Processed 1076 trips for Pattern 8105. There was 1 trip(s) with errors.
Time taken for stop times for 8105: 0.39510581890741986 minutes
Do not have pattern 8139 available currently
Writing out/patterns/pid_9660_stop.parquet and out/patterns/pid_9660_segment.parquet
Processing stop time for all trips of pattern 9660
Originally 1885 trips
Trying to process 1868 trips for Pattern 9660 after filtering
Processed 1863 trips for Pattern 9660. There was 2 trip(s) with errors.
Time taken for stop times for 9660: 4.607559072971344 minutes
Writing out/patterns/pid_7447_stop.parquet and out/patterns/pid_7447_segment.parquet
Processing stop time for all trips of pattern 7447
Originally 1545 trips
Trying to process 1529 trips for Pattern 7447 after filtering
Processed 1518 trips for Pattern 7447. There was 4 trip(s) with errors.
Time taken for stop times for 7447: 0.6638002514839172 minutes
Writing out/patterns/pid_8461_stop.parquet and out/patterns/pid_8461_segment.parquet
Processing stop time for all trips of pattern 8461
Originally 251 trips
Trying to process 251 trips for Pattern 8461 after filtering
Processed 251 trips for Pattern 8461. There was 0 trip(s) with errors.
Time taken for stop times for 8461: 0.09284678300221762 minutes
Writing out/patterns/pid_1500_stop.parquet and out/patterns/pid_1500_segment.parquet
Processing stop time for all trips of pattern 1500
Originally 5331 trips
Trying to process 4581 trips for Pattern 1500 after filtering
Processed 4559 trips for Pattern 1500. There was 6 trip(s) with errors.
Time taken for stop times for 1500: 1.1667081316312153 minutes
Writing out/patterns/pid_6455_stop.parquet and out/patterns/pid_6455_segment.parquet
Processing stop time for all trips of pattern 6455
Originally 162 trips
Trying to process 162 trips for Pattern 6455 after filtering
Processed 162 trips for Pattern 6455. There was 0 trip(s) with errors.
Time taken for stop times for 6455: 0.05344861348470052 minutes
Writing out/patterns/pid_1351_stop.parquet and out/patterns/pid_1351_segment.parquet
Processing stop time for all trips of pattern 1351
Originally 475 trips
Trying to process 446 trips for Pattern 1351 after filtering
Processed 444 trips for Pattern 1351. There was 1 trip(s) with errors.
Time taken for stop times for 1351: 0.2723874489466349 minutes
Writing out/patterns/pid_1345_stop.parquet and out/patterns/pid_1345_segment.parquet
Processing stop time for all trips of pattern 1345
Originally 25351 trips
Trying to process 24988 trips for Pattern 1345 after filtering
Processed 24774 trips for Pattern 1345. There was 97 trip(s) with errors.
Time taken for stop times for 1345: 18.51277360121409 minutes
Writing out/patterns/pid_3258_stop.parquet and out/patterns/pid_3258_segment.parquet
Processing stop time for all trips of pattern 3258
Originally 81 trips
Trying to process 77 trips for Pattern 3258 after filtering
Processed 77 trips for Pattern 3258. There was 0 trip(s) with errors.
Time taken for stop times for 3258: 0.033673163255055746 minutes
Writing out/patterns/pid_6662_stop.parquet and out/patterns/pid_6662_segment.parquet
Processing stop time for all trips of pattern 6662
Originally 44635 trips
Trying to process 43843 trips for Pattern 6662 after filtering
Processed 43606 trips for Pattern 6662. There was 107 trip(s) with errors.
Time taken for stop times for 6662: 24.63432110150655 minutes
Writing out/patterns/pid_6413_stop.parquet and out/patterns/pid_6413_segment.parquet
Processing stop time for all trips of pattern 6413
Originally 6886 trips
Trying to process 6725 trips for Pattern 6413 after filtering
Processed 6681 trips for Pattern 6413. There was 21 trip(s) with errors.
Time taken for stop times for 6413: 4.865827480951945 minutes
Writing out/patterns/pid_7096_stop.parquet and out/patterns/pid_7096_segment.parquet
Processing stop time for all trips of pattern 7096
Originally 319 trips
Trying to process 317 trips for Pattern 7096 after filtering
Processed 315 trips for Pattern 7096. There was 1 trip(s) with errors.
Time taken for stop times for 7096: 0.18759047985076904 minutes
Writing out/patterns/pid_5422_stop.parquet and out/patterns/pid_5422_segment.parquet
Processing stop time for all trips of pattern 5422
Originally 680 trips
Trying to process 673 trips for Pattern 5422 after filtering
Processed 673 trips for Pattern 5422. There was 0 trip(s) with errors.
Time taken for stop times for 5422: 0.27597814400990806 minutes
Writing out/patterns/pid_7444_stop.parquet and out/patterns/pid_7444_segment.parquet
Processing stop time for all trips of pattern 7444
Originally 4681 trips
Trying to process 4627 trips for Pattern 7444 after filtering
Processed 4612 trips for Pattern 7444. There was 6 trip(s) with errors.
Time taken for stop times for 7444: 2.1540065169334413 minutes
Writing out/patterns/pid_4672_stop.parquet and out/patterns/pid_4672_segment.parquet
Processing stop time for all trips of pattern 4672
Originally 14671 trips
Trying to process 14487 trips for Pattern 4672 after filtering
Processed 14445 trips for Pattern 4672. There was 18 trip(s) with errors.
Time taken for stop times for 4672: 5.913068115711212 minutes
Writing out/patterns/pid_2862_stop.parquet and out/patterns/pid_2862_segment.parquet
Processing stop time for all trips of pattern 2862
Originally 5569 trips
Trying to process 5536 trips for Pattern 2862 after filtering
Processed 5522 trips for Pattern 2862. There was 7 trip(s) with errors.
Time taken for stop times for 2862: 2.0952012181282043 minutes
Writing out/patterns/pid_7908_stop.parquet and out/patterns/pid_7908_segment.parquet
Processing stop time for all trips of pattern 7908
Originally 1765 trips
Trying to process 1750 trips for Pattern 7908 after filtering
Processed 1750 trips for Pattern 7908. There was 0 trip(s) with errors.
Time taken for stop times for 7908: 1.106246252854665 minutes
Writing out/patterns/pid_1718_stop.parquet and out/patterns/pid_1718_segment.parquet
Processing stop time for all trips of pattern 1718
Originally 845 trips
Trying to process 830 trips for Pattern 1718 after filtering
Processed 827 trips for Pattern 1718. There was 0 trip(s) with errors.
Time taken for stop times for 1718: 0.3372389356295268 minutes
Writing out/patterns/pid_7910_stop.parquet and out/patterns/pid_7910_segment.parquet
Processing stop time for all trips of pattern 7910
Originally 2954 trips
Trying to process 2943 trips for Pattern 7910 after filtering
Processed 2936 trips for Pattern 7910. There was 3 trip(s) with errors.
Time taken for stop times for 7910: 1.782894766330719 minutes
Writing out/patterns/pid_6776_stop.parquet and out/patterns/pid_6776_segment.parquet
Processing stop time for all trips of pattern 6776
Originally 1592 trips
Trying to process 1587 trips for Pattern 6776 after filtering
Processed 1587 trips for Pattern 6776. There was 0 trip(s) with errors.
Time taken for stop times for 6776: 0.9991088271141052 minutes
Writing out/patterns/pid_4096_stop.parquet and out/patterns/pid_4096_segment.parquet
Processing stop time for all trips of pattern 4096
Originally 12621 trips
Trying to process 12500 trips for Pattern 4096 after filtering
Processed 12485 trips for Pattern 4096. There was 4 trip(s) with errors.
Time taken for stop times for 4096: 9.162987716992696 minutes
Writing out/patterns/pid_7098_stop.parquet and out/patterns/pid_7098_segment.parquet
Processing stop time for all trips of pattern 7098
Originally 289 trips
Trying to process 285 trips for Pattern 7098 after filtering
Processed 285 trips for Pattern 7098. There was 0 trip(s) with errors.
Time taken for stop times for 7098: 0.10600295066833496 minutes
Writing out/patterns/pid_8110_stop.parquet and out/patterns/pid_8110_segment.parquet
Processing stop time for all trips of pattern 8110
Originally 9684 trips
Trying to process 9533 trips for Pattern 8110 after filtering
/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/Interpolation.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trip_df.loc[:, "data_time"] = pd.to_datetime(trip_df.data_time)
Processed 9485 trips for Pattern 8110. There was 20 trip(s) with errors.
Time taken for stop times for 8110: 4.681492952505748 minutes
Writing out/patterns/pid_4729_stop.parquet and out/patterns/pid_4729_segment.parquet
Processing stop time for all trips of pattern 4729
Originally 3495 trips
Trying to process 3387 trips for Pattern 4729 after filtering
Processed 3359 trips for Pattern 4729. There was 12 trip(s) with errors.
Time taken for stop times for 4729: 1.7150485873222352 minutes
Do not have pattern 7446 available currently
Writing out/patterns/pid_5666_stop.parquet and out/patterns/pid_5666_segment.parquet
Processing stop time for all trips of pattern 5666
Originally 3305 trips
Trying to process 3260 trips for Pattern 5666 after filtering
Processed 3239 trips for Pattern 5666. There was 10 trip(s) with errors.
Time taken for stop times for 5666: 1.7067148129145304 minutes
Writing out/patterns/pid_4112_stop.parquet and out/patterns/pid_4112_segment.parquet
Processing stop time for all trips of pattern 4112
Originally 1601 trips
Trying to process 1566 trips for Pattern 4112 after filtering
Processed 1545 trips for Pattern 4112. There was 9 trip(s) with errors.
Time taken for stop times for 4112: 1.0650341312090557 minutes
Writing out/patterns/pid_7873_stop.parquet and out/patterns/pid_7873_segment.parquet
Processing stop time for all trips of pattern 7873
Originally 9338 trips
Trying to process 9222 trips for Pattern 7873 after filtering
Processed 9181 trips for Pattern 7873. There was 18 trip(s) with errors.
Time taken for stop times for 7873: 4.744274437427521 minutes
Writing out/patterns/pid_2642_stop.parquet and out/patterns/pid_2642_segment.parquet
Processing stop time for all trips of pattern 2642
Originally 1797 trips
Trying to process 1773 trips for Pattern 2642 after filtering
Processed 1769 trips for Pattern 2642. There was 2 trip(s) with errors.
Time taken for stop times for 2642: 1.0349720199902852 minutes
Writing out/patterns/pid_5571_stop.parquet and out/patterns/pid_5571_segment.parquet
Processing stop time for all trips of pattern 5571
Originally 18544 trips
Trying to process 17950 trips for Pattern 5571 after filtering
Processed 17913 trips for Pattern 5571. There was 16 trip(s) with errors.
Time taken for stop times for 5571: 11.361329432328542 minutes
Writing out/patterns/pid_7457_stop.parquet and out/patterns/pid_7457_segment.parquet
Processing stop time for all trips of pattern 7457
Originally 811 trips
Trying to process 803 trips for Pattern 7457 after filtering
Processed 801 trips for Pattern 7457. There was 1 trip(s) with errors.
Time taken for stop times for 7457: 0.30532525380452474 minutes
Writing out/patterns/pid_7119_stop.parquet and out/patterns/pid_7119_segment.parquet
Processing stop time for all trips of pattern 7119
Originally 134 trips
Trying to process 133 trips for Pattern 7119 after filtering
Processed 131 trips for Pattern 7119. There was 0 trip(s) with errors.
Time taken for stop times for 7119: 0.07880716721216838 minutes
Writing out/patterns/pid_701_stop.parquet and out/patterns/pid_701_segment.parquet
Processing stop time for all trips of pattern 701
Originally 5477 trips
Trying to process 5429 trips for Pattern 701 after filtering
Processed 5420 trips for Pattern 701. There was 4 trip(s) with errors.
Time taken for stop times for 701: 3.0690083503723145 minutes
Writing out/patterns/pid_15953_stop.parquet and out/patterns/pid_15953_segment.parquet
Processing stop time for all trips of pattern 15953
Originally 4173 trips
Trying to process 3956 trips for Pattern 15953 after filtering
Processed 3894 trips for Pattern 15953. There was 28 trip(s) with errors.
Time taken for stop times for 15953: 2.8731754660606383 minutes
Writing out/patterns/pid_3639_stop.parquet and out/patterns/pid_3639_segment.parquet
Processing stop time for all trips of pattern 3639
Originally 812 trips
Trying to process 776 trips for Pattern 3639 after filtering
Processed 770 trips for Pattern 3639. There was 1 trip(s) with errors.
Time taken for stop times for 3639: 0.28432031472524005 minutes
Writing out/patterns/pid_21483_stop.parquet and out/patterns/pid_21483_segment.parquet
Processing stop time for all trips of pattern 21483
Originally 81 trips
Trying to process 78 trips for Pattern 21483 after filtering
Processed 78 trips for Pattern 21483. There was 0 trip(s) with errors.
Time taken for stop times for 21483: 0.03319664796193441 minutes
Writing out/patterns/pid_7909_stop.parquet and out/patterns/pid_7909_segment.parquet
Processing stop time for all trips of pattern 7909
Originally 106 trips
Trying to process 105 trips for Pattern 7909 after filtering
Processed 105 trips for Pattern 7909. There was 0 trip(s) with errors.
Time taken for stop times for 7909: 0.05544917980829875 minutes
Writing out/patterns/pid_1187_stop.parquet and out/patterns/pid_1187_segment.parquet
Processing stop time for all trips of pattern 1187
Originally 861 trips
Trying to process 158 trips for Pattern 1187 after filtering
Processed 71 trips for Pattern 1187. There was 43 trip(s) with errors.
Time taken for stop times for 1187: 0.0431862473487854 minutes
Writing out/patterns/pid_5917_stop.parquet and out/patterns/pid_5917_segment.parquet
Processing stop time for all trips of pattern 5917
Originally 17985 trips
Trying to process 17822 trips for Pattern 5917 after filtering
Processed 17773 trips for Pattern 5917. There was 14 trip(s) with errors.
Time taken for stop times for 5917: 9.103140898545583 minutes
Writing out/patterns/pid_699_stop.parquet and out/patterns/pid_699_segment.parquet
Processing stop time for all trips of pattern 699
Originally 718 trips
Trying to process 654 trips for Pattern 699 after filtering
Processed 651 trips for Pattern 699. There was 0 trip(s) with errors.
Time taken for stop times for 699: 0.2277318835258484 minutes
Writing out/patterns/pid_4372_stop.parquet and out/patterns/pid_4372_segment.parquet
Processing stop time for all trips of pattern 4372
Originally 16548 trips
Trying to process 16264 trips for Pattern 4372 after filtering
Processed 16212 trips for Pattern 4372. There was 23 trip(s) with errors.
Time taken for stop times for 4372: 9.157486538092295 minutes
Do not have pattern 7733 available currently
Writing out/patterns/pid_845_stop.parquet and out/patterns/pid_845_segment.parquet
Processing stop time for all trips of pattern 845
Originally 107 trips
Trying to process 104 trips for Pattern 845 after filtering
Processed 104 trips for Pattern 845. There was 0 trip(s) with errors.
Time taken for stop times for 845: 0.064225705464681 minutes
Writing out/patterns/pid_4340_stop.parquet and out/patterns/pid_4340_segment.parquet
Processing stop time for all trips of pattern 4340
Originally 32 trips
Trying to process 32 trips for Pattern 4340 after filtering
Processed 32 trips for Pattern 4340. There was 0 trip(s) with errors.
Time taken for stop times for 4340: 0.019397799173990884 minutes
Writing out/patterns/pid_4338_stop.parquet and out/patterns/pid_4338_segment.parquet
Processing stop time for all trips of pattern 4338
Originally 7 trips
Trying to process 7 trips for Pattern 4338 after filtering
Processed 7 trips for Pattern 4338. There was 0 trip(s) with errors.
Time taken for stop times for 4338: 0.003924516836802164 minutes
Writing out/patterns/pid_7134_stop.parquet and out/patterns/pid_7134_segment.parquet
Processing stop time for all trips of pattern 7134
Originally 14759 trips
Trying to process 14646 trips for Pattern 7134 after filtering
Processed 14540 trips for Pattern 7134. There was 48 trip(s) with errors.
Time taken for stop times for 7134: 5.103921083609263 minutes
Writing out/patterns/pid_3954_stop.parquet and out/patterns/pid_3954_segment.parquet
Processing stop time for all trips of pattern 3954
Originally 357 trips
Trying to process 354 trips for Pattern 3954 after filtering
Processed 347 trips for Pattern 3954. There was 3 trip(s) with errors.
Time taken for stop times for 3954: 0.1529833197593689 minutes
Writing out/patterns/pid_6394_stop.parquet and out/patterns/pid_6394_segment.parquet
Processing stop time for all trips of pattern 6394
Originally 717 trips
Trying to process 706 trips for Pattern 6394 after filtering
Processed 706 trips for Pattern 6394. There was 0 trip(s) with errors.
Time taken for stop times for 6394: 0.35138397216796874 minutes
Writing out/patterns/pid_2823_stop.parquet and out/patterns/pid_2823_segment.parquet
Processing stop time for all trips of pattern 2823
Originally 1204 trips
Trying to process 1198 trips for Pattern 2823 after filtering
Processed 1196 trips for Pattern 2823. There was 1 trip(s) with errors.
Time taken for stop times for 2823: 0.5929419716199239 minutes
Writing out/patterns/pid_4590_stop.parquet and out/patterns/pid_4590_segment.parquet
Processing stop time for all trips of pattern 4590
Originally 27273 trips
Trying to process 26687 trips for Pattern 4590 after filtering
Processed 26643 trips for Pattern 4590. There was 18 trip(s) with errors.
Time taken for stop times for 4590: 13.63206961552302 minutes
Writing out/patterns/pid_9792_stop.parquet and out/patterns/pid_9792_segment.parquet
Processing stop time for all trips of pattern 9792
Originally 1196 trips
Trying to process 1193 trips for Pattern 9792 after filtering
Processed 1188 trips for Pattern 9792. There was 2 trip(s) with errors.
Time taken for stop times for 9792: 0.693318231900533 minutes
Writing out/patterns/pid_1274_stop.parquet and out/patterns/pid_1274_segment.parquet
Processing stop time for all trips of pattern 1274
Originally 4671 trips
Trying to process 4648 trips for Pattern 1274 after filtering
Processed 4644 trips for Pattern 1274. There was 2 trip(s) with errors.
Time taken for stop times for 1274: 2.763495683670044 minutes
Writing out/patterns/pid_7170_stop.parquet and out/patterns/pid_7170_segment.parquet
Processing stop time for all trips of pattern 7170
Originally 13491 trips
Trying to process 13423 trips for Pattern 7170 after filtering
Processed 13406 trips for Pattern 7170. There was 7 trip(s) with errors.
Time taken for stop times for 7170: 5.538511900107066 minutes
Writing out/patterns/pid_7131_stop.parquet and out/patterns/pid_7131_segment.parquet
Processing stop time for all trips of pattern 7131
Originally 8971 trips
Trying to process 8923 trips for Pattern 7131 after filtering
Processed 8915 trips for Pattern 7131. There was 3 trip(s) with errors.
Time taken for stop times for 7131: 3.99145214955012 minutes
Writing out/patterns/pid_2170_stop.parquet and out/patterns/pid_2170_segment.parquet
Processing stop time for all trips of pattern 2170
Originally 20196 trips
Trying to process 20081 trips for Pattern 2170 after filtering
Processed 20057 trips for Pattern 2170. There was 11 trip(s) with errors.
Time taken for stop times for 2170: 10.649285284678141 minutes
Writing out/patterns/pid_8086_stop.parquet and out/patterns/pid_8086_segment.parquet
Processing stop time for all trips of pattern 8086
Originally 2954 trips
Trying to process 2919 trips for Pattern 8086 after filtering
Processed 2620 trips for Pattern 8086. There was 147 trip(s) with errors.
Time taken for stop times for 8086: 0.8863908529281617 minutes
Writing out/patterns/pid_10919_stop.parquet and out/patterns/pid_10919_segment.parquet
Processing stop time for all trips of pattern 10919
Originally 2221 trips
Trying to process 2213 trips for Pattern 10919 after filtering
Processed 2211 trips for Pattern 10919. There was 1 trip(s) with errors.
Time taken for stop times for 10919: 1.0283644159634908 minutes
Writing out/patterns/pid_14121_stop.parquet and out/patterns/pid_14121_segment.parquet
Processing stop time for all trips of pattern 14121
Originally 226 trips
Trying to process 226 trips for Pattern 14121 after filtering
Processed 222 trips for Pattern 14121. There was 2 trip(s) with errors.
Time taken for stop times for 14121: 0.08430878321329753 minutes
Writing out/patterns/pid_7743_stop.parquet and out/patterns/pid_7743_segment.parquet
Processing stop time for all trips of pattern 7743
Originally 16151 trips
Trying to process 16006 trips for Pattern 7743 after filtering
Processed 15959 trips for Pattern 7743. There was 23 trip(s) with errors.
Time taken for stop times for 7743: 8.71009818315506 minutes
Writing out/patterns/pid_4615_stop.parquet and out/patterns/pid_4615_segment.parquet
Processing stop time for all trips of pattern 4615
Originally 897 trips
Trying to process 894 trips for Pattern 4615 after filtering
Processed 892 trips for Pattern 4615. There was 1 trip(s) with errors.
Time taken for stop times for 4615: 0.38976708650588987 minutes
Do not have pattern 1458 available currently
Writing out/patterns/pid_7570_stop.parquet and out/patterns/pid_7570_segment.parquet
Processing stop time for all trips of pattern 7570
Originally 29951 trips
Trying to process 28952 trips for Pattern 7570 after filtering
Processed 28542 trips for Pattern 7570. There was 176 trip(s) with errors.
Time taken for stop times for 7570: 16.119699410597484 minutes
Writing out/patterns/pid_5907_stop.parquet and out/patterns/pid_5907_segment.parquet
Processing stop time for all trips of pattern 5907
Originally 19500 trips
Trying to process 19154 trips for Pattern 5907 after filtering
Processed 18959 trips for Pattern 5907. There was 94 trip(s) with errors.
Time taken for stop times for 5907: 11.259670114517212 minutes
Writing out/patterns/pid_14120_stop.parquet and out/patterns/pid_14120_segment.parquet
Processing stop time for all trips of pattern 14120
Originally 24229 trips
Trying to process 24051 trips for Pattern 14120 after filtering
Processed 23993 trips for Pattern 14120. There was 22 trip(s) with errors.
Time taken for stop times for 14120: 13.094580213228861 minutes
Writing out/patterns/pid_1604_stop.parquet and out/patterns/pid_1604_segment.parquet
Processing stop time for all trips of pattern 1604
Originally 6428 trips
Trying to process 6264 trips for Pattern 1604 after filtering
Processed 6257 trips for Pattern 1604. There was 3 trip(s) with errors.
Time taken for stop times for 1604: 2.56490695476532 minutes
Writing out/patterns/pid_6643_stop.parquet and out/patterns/pid_6643_segment.parquet
Processing stop time for all trips of pattern 6643
Originally 5515 trips
Trying to process 5478 trips for Pattern 6643 after filtering
Processed 5460 trips for Pattern 6643. There was 8 trip(s) with errors.
Time taken for stop times for 6643: 3.400438133875529 minutes
Writing out/patterns/pid_4321_stop.parquet and out/patterns/pid_4321_segment.parquet
Processing stop time for all trips of pattern 4321
Originally 246 trips
Trying to process 244 trips for Pattern 4321 after filtering
Processed 244 trips for Pattern 4321. There was 0 trip(s) with errors.
Time taken for stop times for 4321: 0.11824466784795125 minutes
Writing out/patterns/pid_3626_stop.parquet and out/patterns/pid_3626_segment.parquet
Processing stop time for all trips of pattern 3626
Originally 701 trips
Trying to process 683 trips for Pattern 3626 after filtering
Processed 681 trips for Pattern 3626. There was 1 trip(s) with errors.
Time taken for stop times for 3626: 0.30579306681950885 minutes
Do not have pattern 6672 available currently
Writing out/patterns/pid_757_stop.parquet and out/patterns/pid_757_segment.parquet
Processing stop time for all trips of pattern 757
Originally 28405 trips
Trying to process 27722 trips for Pattern 757 after filtering
Processed 27600 trips for Pattern 757. There was 49 trip(s) with errors.
Time taken for stop times for 757: 14.500871368249257 minutes
Writing out/patterns/pid_7454_stop.parquet and out/patterns/pid_7454_segment.parquet
Processing stop time for all trips of pattern 7454
Originally 152 trips
Trying to process 143 trips for Pattern 7454 after filtering
Processed 141 trips for Pattern 7454. There was 1 trip(s) with errors.
Time taken for stop times for 7454: 0.05765469868977865 minutes
Writing out/patterns/pid_14108_stop.parquet and out/patterns/pid_14108_segment.parquet
Processing stop time for all trips of pattern 14108
Originally 10379 trips
Trying to process 9942 trips for Pattern 14108 after filtering
Processed 9876 trips for Pattern 14108. There was 26 trip(s) with errors.
Time taken for stop times for 14108: 3.5900516311327615 minutes
Writing out/patterns/pid_8226_stop.parquet and out/patterns/pid_8226_segment.parquet
Processing stop time for all trips of pattern 8226
Originally 1596 trips
Trying to process 1590 trips for Pattern 8226 after filtering
Processed 1589 trips for Pattern 8226. There was 0 trip(s) with errors.
Time taken for stop times for 8226: 1.130398734410604 minutes
Writing out/patterns/pid_21485_stop.parquet and out/patterns/pid_21485_segment.parquet
Processing stop time for all trips of pattern 21485
Originally 91 trips
Trying to process 90 trips for Pattern 21485 after filtering
Processed 88 trips for Pattern 21485. There was 1 trip(s) with errors.
Time taken for stop times for 21485: 0.04152531623840332 minutes
Do not have pattern 4219 available currently
Writing out/patterns/pid_23279_stop.parquet and out/patterns/pid_23279_segment.parquet
Processing stop time for all trips of pattern 23279
Originally 4 trips
Trying to process 4 trips for Pattern 23279 after filtering
Processed 4 trips for Pattern 23279. There was 0 trip(s) with errors.
Time taken for stop times for 23279: 0.0027230143547058105 minutes
Writing out/patterns/pid_1603_stop.parquet and out/patterns/pid_1603_segment.parquet
Processing stop time for all trips of pattern 1603
Originally 3780 trips
Trying to process 3710 trips for Pattern 1603 after filtering
Processed 3702 trips for Pattern 1603. There was 3 trip(s) with errors.
Time taken for stop times for 1603: 1.61762193441391 minutes
Do not have pattern 7906 available currently
Writing out/patterns/pid_6665_stop.parquet and out/patterns/pid_6665_segment.parquet
Processing stop time for all trips of pattern 6665
Originally 45464 trips
Trying to process 44695 trips for Pattern 6665 after filtering
Processed 44530 trips for Pattern 6665. There was 64 trip(s) with errors.
Time taken for stop times for 6665: 31.20148855050405 minutes
Writing out/patterns/pid_1116_stop.parquet and out/patterns/pid_1116_segment.parquet
Processing stop time for all trips of pattern 1116
Originally 5320 trips
Trying to process 5270 trips for Pattern 1116 after filtering
Processed 5257 trips for Pattern 1116. There was 5 trip(s) with errors.
Time taken for stop times for 1116: 2.8674564123153687 minutes
Writing out/patterns/pid_7020_stop.parquet and out/patterns/pid_7020_segment.parquet
Processing stop time for all trips of pattern 7020
Originally 16631 trips
Trying to process 16536 trips for Pattern 7020 after filtering
Processed 16518 trips for Pattern 7020. There was 8 trip(s) with errors.
Time taken for stop times for 7020: 8.05281740029653 minutes
Writing out/patterns/pid_19377_stop.parquet and out/patterns/pid_19377_segment.parquet
Processing stop time for all trips of pattern 19377
Originally 424 trips
Trying to process 407 trips for Pattern 19377 after filtering
Processed 397 trips for Pattern 19377. There was 2 trip(s) with errors.
Time taken for stop times for 19377: 0.12380826473236084 minutes
Writing out/patterns/pid_2863_stop.parquet and out/patterns/pid_2863_segment.parquet
Processing stop time for all trips of pattern 2863
Originally 3027 trips
Trying to process 3005 trips for Pattern 2863 after filtering
Processed 2983 trips for Pattern 2863. There was 8 trip(s) with errors.
Time taken for stop times for 2863: 1.0518164873123168 minutes
Writing out/patterns/pid_1763_stop.parquet and out/patterns/pid_1763_segment.parquet
Processing stop time for all trips of pattern 1763
Originally 625 trips
Trying to process 595 trips for Pattern 1763 after filtering
Processed 589 trips for Pattern 1763. There was 2 trip(s) with errors.
Time taken for stop times for 1763: 0.26238412062327066 minutes
Writing out/patterns/pid_7739_stop.parquet and out/patterns/pid_7739_segment.parquet
Processing stop time for all trips of pattern 7739
Originally 185 trips
Trying to process 184 trips for Pattern 7739 after filtering
Processed 184 trips for Pattern 7739. There was 0 trip(s) with errors.
Time taken for stop times for 7739: 0.09079298178354898 minutes
Writing out/patterns/pid_6772_stop.parquet and out/patterns/pid_6772_segment.parquet
Processing stop time for all trips of pattern 6772
Originally 5813 trips
Trying to process 5796 trips for Pattern 6772 after filtering
Processed 5792 trips for Pattern 6772. There was 1 trip(s) with errors.
Time taken for stop times for 6772: 3.33286315202713 minutes
Writing out/patterns/pid_7176_stop.parquet and out/patterns/pid_7176_segment.parquet
Processing stop time for all trips of pattern 7176
Originally 4 trips
Trying to process 4 trips for Pattern 7176 after filtering
Processed 4 trips for Pattern 7176. There was 0 trip(s) with errors.
Time taken for stop times for 7176: 0.003065784772237142 minutes
Writing out/patterns/pid_4611_stop.parquet and out/patterns/pid_4611_segment.parquet
Processing stop time for all trips of pattern 4611
Originally 10195 trips
Trying to process 10119 trips for Pattern 4611 after filtering
Processed 10097 trips for Pattern 4611. There was 9 trip(s) with errors.
Time taken for stop times for 4611: 6.5161600311597185 minutes
Writing out/patterns/pid_19382_stop.parquet and out/patterns/pid_19382_segment.parquet
Processing stop time for all trips of pattern 19382
Originally 3367 trips
Trying to process 3085 trips for Pattern 19382 after filtering
Processed 2934 trips for Pattern 19382. There was 68 trip(s) with errors.
Time taken for stop times for 19382: 2.478540352980296 minutes
Writing out/patterns/pid_1599_stop.parquet and out/patterns/pid_1599_segment.parquet
Processing stop time for all trips of pattern 1599
Originally 26100 trips
Trying to process 25732 trips for Pattern 1599 after filtering
Processed 25676 trips for Pattern 1599. There was 19 trip(s) with errors.
Time taken for stop times for 1599: 22.193846265474956 minutes
Writing out/patterns/pid_4492_stop.parquet and out/patterns/pid_4492_segment.parquet
Processing stop time for all trips of pattern 4492
Originally 4732 trips
Trying to process 4596 trips for Pattern 4492 after filtering
Processed 4544 trips for Pattern 4492. There was 14 trip(s) with errors.
Time taken for stop times for 4492: 1.672759199142456 minutes
Writing out/patterns/pid_4708_stop.parquet and out/patterns/pid_4708_segment.parquet
Processing stop time for all trips of pattern 4708
Originally 14134 trips
Trying to process 13809 trips for Pattern 4708 after filtering
Processed 13637 trips for Pattern 4708. There was 46 trip(s) with errors.
Time taken for stop times for 4708: 5.032398935159048 minutes
Writing out/patterns/pid_953_stop.parquet and out/patterns/pid_953_segment.parquet
Processing stop time for all trips of pattern 953
Originally 8 trips
Trying to process 7 trips for Pattern 953 after filtering
Processed 7 trips for Pattern 953. There was 0 trip(s) with errors.
Time taken for stop times for 953: 0.00507971445719401 minutes
Writing out/patterns/pid_7534_stop.parquet and out/patterns/pid_7534_segment.parquet
Processing stop time for all trips of pattern 7534
Originally 336 trips
Trying to process 201 trips for Pattern 7534 after filtering
Processed 195 trips for Pattern 7534. There was 2 trip(s) with errors.
Time taken for stop times for 7534: 0.06962881485621135 minutes
Writing out/patterns/pid_8112_stop.parquet and out/patterns/pid_8112_segment.parquet
Processing stop time for all trips of pattern 8112
Originally 96 trips
Trying to process 95 trips for Pattern 8112 after filtering
Processed 95 trips for Pattern 8112. There was 0 trip(s) with errors.
Time taken for stop times for 8112: 0.04473578135172526 minutes
Writing out/patterns/pid_4625_stop.parquet and out/patterns/pid_4625_segment.parquet
Processing stop time for all trips of pattern 4625
Originally 90 trips
Trying to process 86 trips for Pattern 4625 after filtering
Processed 86 trips for Pattern 4625. There was 0 trip(s) with errors.
Time taken for stop times for 4625: 0.03712703784306844 minutes
Writing out/patterns/pid_5066_stop.parquet and out/patterns/pid_5066_segment.parquet
Processing stop time for all trips of pattern 5066
Originally 16934 trips
Trying to process 16749 trips for Pattern 5066 after filtering
Processed 16727 trips for Pattern 5066. There was 8 trip(s) with errors.
Time taken for stop times for 5066: 11.349568796157836 minutes
Writing out/patterns/pid_7022_stop.parquet and out/patterns/pid_7022_segment.parquet
Processing stop time for all trips of pattern 7022
Originally 819 trips
Trying to process 814 trips for Pattern 7022 after filtering
Processed 812 trips for Pattern 7022. There was 0 trip(s) with errors.
Time taken for stop times for 7022: 0.4321571350097656 minutes
Writing out/patterns/pid_20426_stop.parquet and out/patterns/pid_20426_segment.parquet
Processing stop time for all trips of pattern 20426
Originally 27 trips
Trying to process 26 trips for Pattern 20426 after filtering
Processed 26 trips for Pattern 20426. There was 0 trip(s) with errors.
Time taken for stop times for 20426: 0.013424499829610189 minutes
Writing out/patterns/pid_1326_stop.parquet and out/patterns/pid_1326_segment.parquet
Processing stop time for all trips of pattern 1326
Originally 17828 trips
Trying to process 17717 trips for Pattern 1326 after filtering
Processed 17692 trips for Pattern 1326. There was 11 trip(s) with errors.
Time taken for stop times for 1326: 24.963121553262074 minutes
Writing out/patterns/pid_7021_stop.parquet and out/patterns/pid_7021_segment.parquet
Processing stop time for all trips of pattern 7021
Originally 16649 trips
Trying to process 16509 trips for Pattern 7021 after filtering
Processed 16457 trips for Pattern 7021. There was 24 trip(s) with errors.
Time taken for stop times for 7021: 40.406322785218556 minutes
Writing out/patterns/pid_4537_stop.parquet and out/patterns/pid_4537_segment.parquet
Processing stop time for all trips of pattern 4537
Originally 704 trips
Trying to process 701 trips for Pattern 4537 after filtering
Processed 701 trips for Pattern 4537. There was 0 trip(s) with errors.
Time taken for stop times for 4537: 0.23796336650848388 minutes
Writing out/patterns/pid_22569_stop.parquet and out/patterns/pid_22569_segment.parquet
Processing stop time for all trips of pattern 22569
Originally 81 trips
Trying to process 76 trips for Pattern 22569 after filtering
Processed 76 trips for Pattern 22569. There was 0 trip(s) with errors.
Time taken for stop times for 22569: 0.02556229829788208 minutes
Writing out/patterns/pid_7016_stop.parquet and out/patterns/pid_7016_segment.parquet
Processing stop time for all trips of pattern 7016
Originally 418 trips
Trying to process 406 trips for Pattern 7016 after filtering
Processed 406 trips for Pattern 7016. There was 0 trip(s) with errors.
Time taken for stop times for 7016: 0.12623708248138427 minutes
Writing out/patterns/pid_8128_stop.parquet and out/patterns/pid_8128_segment.parquet
Processing stop time for all trips of pattern 8128
Originally 6373 trips
Trying to process 6350 trips for Pattern 8128 after filtering
Processed 6347 trips for Pattern 8128. There was 1 trip(s) with errors.
Time taken for stop times for 8128: 10.085901927947997 minutes
Writing out/patterns/pid_10912_stop.parquet and out/patterns/pid_10912_segment.parquet
Processing stop time for all trips of pattern 10912
Originally 672 trips
Trying to process 658 trips for Pattern 10912 after filtering
Processed 656 trips for Pattern 10912. There was 1 trip(s) with errors.
Time taken for stop times for 10912: 0.9863879323005676 minutes
Writing out/patterns/pid_5351_stop.parquet and out/patterns/pid_5351_segment.parquet
Processing stop time for all trips of pattern 5351
Originally 1155 trips
Trying to process 1141 trips for Pattern 5351 after filtering
Processed 1141 trips for Pattern 5351. There was 0 trip(s) with errors.
Time taken for stop times for 5351: 0.31898818016052244 minutes
Writing out/patterns/pid_898_stop.parquet and out/patterns/pid_898_segment.parquet
Processing stop time for all trips of pattern 898
Originally 168 trips
Trying to process 167 trips for Pattern 898 after filtering
Processed 167 trips for Pattern 898. There was 0 trip(s) with errors.
Time taken for stop times for 898: 0.07439194917678833 minutes
Writing out/patterns/pid_14134_stop.parquet and out/patterns/pid_14134_segment.parquet
Processing stop time for all trips of pattern 14134
Originally 26541 trips
Trying to process 25799 trips for Pattern 14134 after filtering
Processed 25582 trips for Pattern 14134. There was 83 trip(s) with errors.
Time taken for stop times for 14134: 56.11268451611201 minutes
Writing out/patterns/pid_7204_stop.parquet and out/patterns/pid_7204_segment.parquet
Processing stop time for all trips of pattern 7204
Originally 143 trips
Trying to process 142 trips for Pattern 7204 after filtering
Processed 142 trips for Pattern 7204. There was 0 trip(s) with errors.
Time taken for stop times for 7204: 0.06877969900767009 minutes
Writing out/patterns/pid_3086_stop.parquet and out/patterns/pid_3086_segment.parquet
Processing stop time for all trips of pattern 3086
Originally 1056 trips
Trying to process 1038 trips for Pattern 3086 after filtering
Processed 1036 trips for Pattern 3086. There was 0 trip(s) with errors.
Time taken for stop times for 3086: 0.44337270259857176 minutes
Writing out/patterns/pid_1611_stop.parquet and out/patterns/pid_1611_segment.parquet
Processing stop time for all trips of pattern 1611
Originally 172 trips
Trying to process 147 trips for Pattern 1611 after filtering
Processed 143 trips for Pattern 1611. There was 2 trip(s) with errors.
Time taken for stop times for 1611: 0.1736204465230306 minutes
Writing out/patterns/pid_14102_stop.parquet and out/patterns/pid_14102_segment.parquet
Processing stop time for all trips of pattern 14102
Originally 3017 trips
Trying to process 2990 trips for Pattern 14102 after filtering
Processed 2987 trips for Pattern 14102. There was 0 trip(s) with errors.
Time taken for stop times for 14102: 1.2628623525301614 minutes
Writing out/patterns/pid_7111_stop.parquet and out/patterns/pid_7111_segment.parquet
Processing stop time for all trips of pattern 7111
Originally 22750 trips
Trying to process 22426 trips for Pattern 7111 after filtering
Processed 22362 trips for Pattern 7111. There was 25 trip(s) with errors.
Time taken for stop times for 7111: 23.870034515857697 minutes
Writing out/patterns/pid_246_stop.parquet and out/patterns/pid_246_segment.parquet
Processing stop time for all trips of pattern 246
Originally 4182 trips
Trying to process 4050 trips for Pattern 246 after filtering
Processed 4018 trips for Pattern 246. There was 13 trip(s) with errors.
Time taken for stop times for 246: 3.486259682973226 minutes
Writing out/patterns/pid_18415_stop.parquet and out/patterns/pid_18415_segment.parquet
Processing stop time for all trips of pattern 18415
Originally 33449 trips
Trying to process 32604 trips for Pattern 18415 after filtering
Processed 32453 trips for Pattern 18415. There was 59 trip(s) with errors.
Time taken for stop times for 18415: 32.24871978759766 minutes
Writing out/patterns/pid_4095_stop.parquet and out/patterns/pid_4095_segment.parquet
Processing stop time for all trips of pattern 4095
Originally 1012 trips
Trying to process 1003 trips for Pattern 4095 after filtering
Processed 1003 trips for Pattern 4095. There was 0 trip(s) with errors.
Time taken for stop times for 4095: 0.5802763819694519 minutes
Do not have pattern 17470 available currently
Writing out/patterns/pid_10940_stop.parquet and out/patterns/pid_10940_segment.parquet
Processing stop time for all trips of pattern 10940
Originally 393 trips
Trying to process 389 trips for Pattern 10940 after filtering
Processed 389 trips for Pattern 10940. There was 0 trip(s) with errors.
Time taken for stop times for 10940: 0.16750766436258951 minutes
Writing out/patterns/pid_1540_stop.parquet and out/patterns/pid_1540_segment.parquet
Processing stop time for all trips of pattern 1540
Originally 23893 trips
Trying to process 23611 trips for Pattern 1540 after filtering
Processed 23546 trips for Pattern 1540. There was 24 trip(s) with errors.
Time taken for stop times for 1540: 13.240092933177948 minutes
Writing out/patterns/pid_18417_stop.parquet and out/patterns/pid_18417_segment.parquet
Processing stop time for all trips of pattern 18417
Originally 159 trips
Trying to process 154 trips for Pattern 18417 after filtering
Processed 152 trips for Pattern 18417. There was 1 trip(s) with errors.
Time taken for stop times for 18417: 0.09147849877675375 minutes
Writing out/patterns/pid_2671_stop.parquet and out/patterns/pid_2671_segment.parquet
Processing stop time for all trips of pattern 2671
Originally 2421 trips
Trying to process 2364 trips for Pattern 2671 after filtering
Processed 2279 trips for Pattern 2671. There was 40 trip(s) with errors.
Time taken for stop times for 2671: 0.9080368876457214 minutes
Writing out/patterns/pid_1334_stop.parquet and out/patterns/pid_1334_segment.parquet
Processing stop time for all trips of pattern 1334
Originally 890 trips
Trying to process 876 trips for Pattern 1334 after filtering
Processed 873 trips for Pattern 1334. There was 1 trip(s) with errors.
Time taken for stop times for 1334: 0.34388711452484133 minutes
Writing out/patterns/pid_1093_stop.parquet and out/patterns/pid_1093_segment.parquet
Processing stop time for all trips of pattern 1093
Originally 377 trips
Trying to process 369 trips for Pattern 1093 after filtering
Processed 365 trips for Pattern 1093. There was 2 trip(s) with errors.
Time taken for stop times for 1093: 0.1665188988049825 minutes
Writing out/patterns/pid_1166_stop.parquet and out/patterns/pid_1166_segment.parquet
Processing stop time for all trips of pattern 1166
Originally 1211 trips
Trying to process 825 trips for Pattern 1166 after filtering
Processed 813 trips for Pattern 1166. There was 6 trip(s) with errors.
Time taken for stop times for 1166: 0.25793262720108034 minutes
Writing out/patterns/pid_4343_stop.parquet and out/patterns/pid_4343_segment.parquet
Processing stop time for all trips of pattern 4343
Originally 387 trips
Trying to process 371 trips for Pattern 4343 after filtering
Processed 367 trips for Pattern 4343. There was 1 trip(s) with errors.
Time taken for stop times for 4343: 0.2654622515042623 minutes
Writing out/patterns/pid_890_stop.parquet and out/patterns/pid_890_segment.parquet
Processing stop time for all trips of pattern 890
Originally 1317 trips
Trying to process 1304 trips for Pattern 890 after filtering
Processed 1299 trips for Pattern 890. There was 2 trip(s) with errors.
Time taken for stop times for 890: 0.9227096478144328 minutes
Writing out/patterns/pid_1232_stop.parquet and out/patterns/pid_1232_segment.parquet
Processing stop time for all trips of pattern 1232
Originally 32080 trips
Trying to process 31666 trips for Pattern 1232 after filtering
Processed 31588 trips for Pattern 1232. There was 29 trip(s) with errors.
Time taken for stop times for 1232: 19.933537232875825 minutes
Writing out/patterns/pid_959_stop.parquet and out/patterns/pid_959_segment.parquet
Processing stop time for all trips of pattern 959
Originally 4478 trips
Trying to process 4181 trips for Pattern 959 after filtering
Processed 4162 trips for Pattern 959. There was 6 trip(s) with errors.
Time taken for stop times for 959: 2.2697479168574017 minutes
Writing out/patterns/pid_6403_stop.parquet and out/patterns/pid_6403_segment.parquet
Processing stop time for all trips of pattern 6403
Originally 3536 trips
Trying to process 3509 trips for Pattern 6403 after filtering
Processed 3489 trips for Pattern 6403. There was 4 trip(s) with errors.
Time taken for stop times for 6403: 1.622664753595988 minutes
Writing out/patterns/pid_8140_stop.parquet and out/patterns/pid_8140_segment.parquet
Processing stop time for all trips of pattern 8140
Originally 326 trips
Trying to process 316 trips for Pattern 8140 after filtering
Processed 316 trips for Pattern 8140. There was 0 trip(s) with errors.
Time taken for stop times for 8140: 0.1402629812558492 minutes
Writing out/patterns/pid_8235_stop.parquet and out/patterns/pid_8235_segment.parquet
Processing stop time for all trips of pattern 8235
Originally 323 trips
Trying to process 305 trips for Pattern 8235 after filtering
Processed 266 trips for Pattern 8235. There was 14 trip(s) with errors.
Time taken for stop times for 8235: 0.1386260708173116 minutes
Writing out/patterns/pid_1628_stop.parquet and out/patterns/pid_1628_segment.parquet
Processing stop time for all trips of pattern 1628
Originally 21815 trips
Trying to process 21552 trips for Pattern 1628 after filtering
Processed 21506 trips for Pattern 1628. There was 19 trip(s) with errors.
Time taken for stop times for 1628: 9.172031116485595 minutes
Writing out/patterns/pid_4671_stop.parquet and out/patterns/pid_4671_segment.parquet
Processing stop time for all trips of pattern 4671
Originally 2423 trips
Trying to process 2375 trips for Pattern 4671 after filtering
Processed 2364 trips for Pattern 4671. There was 5 trip(s) with errors.
Time taken for stop times for 4671: 0.812518564860026 minutes
Writing out/patterns/pid_11678_stop.parquet and out/patterns/pid_11678_segment.parquet
Processing stop time for all trips of pattern 11678
Originally 1353 trips
Trying to process 1346 trips for Pattern 11678 after filtering
Processed 1346 trips for Pattern 11678. There was 0 trip(s) with errors.
Time taken for stop times for 11678: 0.5268131653467815 minutes
Writing out/patterns/pid_4586_stop.parquet and out/patterns/pid_4586_segment.parquet
Processing stop time for all trips of pattern 4586
Originally 1097 trips
Trying to process 1091 trips for Pattern 4586 after filtering
Processed 1091 trips for Pattern 4586. There was 0 trip(s) with errors.
Time taken for stop times for 4586: 0.3400652805964152 minutes
Writing out/patterns/pid_11676_stop.parquet and out/patterns/pid_11676_segment.parquet
Processing stop time for all trips of pattern 11676
Originally 1067 trips
Trying to process 1028 trips for Pattern 11676 after filtering
Processed 1023 trips for Pattern 11676. There was 1 trip(s) with errors.
Time taken for stop times for 11676: 0.37721674839655556 minutes
Writing out/patterns/pid_9373_stop.parquet and out/patterns/pid_9373_segment.parquet
Processing stop time for all trips of pattern 9373
Originally 1202 trips
Trying to process 1197 trips for Pattern 9373 after filtering
Processed 1196 trips for Pattern 9373. There was 0 trip(s) with errors.
Time taken for stop times for 9373: 0.7210758368174235 minutes
Writing out/patterns/pid_100_stop.parquet and out/patterns/pid_100_segment.parquet
Processing stop time for all trips of pattern 100
Originally 3362 trips
Trying to process 3280 trips for Pattern 100 after filtering
Processed 3279 trips for Pattern 100. There was 0 trip(s) with errors.
Time taken for stop times for 100: 1.0747288544972737 minutes
Writing out/patterns/pid_1652_stop.parquet and out/patterns/pid_1652_segment.parquet
Processing stop time for all trips of pattern 1652
Originally 3424 trips
Trying to process 3372 trips for Pattern 1652 after filtering
Processed 3369 trips for Pattern 1652. There was 1 trip(s) with errors.
Time taken for stop times for 1652: 1.1345552325248718 minutes
Writing out/patterns/pid_8096_stop.parquet and out/patterns/pid_8096_segment.parquet
Processing stop time for all trips of pattern 8096
Originally 1056 trips
Trying to process 1052 trips for Pattern 8096 after filtering
Processed 1052 trips for Pattern 8096. There was 0 trip(s) with errors.
Time taken for stop times for 8096: 0.4010661323865255 minutes
Writing out/patterns/pid_1802_stop.parquet and out/patterns/pid_1802_segment.parquet
Processing stop time for all trips of pattern 1802
Originally 13890 trips
Trying to process 13804 trips for Pattern 1802 after filtering
Processed 13786 trips for Pattern 1802. There was 8 trip(s) with errors.
Time taken for stop times for 1802: 6.017219316959381 minutes
Writing out/patterns/pid_1643_stop.parquet and out/patterns/pid_1643_segment.parquet
Processing stop time for all trips of pattern 1643
Originally 230 trips
Trying to process 229 trips for Pattern 1643 after filtering
Processed 229 trips for Pattern 1643. There was 0 trip(s) with errors.
Time taken for stop times for 1643: 0.08391149441401163 minutes
Writing out/patterns/pid_6369_stop.parquet and out/patterns/pid_6369_segment.parquet
Processing stop time for all trips of pattern 6369
Originally 2793 trips
Trying to process 2764 trips for Pattern 6369 after filtering
Processed 2743 trips for Pattern 6369. There was 7 trip(s) with errors.
Time taken for stop times for 6369: 0.7859574834505717 minutes
Writing out/patterns/pid_8421_stop.parquet and out/patterns/pid_8421_segment.parquet
Processing stop time for all trips of pattern 8421
Originally 1714 trips
Trying to process 1688 trips for Pattern 8421 after filtering
Processed 1687 trips for Pattern 8421. There was 0 trip(s) with errors.
Time taken for stop times for 8421: 0.5234825134277343 minutes
Writing out/patterns/pid_13247_stop.parquet and out/patterns/pid_13247_segment.parquet
Processing stop time for all trips of pattern 13247
Originally 1917 trips
Trying to process 1908 trips for Pattern 13247 after filtering
Processed 1904 trips for Pattern 13247. There was 2 trip(s) with errors.
Time taken for stop times for 13247: 0.7942749341328939 minutes
Writing out/patterns/pid_1347_stop.parquet and out/patterns/pid_1347_segment.parquet
Processing stop time for all trips of pattern 1347
Originally 1783 trips
Trying to process 1761 trips for Pattern 1347 after filtering
Processed 1757 trips for Pattern 1347. There was 0 trip(s) with errors.
Time taken for stop times for 1347: 0.7802266995112102 minutes
Writing out/patterns/pid_14127_stop.parquet and out/patterns/pid_14127_segment.parquet
Processing stop time for all trips of pattern 14127
Originally 331 trips
Trying to process 310 trips for Pattern 14127 after filtering
Processed 290 trips for Pattern 14127. There was 10 trip(s) with errors.
Time taken for stop times for 14127: 0.09065733353296916 minutes
Writing out/patterns/pid_7443_stop.parquet and out/patterns/pid_7443_segment.parquet
Processing stop time for all trips of pattern 7443
Originally 498 trips
Trying to process 491 trips for Pattern 7443 after filtering
Processed 484 trips for Pattern 7443. There was 3 trip(s) with errors.
Time taken for stop times for 7443: 0.16464191675186157 minutes
Writing out/patterns/pid_4621_stop.parquet and out/patterns/pid_4621_segment.parquet
Processing stop time for all trips of pattern 4621
Originally 21838 trips
Trying to process 21272 trips for Pattern 4621 after filtering
Processed 21150 trips for Pattern 4621. There was 57 trip(s) with errors.
Time taken for stop times for 4621: 9.739507937431336 minutes
Do not have pattern 19385 available currently
Do not have pattern 10908 available currently
Writing out/patterns/pid_13245_stop.parquet and out/patterns/pid_13245_segment.parquet
Processing stop time for all trips of pattern 13245
Originally 238 trips
Trying to process 234 trips for Pattern 13245 after filtering
Processed 234 trips for Pattern 13245. There was 0 trip(s) with errors.
Time taken for stop times for 13245: 0.09693188269933065 minutes
Writing out/patterns/pid_7032_stop.parquet and out/patterns/pid_7032_segment.parquet
Processing stop time for all trips of pattern 7032
Originally 26441 trips
Trying to process 25502 trips for Pattern 7032 after filtering
Processed 25298 trips for Pattern 7032. There was 96 trip(s) with errors.
Time taken for stop times for 7032: 22.49592404762904 minutes
Writing out/patterns/pid_19371_stop.parquet and out/patterns/pid_19371_segment.parquet
Processing stop time for all trips of pattern 19371
Originally 30 trips
Trying to process 28 trips for Pattern 19371 after filtering
Processed 28 trips for Pattern 19371. There was 0 trip(s) with errors.
Time taken for stop times for 19371: 0.01687369743982951 minutes
Writing out/patterns/pid_6356_stop.parquet and out/patterns/pid_6356_segment.parquet
Processing stop time for all trips of pattern 6356
Originally 21923 trips
Trying to process 21750 trips for Pattern 6356 after filtering
Processed 21702 trips for Pattern 6356. There was 22 trip(s) with errors.
Time taken for stop times for 6356: 12.118653869628906 minutes
Writing out/patterns/pid_5908_stop.parquet and out/patterns/pid_5908_segment.parquet
Processing stop time for all trips of pattern 5908
Originally 1815 trips
Trying to process 1799 trips for Pattern 5908 after filtering
Processed 1799 trips for Pattern 5908. There was 0 trip(s) with errors.
Time taken for stop times for 5908: 0.5973760485649109 minutes
Writing out/patterns/pid_10949_stop.parquet and out/patterns/pid_10949_segment.parquet
Processing stop time for all trips of pattern 10949
Originally 1565 trips
Trying to process 1547 trips for Pattern 10949 after filtering
Processed 1535 trips for Pattern 10949. There was 6 trip(s) with errors.
Time taken for stop times for 10949: 0.6736728509267171 minutes
Writing out/patterns/pid_7175_stop.parquet and out/patterns/pid_7175_segment.parquet
Processing stop time for all trips of pattern 7175
Originally 8329 trips
Trying to process 8248 trips for Pattern 7175 after filtering
Processed 8211 trips for Pattern 7175. There was 16 trip(s) with errors.
Time taken for stop times for 7175: 18.148377418518066 minutes
Writing out/patterns/pid_6653_stop.parquet and out/patterns/pid_6653_segment.parquet
Processing stop time for all trips of pattern 6653
Originally 1063 trips
Trying to process 1021 trips for Pattern 6653 after filtering
Processed 1017 trips for Pattern 6653. There was 2 trip(s) with errors.
Time taken for stop times for 6653: 0.2998620351155599 minutes
Writing out/patterns/pid_3635_stop.parquet and out/patterns/pid_3635_segment.parquet
Processing stop time for all trips of pattern 3635
Originally 835 trips
Trying to process 798 trips for Pattern 3635 after filtering
Processed 792 trips for Pattern 3635. There was 2 trip(s) with errors.
Time taken for stop times for 3635: 0.21129218339920045 minutes
Writing out/patterns/pid_10904_stop.parquet and out/patterns/pid_10904_segment.parquet
Processing stop time for all trips of pattern 10904
Originally 1279 trips
Trying to process 1266 trips for Pattern 10904 after filtering
Processed 1230 trips for Pattern 10904. There was 18 trip(s) with errors.
Time taken for stop times for 10904: 0.6740582982699076 minutes
Writing out/patterns/pid_10909_stop.parquet and out/patterns/pid_10909_segment.parquet
Processing stop time for all trips of pattern 10909
Originally 5079 trips
Trying to process 4931 trips for Pattern 10909 after filtering
Processed 4919 trips for Pattern 10909. There was 2 trip(s) with errors.
Time taken for stop times for 10909: 2.6233051141103108 minutes
Writing out/patterns/pid_5679_stop.parquet and out/patterns/pid_5679_segment.parquet
Processing stop time for all trips of pattern 5679
Originally 356 trips
Trying to process 352 trips for Pattern 5679 after filtering
Processed 352 trips for Pattern 5679. There was 0 trip(s) with errors.
Time taken for stop times for 5679: 0.11087008317311604 minutes
Writing out/patterns/pid_3918_stop.parquet and out/patterns/pid_3918_segment.parquet
Processing stop time for all trips of pattern 3918
Originally 16199 trips
Trying to process 16129 trips for Pattern 3918 after filtering
Processed 16113 trips for Pattern 3918. There was 6 trip(s) with errors.
Time taken for stop times for 3918: 14.847068564097087 minutes
Writing out/patterns/pid_957_stop.parquet and out/patterns/pid_957_segment.parquet
Processing stop time for all trips of pattern 957
Originally 18856 trips
Trying to process 18573 trips for Pattern 957 after filtering
Processed 18508 trips for Pattern 957. There was 21 trip(s) with errors.
Time taken for stop times for 957: 7.296372747421264 minutes
Writing out/patterns/pid_5529_stop.parquet and out/patterns/pid_5529_segment.parquet
Processing stop time for all trips of pattern 5529
Originally 3138 trips
Trying to process 3108 trips for Pattern 5529 after filtering
Processed 3092 trips for Pattern 5529. There was 8 trip(s) with errors.
Time taken for stop times for 5529: 1.3071442008018495 minutes
Writing out/patterns/pid_5069_stop.parquet and out/patterns/pid_5069_segment.parquet
Processing stop time for all trips of pattern 5069
Originally 1750 trips
Trying to process 1732 trips for Pattern 5069 after filtering
Processed 1728 trips for Pattern 5069. There was 2 trip(s) with errors.
Time taken for stop times for 5069: 0.9435512820879618 minutes
Writing out/patterns/pid_5680_stop.parquet and out/patterns/pid_5680_segment.parquet
Processing stop time for all trips of pattern 5680
Originally 853 trips
Trying to process 850 trips for Pattern 5680 after filtering
Processed 850 trips for Pattern 5680. There was 0 trip(s) with errors.
Time taken for stop times for 5680: 0.35870749950408937 minutes
Do not have pattern 4347 available currently
Writing out/patterns/pid_8091_stop.parquet and out/patterns/pid_8091_segment.parquet
Processing stop time for all trips of pattern 8091
Originally 1357 trips
Trying to process 1348 trips for Pattern 8091 after filtering
Processed 1343 trips for Pattern 8091. There was 2 trip(s) with errors.
Time taken for stop times for 8091: 0.3638794819513957 minutes
Writing out/patterns/pid_15022_stop.parquet and out/patterns/pid_15022_segment.parquet
Processing stop time for all trips of pattern 15022
Originally 150 trips
Trying to process 148 trips for Pattern 15022 after filtering
Processed 148 trips for Pattern 15022. There was 0 trip(s) with errors.
Time taken for stop times for 15022: 0.05117358764012655 minutes
Do not have pattern 7907 available currently
Writing out/patterns/pid_8116_stop.parquet and out/patterns/pid_8116_segment.parquet
Processing stop time for all trips of pattern 8116
Originally 501 trips
Trying to process 499 trips for Pattern 8116 after filtering
Processed 499 trips for Pattern 8116. There was 0 trip(s) with errors.
Time taken for stop times for 8116: 0.17573284705479938 minutes
Writing out/patterns/pid_7110_stop.parquet and out/patterns/pid_7110_segment.parquet
Processing stop time for all trips of pattern 7110
Originally 1762 trips
Trying to process 1741 trips for Pattern 7110 after filtering
Processed 1738 trips for Pattern 7110. There was 1 trip(s) with errors.
Time taken for stop times for 7110: 0.9169850985209147 minutes
Writing out/patterns/pid_4610_stop.parquet and out/patterns/pid_4610_segment.parquet
Processing stop time for all trips of pattern 4610
Originally 11838 trips
Trying to process 11613 trips for Pattern 4610 after filtering
Processed 11590 trips for Pattern 4610. There was 11 trip(s) with errors.
Time taken for stop times for 4610: 5.661413033803304 minutes
Writing out/patterns/pid_7560_stop.parquet and out/patterns/pid_7560_segment.parquet
Processing stop time for all trips of pattern 7560
Originally 2413 trips
Trying to process 2363 trips for Pattern 7560 after filtering
Processed 2358 trips for Pattern 7560. There was 1 trip(s) with errors.
Time taken for stop times for 7560: 1.6654713869094848 minutes
Writing out/patterns/pid_8184_stop.parquet and out/patterns/pid_8184_segment.parquet
Processing stop time for all trips of pattern 8184
Originally 550 trips
Trying to process 547 trips for Pattern 8184 after filtering
Processed 545 trips for Pattern 8184. There was 1 trip(s) with errors.
Time taken for stop times for 8184: 0.22948856751124064 minutes
Writing out/patterns/pid_4353_stop.parquet and out/patterns/pid_4353_segment.parquet
Processing stop time for all trips of pattern 4353
Originally 1363 trips
Trying to process 1355 trips for Pattern 4353 after filtering
Processed 1354 trips for Pattern 4353. There was 0 trip(s) with errors.
Time taken for stop times for 4353: 0.5103959321975708 minutes
Writing out/patterns/pid_1178_stop.parquet and out/patterns/pid_1178_segment.parquet
Processing stop time for all trips of pattern 1178
Originally 36557 trips
Trying to process 35761 trips for Pattern 1178 after filtering
Processed 35568 trips for Pattern 1178. There was 85 trip(s) with errors.
Time taken for stop times for 1178: 100.81285161972046 minutes
Writing out/patterns/pid_4526_stop.parquet and out/patterns/pid_4526_segment.parquet
Processing stop time for all trips of pattern 4526
Originally 21016 trips
Trying to process 20532 trips for Pattern 4526 after filtering
Processed 20249 trips for Pattern 4526. There was 124 trip(s) with errors.
Time taken for stop times for 4526: 55.26659513314565 minutes
Writing out/patterns/pid_12429_stop.parquet and out/patterns/pid_12429_segment.parquet
Processing stop time for all trips of pattern 12429
Originally 2181 trips
Trying to process 2170 trips for Pattern 12429 after filtering
Processed 2165 trips for Pattern 12429. There was 2 trip(s) with errors.
Time taken for stop times for 12429: 1.4504134019215902 minutes
Writing out/patterns/pid_4601_stop.parquet and out/patterns/pid_4601_segment.parquet
Processing stop time for all trips of pattern 4601
Originally 6391 trips
Trying to process 4355 trips for Pattern 4601 after filtering
Processed 4332 trips for Pattern 4601. There was 3 trip(s) with errors.
Time taken for stop times for 4601: 1.435852599143982 minutes
Writing out/patterns/pid_9250_stop.parquet and out/patterns/pid_9250_segment.parquet
Processing stop time for all trips of pattern 9250
Originally 175 trips
Trying to process 171 trips for Pattern 9250 after filtering
Processed 169 trips for Pattern 9250. There was 1 trip(s) with errors.
Time taken for stop times for 9250: 0.08626131614049276 minutes
Writing out/patterns/pid_703_stop.parquet and out/patterns/pid_703_segment.parquet
Processing stop time for all trips of pattern 703
Originally 5686 trips
Trying to process 5630 trips for Pattern 703 after filtering
Processed 5618 trips for Pattern 703. There was 5 trip(s) with errors.
Time taken for stop times for 703: 2.2485791365305583 minutes
Writing out/patterns/pid_96_stop.parquet and out/patterns/pid_96_segment.parquet
Processing stop time for all trips of pattern 96
Originally 2408 trips
Trying to process 2345 trips for Pattern 96 after filtering
Processed 2339 trips for Pattern 96. There was 3 trip(s) with errors.
Time taken for stop times for 96: 1.3766564806302388 minutes
Writing out/patterns/pid_702_stop.parquet and out/patterns/pid_702_segment.parquet
Processing stop time for all trips of pattern 702
Originally 169 trips
Trying to process 165 trips for Pattern 702 after filtering
Processed 163 trips for Pattern 702. There was 1 trip(s) with errors.
Time taken for stop times for 702: 0.07228875160217285 minutes
Writing out/patterns/pid_5912_stop.parquet and out/patterns/pid_5912_segment.parquet
Processing stop time for all trips of pattern 5912
Originally 14602 trips
Trying to process 14455 trips for Pattern 5912 after filtering
Processed 14379 trips for Pattern 5912. There was 37 trip(s) with errors.
Time taken for stop times for 5912: 7.907537269592285 minutes
Writing out/patterns/pid_4619_stop.parquet and out/patterns/pid_4619_segment.parquet
Processing stop time for all trips of pattern 4619
Originally 20776 trips
Trying to process 20539 trips for Pattern 4619 after filtering
Processed 20528 trips for Pattern 4619. There was 5 trip(s) with errors.
Time taken for stop times for 4619: 23.495615764458975 minutes
Writing out/patterns/pid_8419_stop.parquet and out/patterns/pid_8419_segment.parquet
Processing stop time for all trips of pattern 8419
Originally 3896 trips
Trying to process 3879 trips for Pattern 8419 after filtering
Processed 3877 trips for Pattern 8419. There was 1 trip(s) with errors.
Time taken for stop times for 8419: 1.7981579343477885 minutes
Writing out/patterns/pid_4664_stop.parquet and out/patterns/pid_4664_segment.parquet
Processing stop time for all trips of pattern 4664
Originally 9856 trips
Trying to process 9740 trips for Pattern 4664 after filtering
Processed 9718 trips for Pattern 4664. There was 8 trip(s) with errors.
Time taken for stop times for 4664: 2.7629631996154784 minutes
Writing out/patterns/pid_4106_stop.parquet and out/patterns/pid_4106_segment.parquet
Processing stop time for all trips of pattern 4106
Originally 1593 trips
Trying to process 1566 trips for Pattern 4106 after filtering
Processed 1563 trips for Pattern 4106. There was 1 trip(s) with errors.
Time taken for stop times for 4106: 0.44753485520680747 minutes
Writing out/patterns/pid_711_stop.parquet and out/patterns/pid_711_segment.parquet
Processing stop time for all trips of pattern 711
Originally 18273 trips
Trying to process 18003 trips for Pattern 711 after filtering
Processed 17480 trips for Pattern 711. There was 158 trip(s) with errors.
Time taken for stop times for 711: 39.946422715981804 minutes
Writing out/patterns/pid_5210_stop.parquet and out/patterns/pid_5210_segment.parquet
Processing stop time for all trips of pattern 5210
Originally 3953 trips
Trying to process 3883 trips for Pattern 5210 after filtering
Processed 3853 trips for Pattern 5210. There was 12 trip(s) with errors.
Time taken for stop times for 5210: 9.770533835887909 minutes
Writing out/patterns/pid_10918_stop.parquet and out/patterns/pid_10918_segment.parquet
Processing stop time for all trips of pattern 10918
Originally 2294 trips
Trying to process 2277 trips for Pattern 10918 after filtering
Processed 2273 trips for Pattern 10918. There was 1 trip(s) with errors.
Time taken for stop times for 10918: 0.9990947683652242 minutes
Writing out/patterns/pid_8099_stop.parquet and out/patterns/pid_8099_segment.parquet
Processing stop time for all trips of pattern 8099
Originally 1194 trips
Trying to process 1188 trips for Pattern 8099 after filtering
Processed 1184 trips for Pattern 8099. There was 1 trip(s) with errors.
Time taken for stop times for 8099: 0.5757362802823385 minutes
Writing out/patterns/pid_1659_stop.parquet and out/patterns/pid_1659_segment.parquet
Processing stop time for all trips of pattern 1659
Originally 317 trips
Trying to process 314 trips for Pattern 1659 after filtering
Processed 312 trips for Pattern 1659. There was 1 trip(s) with errors.
Time taken for stop times for 1659: 0.12813559770584107 minutes
Writing out/patterns/pid_717_stop.parquet and out/patterns/pid_717_segment.parquet
Processing stop time for all trips of pattern 717
Originally 17076 trips
Trying to process 16852 trips for Pattern 717 after filtering
Processed 16781 trips for Pattern 717. There was 30 trip(s) with errors.
Time taken for stop times for 717: 55.12051318089167 minutes
Writing out/patterns/pid_8423_stop.parquet and out/patterns/pid_8423_segment.parquet
Processing stop time for all trips of pattern 8423
Originally 1206 trips
Trying to process 1194 trips for Pattern 8423 after filtering
Processed 1189 trips for Pattern 8423. There was 2 trip(s) with errors.
Time taken for stop times for 8423: 0.43196190198262535 minutes
Writing out/patterns/pid_4508_stop.parquet and out/patterns/pid_4508_segment.parquet
Processing stop time for all trips of pattern 4508
Originally 2845 trips
Trying to process 2830 trips for Pattern 4508 after filtering
Processed 2827 trips for Pattern 4508. There was 1 trip(s) with errors.
Time taken for stop times for 4508: 0.908903980255127 minutes
Writing out/patterns/pid_7027_stop.parquet and out/patterns/pid_7027_segment.parquet
Processing stop time for all trips of pattern 7027
Originally 1071 trips
Trying to process 1045 trips for Pattern 7027 after filtering
Processed 1042 trips for Pattern 7027. There was 1 trip(s) with errors.
Time taken for stop times for 7027: 0.2968484322230021 minutes
Writing out/patterns/pid_8234_stop.parquet and out/patterns/pid_8234_segment.parquet
Processing stop time for all trips of pattern 8234
Originally 370 trips
Trying to process 363 trips for Pattern 8234 after filtering
Processed 360 trips for Pattern 8234. There was 1 trip(s) with errors.
Time taken for stop times for 8234: 0.12443974812825521 minutes
Writing out/patterns/pid_6410_stop.parquet and out/patterns/pid_6410_segment.parquet
Processing stop time for all trips of pattern 6410
Originally 2144 trips
Trying to process 2109 trips for Pattern 6410 after filtering
Processed 2097 trips for Pattern 6410. There was 4 trip(s) with errors.
Time taken for stop times for 6410: 0.7705791672070821 minutes
Do not have pattern 8463 available currently
Do not have pattern 1466 available currently
Writing out/patterns/pid_1800_stop.parquet and out/patterns/pid_1800_segment.parquet
Processing stop time for all trips of pattern 1800
Originally 13901 trips
Trying to process 13806 trips for Pattern 1800 after filtering
Processed 13773 trips for Pattern 1800. There was 14 trip(s) with errors.
Time taken for stop times for 1800: 7.546640014648437 minutes
Writing out/patterns/pid_2725_stop.parquet and out/patterns/pid_2725_segment.parquet
Processing stop time for all trips of pattern 2725
Originally 178 trips
Trying to process 174 trips for Pattern 2725 after filtering
Processed 172 trips for Pattern 2725. There was 1 trip(s) with errors.
Time taken for stop times for 2725: 0.04542883634567261 minutes
Writing out/patterns/pid_8458_stop.parquet and out/patterns/pid_8458_segment.parquet
Processing stop time for all trips of pattern 8458
Originally 262 trips
Trying to process 259 trips for Pattern 8458 after filtering
Processed 253 trips for Pattern 8458. There was 3 trip(s) with errors.
Time taken for stop times for 8458: 0.10683588186899821 minutes
Writing out/patterns/pid_4585_stop.parquet and out/patterns/pid_4585_segment.parquet
Processing stop time for all trips of pattern 4585
Originally 2660 trips
Trying to process 2641 trips for Pattern 4585 after filtering
Processed 2638 trips for Pattern 4585. There was 1 trip(s) with errors.
Time taken for stop times for 4585: 0.7181507150332133 minutes
Writing out/patterns/pid_8959_stop.parquet and out/patterns/pid_8959_segment.parquet
Processing stop time for all trips of pattern 8959
Originally 1076 trips
Trying to process 1067 trips for Pattern 8959 after filtering
Processed 1067 trips for Pattern 8959. There was 0 trip(s) with errors.
Time taken for stop times for 8959: 0.4847640633583069 minutes
Writing out/patterns/pid_4116_stop.parquet and out/patterns/pid_4116_segment.parquet
Processing stop time for all trips of pattern 4116
Originally 598 trips
Trying to process 581 trips for Pattern 4116 after filtering
Processed 581 trips for Pattern 4116. There was 0 trip(s) with errors.
Time taken for stop times for 4116: 0.18808335065841675 minutes
Writing out/patterns/pid_5021_stop.parquet and out/patterns/pid_5021_segment.parquet
Processing stop time for all trips of pattern 5021
Originally 2741 trips
Trying to process 2679 trips for Pattern 5021 after filtering
Processed 2665 trips for Pattern 5021. There was 5 trip(s) with errors.
Time taken for stop times for 5021: 0.7480378826459249 minutes
Writing out/patterns/pid_1031_stop.parquet and out/patterns/pid_1031_segment.parquet
Processing stop time for all trips of pattern 1031
Originally 1592 trips
Trying to process 1580 trips for Pattern 1031 after filtering
Processed 1576 trips for Pattern 1031. There was 2 trip(s) with errors.
Time taken for stop times for 1031: 0.8137016812960307 minutes
Do not have pattern 3226 available currently
Writing out/patterns/pid_19396_stop.parquet and out/patterns/pid_19396_segment.parquet
Processing stop time for all trips of pattern 19396
Originally 1893 trips
Trying to process 1881 trips for Pattern 19396 after filtering
Processed 1875 trips for Pattern 19396. There was 3 trip(s) with errors.
Time taken for stop times for 19396: 0.8536951979001363 minutes
Writing out/patterns/pid_7749_stop.parquet and out/patterns/pid_7749_segment.parquet
Processing stop time for all trips of pattern 7749
Originally 162 trips
Trying to process 160 trips for Pattern 7749 after filtering
Processed 155 trips for Pattern 7749. There was 2 trip(s) with errors.
Time taken for stop times for 7749: 0.047148231665293375 minutes
Writing out/patterns/pid_1295_stop.parquet and out/patterns/pid_1295_segment.parquet
Processing stop time for all trips of pattern 1295
Originally 2361 trips
Trying to process 1970 trips for Pattern 1295 after filtering
Processed 1968 trips for Pattern 1295. There was 0 trip(s) with errors.
Time taken for stop times for 1295: 16.995746449629465 minutes
Writing out/patterns/pid_4604_stop.parquet and out/patterns/pid_4604_segment.parquet
Processing stop time for all trips of pattern 4604
Originally 294 trips
Trying to process 287 trips for Pattern 4604 after filtering
Processed 281 trips for Pattern 4604. There was 3 trip(s) with errors.
Time taken for stop times for 4604: 0.13484385013580322 minutes
Writing out/patterns/pid_8144_stop.parquet and out/patterns/pid_8144_segment.parquet
Processing stop time for all trips of pattern 8144
Originally 487 trips
Trying to process 360 trips for Pattern 8144 after filtering
Processed 353 trips for Pattern 8144. There was 3 trip(s) with errors.
Time taken for stop times for 8144: 0.1349273165067037 minutes
Writing out/patterns/pid_5906_stop.parquet and out/patterns/pid_5906_segment.parquet
Processing stop time for all trips of pattern 5906
Originally 7029 trips
Trying to process 6756 trips for Pattern 5906 after filtering
Processed 6685 trips for Pattern 5906. There was 34 trip(s) with errors.
Time taken for stop times for 5906: 18.046577906608583 minutes
Writing out/patterns/pid_420_stop.parquet and out/patterns/pid_420_segment.parquet
Processing stop time for all trips of pattern 420
Originally 487 trips
Trying to process 473 trips for Pattern 420 after filtering
Processed 471 trips for Pattern 420. There was 0 trip(s) with errors.
Time taken for stop times for 420: 0.1619661529858907 minutes
Writing out/patterns/pid_8108_stop.parquet and out/patterns/pid_8108_segment.parquet
Processing stop time for all trips of pattern 8108
Originally 430 trips
Trying to process 427 trips for Pattern 8108 after filtering
Processed 427 trips for Pattern 8108. There was 0 trip(s) with errors.
Time taken for stop times for 8108: 0.13382010062535604 minutes
Writing out/patterns/pid_10956_stop.parquet and out/patterns/pid_10956_segment.parquet
Processing stop time for all trips of pattern 10956
Originally 23203 trips
Trying to process 22923 trips for Pattern 10956 after filtering
/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/Interpolation.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trip_df.loc[:, "data_time"] = pd.to_datetime(trip_df.data_time)
Processed 22864 trips for Pattern 10956. There was 22 trip(s) with errors.
Time taken for stop times for 10956: 50.906287586689 minutes
Writing out/patterns/pid_1708_stop.parquet and out/patterns/pid_1708_segment.parquet
Processing stop time for all trips of pattern 1708
Originally 2850 trips
Trying to process 1647 trips for Pattern 1708 after filtering
Processed 1641 trips for Pattern 1708. There was 2 trip(s) with errors.
Time taken for stop times for 1708: 0.3910627841949463 minutes
Writing out/patterns/pid_8231_stop.parquet and out/patterns/pid_8231_segment.parquet
Processing stop time for all trips of pattern 8231
Originally 306 trips
Trying to process 50 trips for Pattern 8231 after filtering
Processed 47 trips for Pattern 8231. There was 1 trip(s) with errors.
Time taken for stop times for 8231: 0.012746047973632813 minutes
Writing out/patterns/pid_6363_stop.parquet and out/patterns/pid_6363_segment.parquet
Processing stop time for all trips of pattern 6363
Originally 1986 trips
Trying to process 1920 trips for Pattern 6363 after filtering
Processed 1900 trips for Pattern 6363. There was 10 trip(s) with errors.
Time taken for stop times for 6363: 0.48504008452097574 minutes
Writing out/patterns/pid_19395_stop.parquet and out/patterns/pid_19395_segment.parquet
Processing stop time for all trips of pattern 19395
Originally 1166 trips
Trying to process 1161 trips for Pattern 19395 after filtering
Processed 1161 trips for Pattern 19395. There was 0 trip(s) with errors.
Time taken for stop times for 19395: 0.5029574672381083 minutes
Writing out/patterns/pid_8383_stop.parquet and out/patterns/pid_8383_segment.parquet
Processing stop time for all trips of pattern 8383
Originally 324 trips
Trying to process 323 trips for Pattern 8383 after filtering
Processed 321 trips for Pattern 8383. There was 1 trip(s) with errors.
Time taken for stop times for 8383: 0.14411199887593587 minutes
Writing out/patterns/pid_5693_stop.parquet and out/patterns/pid_5693_segment.parquet
Processing stop time for all trips of pattern 5693
Originally 678 trips
Trying to process 666 trips for Pattern 5693 after filtering
Processed 666 trips for Pattern 5693. There was 0 trip(s) with errors.
Time taken for stop times for 5693: 0.3987273971239726 minutes
Writing out/patterns/pid_786_stop.parquet and out/patterns/pid_786_segment.parquet
Processing stop time for all trips of pattern 786
Originally 160 trips
Trying to process 159 trips for Pattern 786 after filtering
Processed 159 trips for Pattern 786. There was 0 trip(s) with errors.
Time taken for stop times for 786: 0.057368910312652587 minutes
Writing out/patterns/pid_582_stop.parquet and out/patterns/pid_582_segment.parquet
Processing stop time for all trips of pattern 582
Originally 7495 trips
Trying to process 7407 trips for Pattern 582 after filtering
Processed 7366 trips for Pattern 582. There was 11 trip(s) with errors.
Time taken for stop times for 582: 48.31503427028656 minutes
Writing out/patterns/pid_12428_stop.parquet and out/patterns/pid_12428_segment.parquet
Processing stop time for all trips of pattern 12428
Originally 2454 trips
Trying to process 2420 trips for Pattern 12428 after filtering
Processed 2401 trips for Pattern 12428. There was 8 trip(s) with errors.
Time taken for stop times for 12428: 16.48893726269404 minutes
Writing out/patterns/pid_893_stop.parquet and out/patterns/pid_893_segment.parquet
Processing stop time for all trips of pattern 893
Originally 342 trips
Trying to process 336 trips for Pattern 893 after filtering
Processed 334 trips for Pattern 893. There was 1 trip(s) with errors.
Time taken for stop times for 893: 0.13626923163731894 minutes
Writing out/patterns/pid_6128_stop.parquet and out/patterns/pid_6128_segment.parquet
Processing stop time for all trips of pattern 6128
Originally 20761 trips
Trying to process 20250 trips for Pattern 6128 after filtering
Processed 20055 trips for Pattern 6128. There was 95 trip(s) with errors.
Time taken for stop times for 6128: 27.00850263039271 minutes
Writing out/patterns/pid_14112_stop.parquet and out/patterns/pid_14112_segment.parquet
Processing stop time for all trips of pattern 14112
Originally 468 trips
Trying to process 465 trips for Pattern 14112 after filtering
Processed 463 trips for Pattern 14112. There was 1 trip(s) with errors.
Time taken for stop times for 14112: 0.17973659833272299 minutes
Writing out/patterns/pid_1497_stop.parquet and out/patterns/pid_1497_segment.parquet
Processing stop time for all trips of pattern 1497
Originally 210 trips
Trying to process 207 trips for Pattern 1497 after filtering
Processed 205 trips for Pattern 1497. There was 1 trip(s) with errors.
Time taken for stop times for 1497: 0.08051593701044718 minutes
Writing out/patterns/pid_8107_stop.parquet and out/patterns/pid_8107_segment.parquet
Processing stop time for all trips of pattern 8107
Originally 904 trips
Trying to process 900 trips for Pattern 8107 after filtering
Processed 898 trips for Pattern 8107. There was 1 trip(s) with errors.
Time taken for stop times for 8107: 0.2723437984784444 minutes
Writing out/patterns/pid_3084_stop.parquet and out/patterns/pid_3084_segment.parquet
Processing stop time for all trips of pattern 3084
Originally 20454 trips
Trying to process 20267 trips for Pattern 3084 after filtering
Processed 20211 trips for Pattern 3084. There was 27 trip(s) with errors.
Time taken for stop times for 3084: 12.148534548282623 minutes
Writing out/patterns/pid_4493_stop.parquet and out/patterns/pid_4493_segment.parquet
Processing stop time for all trips of pattern 4493
Originally 15853 trips
Trying to process 15631 trips for Pattern 4493 after filtering
Processed 15493 trips for Pattern 4493. There was 58 trip(s) with errors.
Time taken for stop times for 4493: 4.771546566486359 minutes
Do not have pattern 19399 available currently
Writing out/patterns/pid_4361_stop.parquet and out/patterns/pid_4361_segment.parquet
Processing stop time for all trips of pattern 4361
Originally 14170 trips
Trying to process 14118 trips for Pattern 4361 after filtering
Processed 14105 trips for Pattern 4361. There was 4 trip(s) with errors.
Time taken for stop times for 4361: 5.615070486068726 minutes
Writing out/patterns/pid_7129_stop.parquet and out/patterns/pid_7129_segment.parquet
Processing stop time for all trips of pattern 7129
Originally 13753 trips
Trying to process 13456 trips for Pattern 7129 after filtering
Processed 13438 trips for Pattern 7129. There was 9 trip(s) with errors.
Time taken for stop times for 7129: 35.574740882714586 minutes
Writing out/patterns/pid_7174_stop.parquet and out/patterns/pid_7174_segment.parquet
Processing stop time for all trips of pattern 7174
Originally 8315 trips
Trying to process 8196 trips for Pattern 7174 after filtering
Processed 8167 trips for Pattern 7174. There was 14 trip(s) with errors.
Time taken for stop times for 7174: 18.26685704787572 minutes
Do not have pattern 8386 available currently
Writing out/patterns/pid_9794_stop.parquet and out/patterns/pid_9794_segment.parquet
Processing stop time for all trips of pattern 9794
Originally 488 trips
Trying to process 485 trips for Pattern 9794 after filtering
Processed 485 trips for Pattern 9794. There was 0 trip(s) with errors.
Time taken for stop times for 9794: 0.26467068592707316 minutes
Writing out/patterns/pid_896_stop.parquet and out/patterns/pid_896_segment.parquet
Processing stop time for all trips of pattern 896
Originally 2000 trips
Trying to process 1956 trips for Pattern 896 after filtering
Processed 1952 trips for Pattern 896. There was 1 trip(s) with errors.
Time taken for stop times for 896: 0.5517413020133972 minutes
Writing out/patterns/pid_4512_stop.parquet and out/patterns/pid_4512_segment.parquet
Processing stop time for all trips of pattern 4512
Originally 3067 trips
Trying to process 3054 trips for Pattern 4512 after filtering
Processed 3054 trips for Pattern 4512. There was 0 trip(s) with errors.
Time taken for stop times for 4512: 17.85328807036082 minutes
Writing out/patterns/pid_7445_stop.parquet and out/patterns/pid_7445_segment.parquet
Processing stop time for all trips of pattern 7445
Originally 471 trips
Trying to process 466 trips for Pattern 7445 after filtering
Processed 466 trips for Pattern 7445. There was 0 trip(s) with errors.
Time taken for stop times for 7445: 0.18286243677139283 minutes
Writing out/patterns/pid_17464_stop.parquet and out/patterns/pid_17464_segment.parquet
Processing stop time for all trips of pattern 17464
Originally 467 trips
Trying to process 458 trips for Pattern 17464 after filtering
Processed 448 trips for Pattern 17464. There was 5 trip(s) with errors.
Time taken for stop times for 17464: 0.2092473864555359 minutes
Writing out/patterns/pid_13244_stop.parquet and out/patterns/pid_13244_segment.parquet
Processing stop time for all trips of pattern 13244
Originally 170 trips
Trying to process 169 trips for Pattern 13244 after filtering
Processed 169 trips for Pattern 13244. There was 0 trip(s) with errors.
Time taken for stop times for 13244: 0.07279469966888427 minutes
Writing out/patterns/pid_4320_stop.parquet and out/patterns/pid_4320_segment.parquet
Processing stop time for all trips of pattern 4320
Originally 246 trips
Trying to process 246 trips for Pattern 4320 after filtering
Processed 246 trips for Pattern 4320. There was 0 trip(s) with errors.
Time taken for stop times for 4320: 0.09487263361612956 minutes
Writing out/patterns/pid_7740_stop.parquet and out/patterns/pid_7740_segment.parquet
Processing stop time for all trips of pattern 7740
Originally 15657 trips
Trying to process 15572 trips for Pattern 7740 after filtering
Processed 15565 trips for Pattern 7740. There was 2 trip(s) with errors.
Time taken for stop times for 7740: 7.021789864699046 minutes
Writing out/patterns/pid_19394_stop.parquet and out/patterns/pid_19394_segment.parquet
Processing stop time for all trips of pattern 19394
Originally 865 trips
Trying to process 862 trips for Pattern 19394 after filtering
Processed 862 trips for Pattern 19394. There was 0 trip(s) with errors.
Time taken for stop times for 19394: 0.3701095819473267 minutes
Do not have pattern 13252 available currently
Writing out/patterns/pid_11682_stop.parquet and out/patterns/pid_11682_segment.parquet
Processing stop time for all trips of pattern 11682
Originally 687 trips
Trying to process 677 trips for Pattern 11682 after filtering
Processed 670 trips for Pattern 11682. There was 3 trip(s) with errors.
Time taken for stop times for 11682: 0.3029555161794027 minutes
Writing out/patterns/pid_7471_stop.parquet and out/patterns/pid_7471_segment.parquet
Processing stop time for all trips of pattern 7471
Originally 28641 trips
Trying to process 28089 trips for Pattern 7471 after filtering
Processed 27953 trips for Pattern 7471. There was 43 trip(s) with errors.
Time taken for stop times for 7471: 46.78928482929866 minutes
Writing out/patterns/pid_8418_stop.parquet and out/patterns/pid_8418_segment.parquet
Processing stop time for all trips of pattern 8418
Originally 4228 trips
Trying to process 4200 trips for Pattern 8418 after filtering
Processed 4199 trips for Pattern 8418. There was 0 trip(s) with errors.
Time taken for stop times for 8418: 18.386923384666442 minutes
Writing out/patterns/pid_5019_stop.parquet and out/patterns/pid_5019_segment.parquet
Processing stop time for all trips of pattern 5019
Originally 975 trips
Trying to process 678 trips for Pattern 5019 after filtering
Processed 601 trips for Pattern 5019. There was 10 trip(s) with errors.
Time taken for stop times for 5019: 0.17528575261433918 minutes
Writing out/patterns/pid_5528_stop.parquet and out/patterns/pid_5528_segment.parquet
Processing stop time for all trips of pattern 5528
Originally 4315 trips
Trying to process 4261 trips for Pattern 5528 after filtering
Processed 4245 trips for Pattern 5528. There was 6 trip(s) with errors.
Time taken for stop times for 5528: 11.300940465927123 minutes
Writing out/patterns/pid_20427_stop.parquet and out/patterns/pid_20427_segment.parquet
Processing stop time for all trips of pattern 20427
Originally 2542 trips
Trying to process 2250 trips for Pattern 20427 after filtering
Processed 2065 trips for Pattern 20427. There was 91 trip(s) with errors.
Time taken for stop times for 20427: 0.9918231169382731 minutes
Writing out/patterns/pid_13249_stop.parquet and out/patterns/pid_13249_segment.parquet
Processing stop time for all trips of pattern 13249
Originally 763 trips
Trying to process 749 trips for Pattern 13249 after filtering
Processed 746 trips for Pattern 13249. There was 1 trip(s) with errors.
Time taken for stop times for 13249: 0.30673221747080487 minutes
Writing out/patterns/pid_14122_stop.parquet and out/patterns/pid_14122_segment.parquet
Processing stop time for all trips of pattern 14122
Originally 1521 trips
Trying to process 1079 trips for Pattern 14122 after filtering
Processed 1059 trips for Pattern 14122. There was 4 trip(s) with errors.
Time taken for stop times for 14122: 0.2457511027654012 minutes
Writing out/patterns/pid_1598_stop.parquet and out/patterns/pid_1598_segment.parquet
Processing stop time for all trips of pattern 1598
Originally 19361 trips
Trying to process 19045 trips for Pattern 1598 after filtering
Processed 18996 trips for Pattern 1598. There was 20 trip(s) with errors.
Time taken for stop times for 1598: 9.71795380115509 minutes
Do not have pattern 14101 available currently
Writing out/patterns/pid_7205_stop.parquet and out/patterns/pid_7205_segment.parquet
Processing stop time for all trips of pattern 7205
Originally 2520 trips
Trying to process 2489 trips for Pattern 7205 after filtering
Processed 2478 trips for Pattern 7205. There was 5 trip(s) with errors.
Time taken for stop times for 7205: 16.86081216732661 minutes
Writing out/patterns/pid_7105_stop.parquet and out/patterns/pid_7105_segment.parquet
Processing stop time for all trips of pattern 7105
Originally 1412 trips
Trying to process 1399 trips for Pattern 7105 after filtering
Processed 1397 trips for Pattern 7105. There was 1 trip(s) with errors.
Time taken for stop times for 7105: 0.5113450487454733 minutes
Writing out/patterns/pid_5618_stop.parquet and out/patterns/pid_5618_segment.parquet
Processing stop time for all trips of pattern 5618
Originally 288 trips
Trying to process 281 trips for Pattern 5618 after filtering
Processed 281 trips for Pattern 5618. There was 0 trip(s) with errors.
Time taken for stop times for 5618: 0.08272294600804647 minutes
Writing out/patterns/pid_5567_stop.parquet and out/patterns/pid_5567_segment.parquet
Processing stop time for all trips of pattern 5567
Originally 11631 trips
Trying to process 11563 trips for Pattern 5567 after filtering
Processed 11550 trips for Pattern 5567. There was 5 trip(s) with errors.
Time taken for stop times for 5567: 20.73053276538849 minutes
Writing out/patterns/pid_2165_stop.parquet and out/patterns/pid_2165_segment.parquet
Processing stop time for all trips of pattern 2165
Originally 5730 trips
Trying to process 5488 trips for Pattern 2165 after filtering
Processed 5482 trips for Pattern 2165. There was 1 trip(s) with errors.
Time taken for stop times for 2165: 1.658984382947286 minutes
Writing out/patterns/pid_7177_stop.parquet and out/patterns/pid_7177_segment.parquet
Processing stop time for all trips of pattern 7177
Originally 7 trips
Trying to process 7 trips for Pattern 7177 after filtering
Processed 7 trips for Pattern 7177. There was 0 trip(s) with errors.
Time taken for stop times for 7177: 0.003150478998819987 minutes
Writing out/patterns/pid_7124_stop.parquet and out/patterns/pid_7124_segment.parquet
Processing stop time for all trips of pattern 7124
Originally 706 trips
Trying to process 700 trips for Pattern 7124 after filtering
Processed 698 trips for Pattern 7124. There was 1 trip(s) with errors.
Time taken for stop times for 7124: 0.3337709983189901 minutes
Writing out/patterns/pid_2721_stop.parquet and out/patterns/pid_2721_segment.parquet
Processing stop time for all trips of pattern 2721
Originally 189 trips
Trying to process 186 trips for Pattern 2721 after filtering
Processed 185 trips for Pattern 2721. There was 0 trip(s) with errors.
Time taken for stop times for 2721: 0.0655109961827596 minutes
Writing out/patterns/pid_8470_stop.parquet and out/patterns/pid_8470_segment.parquet
Processing stop time for all trips of pattern 8470
Originally 7297 trips
Trying to process 7177 trips for Pattern 8470 after filtering
Processed 7080 trips for Pattern 8470. There was 38 trip(s) with errors.
Time taken for stop times for 8470: 2.9687093337376913 minutes
Writing out/patterns/pid_2106_stop.parquet and out/patterns/pid_2106_segment.parquet
Processing stop time for all trips of pattern 2106
Originally 2708 trips
Trying to process 2698 trips for Pattern 2106 after filtering
Processed 2697 trips for Pattern 2106. There was 0 trip(s) with errors.
Time taken for stop times for 2106: 0.7941967209180196 minutes
Writing out/patterns/pid_3781_stop.parquet and out/patterns/pid_3781_segment.parquet
Processing stop time for all trips of pattern 3781
Originally 186 trips
Trying to process 184 trips for Pattern 3781 after filtering
Processed 183 trips for Pattern 3781. There was 0 trip(s) with errors.
Time taken for stop times for 3781: 0.05237549940745036 minutes
Writing out/patterns/pid_8130_stop.parquet and out/patterns/pid_8130_segment.parquet
Processing stop time for all trips of pattern 8130
Originally 168 trips
Trying to process 167 trips for Pattern 8130 after filtering
Processed 167 trips for Pattern 8130. There was 0 trip(s) with errors.
Time taken for stop times for 8130: 0.059469481309254967 minutes
Writing out/patterns/pid_10930_stop.parquet and out/patterns/pid_10930_segment.parquet
Processing stop time for all trips of pattern 10930
Originally 26214 trips
Trying to process 25670 trips for Pattern 10930 after filtering
Processed 25533 trips for Pattern 10930. There was 63 trip(s) with errors.
Time taken for stop times for 10930: 38.429434637228645 minutes
Writing out/patterns/pid_1803_stop.parquet and out/patterns/pid_1803_segment.parquet
Processing stop time for all trips of pattern 1803
Originally 151 trips
Trying to process 146 trips for Pattern 1803 after filtering
Processed 146 trips for Pattern 1803. There was 0 trip(s) with errors.
Time taken for stop times for 1803: 0.04979936679204305 minutes
Writing out/patterns/pid_2105_stop.parquet and out/patterns/pid_2105_segment.parquet
Processing stop time for all trips of pattern 2105
Originally 2464 trips
Trying to process 2453 trips for Pattern 2105 after filtering
Processed 2447 trips for Pattern 2105. There was 3 trip(s) with errors.
Time taken for stop times for 2105: 0.8443950533866882 minutes
Writing out/patterns/pid_7456_stop.parquet and out/patterns/pid_7456_segment.parquet
Processing stop time for all trips of pattern 7456
Originally 18794 trips
Trying to process 18648 trips for Pattern 7456 after filtering
Processed 18620 trips for Pattern 7456. There was 11 trip(s) with errors.
Time taken for stop times for 7456: 7.681203897794088 minutes
Writing out/patterns/pid_6771_stop.parquet and out/patterns/pid_6771_segment.parquet
Processing stop time for all trips of pattern 6771
Originally 750 trips
Trying to process 641 trips for Pattern 6771 after filtering
Processed 639 trips for Pattern 6771. There was 1 trip(s) with errors.
Time taken for stop times for 6771: 0.17979394992192585 minutes
Writing out/patterns/pid_4563_stop.parquet and out/patterns/pid_4563_segment.parquet
Processing stop time for all trips of pattern 4563
Originally 157 trips
Trying to process 156 trips for Pattern 4563 after filtering
Processed 156 trips for Pattern 4563. There was 0 trip(s) with errors.
Time taken for stop times for 4563: 0.05709528128306071 minutes
Writing out/patterns/pid_7450_stop.parquet and out/patterns/pid_7450_segment.parquet
Processing stop time for all trips of pattern 7450
Originally 720 trips
Trying to process 717 trips for Pattern 7450 after filtering
Processed 714 trips for Pattern 7450. There was 1 trip(s) with errors.
Time taken for stop times for 7450: 0.3010477701822917 minutes
Writing out/patterns/pid_7171_stop.parquet and out/patterns/pid_7171_segment.parquet
Processing stop time for all trips of pattern 7171
Originally 13496 trips
Trying to process 13416 trips for Pattern 7171 after filtering
Processed 13383 trips for Pattern 7171. There was 10 trip(s) with errors.
Time taken for stop times for 7171: 25.348405877749126 minutes
Do not have pattern 6597 available currently
Writing out/patterns/pid_4318_stop.parquet and out/patterns/pid_4318_segment.parquet
Processing stop time for all trips of pattern 4318
Originally 9445 trips
Trying to process 9368 trips for Pattern 4318 after filtering
Processed 9352 trips for Pattern 4318. There was 7 trip(s) with errors.
Time taken for stop times for 4318: 19.45404188235601 minutes
Writing out/patterns/pid_9247_stop.parquet and out/patterns/pid_9247_segment.parquet
Processing stop time for all trips of pattern 9247
Originally 16924 trips
Trying to process 16649 trips for Pattern 9247 after filtering
Processed 16384 trips for Pattern 9247. There was 118 trip(s) with errors.
Time taken for stop times for 9247: 7.812345417340596 minutes
Do not have pattern 2268 available currently
Do not have pattern 6979 available currently
Writing out/patterns/pid_6598_stop.parquet and out/patterns/pid_6598_segment.parquet
Processing stop time for all trips of pattern 6598
Originally 2299 trips
Trying to process 2279 trips for Pattern 6598 after filtering
Processed 2273 trips for Pattern 6598. There was 3 trip(s) with errors.
Time taken for stop times for 6598: 1.1156365871429443 minutes
Writing out/patterns/pid_8416_stop.parquet and out/patterns/pid_8416_segment.parquet
Processing stop time for all trips of pattern 8416
Originally 321 trips
Trying to process 314 trips for Pattern 8416 after filtering
Processed 310 trips for Pattern 8416. There was 2 trip(s) with errors.
Time taken for stop times for 8416: 0.11844613154729207 minutes
Writing out/patterns/pid_3411_stop.parquet and out/patterns/pid_3411_segment.parquet
Processing stop time for all trips of pattern 3411
Originally 694 trips
Trying to process 677 trips for Pattern 3411 after filtering
Processed 658 trips for Pattern 3411. There was 1 trip(s) with errors.
Time taken for stop times for 3411: 0.1816029151280721 minutes
Do not have pattern 7466 available currently
Writing out/patterns/pid_6399_stop.parquet and out/patterns/pid_6399_segment.parquet
Processing stop time for all trips of pattern 6399
Originally 678 trips
Trying to process 671 trips for Pattern 6399 after filtering
Processed 669 trips for Pattern 6399. There was 0 trip(s) with errors.
Time taken for stop times for 6399: 0.2512365460395813 minutes
Writing out/patterns/pid_7010_stop.parquet and out/patterns/pid_7010_segment.parquet
Processing stop time for all trips of pattern 7010
Originally 1739 trips
Trying to process 1604 trips for Pattern 7010 after filtering
Processed 1588 trips for Pattern 7010. There was 8 trip(s) with errors.
Time taken for stop times for 7010: 0.4097429354985555 minutes
Writing out/patterns/pid_8104_stop.parquet and out/patterns/pid_8104_segment.parquet
Processing stop time for all trips of pattern 8104
Originally 356 trips
Trying to process 348 trips for Pattern 8104 after filtering
Processed 348 trips for Pattern 8104. There was 0 trip(s) with errors.
Time taken for stop times for 8104: 0.14196333487828572 minutes
Writing out/patterns/pid_9889_stop.parquet and out/patterns/pid_9889_segment.parquet
Processing stop time for all trips of pattern 9889
Originally 146 trips
Trying to process 144 trips for Pattern 9889 after filtering
Processed 144 trips for Pattern 9889. There was 0 trip(s) with errors.
Time taken for stop times for 9889: 0.04529652198155721 minutes
Writing out/patterns/pid_7029_stop.parquet and out/patterns/pid_7029_segment.parquet
Processing stop time for all trips of pattern 7029
Originally 27005 trips
Trying to process 26501 trips for Pattern 7029 after filtering
Processed 26388 trips for Pattern 7029. There was 48 trip(s) with errors.
Time taken for stop times for 7029: 11.976171366373698 minutes
Writing out/patterns/pid_1463_stop.parquet and out/patterns/pid_1463_segment.parquet
Processing stop time for all trips of pattern 1463
Originally 736 trips
Trying to process 724 trips for Pattern 1463 after filtering
Processed 695 trips for Pattern 1463. There was 10 trip(s) with errors.
Time taken for stop times for 1463: 0.21137096881866455 minutes
Writing out/patterns/pid_4718_stop.parquet and out/patterns/pid_4718_segment.parquet
Processing stop time for all trips of pattern 4718
Originally 3188 trips
Trying to process 2991 trips for Pattern 4718 after filtering
Processed 2971 trips for Pattern 4718. There was 5 trip(s) with errors.
Time taken for stop times for 4718: 1.1249535481135051 minutes
Do not have pattern 19391 available currently
Writing out/patterns/pid_12432_stop.parquet and out/patterns/pid_12432_segment.parquet
Processing stop time for all trips of pattern 12432
Originally 4152 trips
Trying to process 4129 trips for Pattern 12432 after filtering
Processed 4126 trips for Pattern 12432. There was 1 trip(s) with errors.
Time taken for stop times for 12432: 1.829845949014028 minutes
Do not have pattern 8098 available currently
Writing out/patterns/pid_8228_stop.parquet and out/patterns/pid_8228_segment.parquet
Processing stop time for all trips of pattern 8228
Originally 15306 trips
Trying to process 15210 trips for Pattern 8228 after filtering
Processed 15175 trips for Pattern 8228. There was 17 trip(s) with errors.
Time taken for stop times for 8228: 25.611558802922566 minutes
Writing out/patterns/pid_758_stop.parquet and out/patterns/pid_758_segment.parquet
Processing stop time for all trips of pattern 758
Originally 364 trips
Trying to process 363 trips for Pattern 758 after filtering
Processed 360 trips for Pattern 758. There was 1 trip(s) with errors.
Time taken for stop times for 758: 0.13220343192418416 minutes
Writing out/patterns/pid_1522_stop.parquet and out/patterns/pid_1522_segment.parquet
Processing stop time for all trips of pattern 1522
Originally 1966 trips
Trying to process 1884 trips for Pattern 1522 after filtering
Processed 1875 trips for Pattern 1522. There was 4 trip(s) with errors.
Time taken for stop times for 1522: 0.5187374353408813 minutes
Writing out/patterns/pid_1316_stop.parquet and out/patterns/pid_1316_segment.parquet
Processing stop time for all trips of pattern 1316
Originally 130 trips
Trying to process 128 trips for Pattern 1316 after filtering
Processed 126 trips for Pattern 1316. There was 1 trip(s) with errors.
Time taken for stop times for 1316: 4.356680750846863 minutes
Writing out/patterns/pid_10920_stop.parquet and out/patterns/pid_10920_segment.parquet
Processing stop time for all trips of pattern 10920
Originally 195 trips
Trying to process 191 trips for Pattern 10920 after filtering
Processed 190 trips for Pattern 10920. There was 0 trip(s) with errors.
Time taken for stop times for 10920: 0.09458306630452475 minutes
Writing out/patterns/pid_9365_stop.parquet and out/patterns/pid_9365_segment.parquet
Processing stop time for all trips of pattern 9365
Originally 1734 trips
Trying to process 1687 trips for Pattern 9365 after filtering
Processed 1673 trips for Pattern 9365. There was 6 trip(s) with errors.
Time taken for stop times for 9365: 0.9291142662366231 minutes
Writing out/patterns/pid_5676_stop.parquet and out/patterns/pid_5676_segment.parquet
Processing stop time for all trips of pattern 5676
Originally 11331 trips
Trying to process 10902 trips for Pattern 5676 after filtering
Processed 10794 trips for Pattern 5676. There was 47 trip(s) with errors.
Time taken for stop times for 5676: 19.468473450342813 minutes
Writing out/patterns/pid_6656_stop.parquet and out/patterns/pid_6656_segment.parquet
Processing stop time for all trips of pattern 6656
Originally 18788 trips
Trying to process 18278 trips for Pattern 6656 after filtering
Processed 18140 trips for Pattern 6656. There was 64 trip(s) with errors.
Time taken for stop times for 6656: 24.23184393644333 minutes
Writing out/patterns/pid_7449_stop.parquet and out/patterns/pid_7449_segment.parquet
Processing stop time for all trips of pattern 7449
Originally 5246 trips
Trying to process 5185 trips for Pattern 7449 after filtering
Processed 5163 trips for Pattern 7449. There was 9 trip(s) with errors.
Time taken for stop times for 7449: 3.231282333532969 minutes
Writing out/patterns/pid_10922_stop.parquet and out/patterns/pid_10922_segment.parquet
Processing stop time for all trips of pattern 10922
Originally 727 trips
Trying to process 713 trips for Pattern 10922 after filtering
Processed 709 trips for Pattern 10922. There was 2 trip(s) with errors.
Time taken for stop times for 10922: 0.25260873635609943 minutes
Writing out/patterns/pid_3041_stop.parquet and out/patterns/pid_3041_segment.parquet
Processing stop time for all trips of pattern 3041
Originally 20225 trips
Trying to process 20025 trips for Pattern 3041 after filtering
Processed 19977 trips for Pattern 3041. There was 22 trip(s) with errors.
Time taken for stop times for 3041: 60.910730838775635 minutes
Writing out/patterns/pid_10917_stop.parquet and out/patterns/pid_10917_segment.parquet
Processing stop time for all trips of pattern 10917
Originally 27412 trips
Trying to process 26667 trips for Pattern 10917 after filtering
Processed 26474 trips for Pattern 10917. There was 87 trip(s) with errors.
Time taken for stop times for 10917: 17.50832208395004 minutes
Writing out/patterns/pid_8103_stop.parquet and out/patterns/pid_8103_segment.parquet
Processing stop time for all trips of pattern 8103
Originally 611 trips
Trying to process 610 trips for Pattern 8103 after filtering
Processed 610 trips for Pattern 8103. There was 0 trip(s) with errors.
Time taken for stop times for 8103: 0.2401161829630534 minutes
Writing out/patterns/pid_4525_stop.parquet and out/patterns/pid_4525_segment.parquet
Processing stop time for all trips of pattern 4525
Originally 1617 trips
Trying to process 1591 trips for Pattern 4525 after filtering
Processed 1568 trips for Pattern 4525. There was 11 trip(s) with errors.
Time taken for stop times for 4525: 0.6460756381352742 minutes
Writing out/patterns/pid_5670_stop.parquet and out/patterns/pid_5670_segment.parquet
Processing stop time for all trips of pattern 5670
Originally 1516 trips
Trying to process 497 trips for Pattern 5670 after filtering
Processed 497 trips for Pattern 5670. There was 0 trip(s) with errors.
Time taken for stop times for 5670: 0.13014402786890666 minutes
Writing out/patterns/pid_4573_stop.parquet and out/patterns/pid_4573_segment.parquet
Processing stop time for all trips of pattern 4573
Originally 248 trips
Trying to process 247 trips for Pattern 4573 after filtering
Processed 247 trips for Pattern 4573. There was 0 trip(s) with errors.
Time taken for stop times for 4573: 0.09485214551289876 minutes
Do not have pattern 10942 available currently
Do not have pattern 19387 available currently
Writing out/patterns/pid_19380_stop.parquet and out/patterns/pid_19380_segment.parquet
Processing stop time for all trips of pattern 19380
Originally 9496 trips
Trying to process 9190 trips for Pattern 19380 after filtering
/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/Interpolation.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trip_df.loc[:, "data_time"] = pd.to_datetime(trip_df.data_time)
Processed 9110 trips for Pattern 19380. There was 37 trip(s) with errors.
Time taken for stop times for 19380: 5.463810364405314 minutes
Writing out/patterns/pid_3915_stop.parquet and out/patterns/pid_3915_segment.parquet
Processing stop time for all trips of pattern 3915
Originally 917 trips
Trying to process 903 trips for Pattern 3915 after filtering
Processed 903 trips for Pattern 3915. There was 0 trip(s) with errors.
Time taken for stop times for 3915: 0.28006515502929685 minutes
Writing out/patterns/pid_7135_stop.parquet and out/patterns/pid_7135_segment.parquet
Processing stop time for all trips of pattern 7135
Originally 14698 trips
Trying to process 14319 trips for Pattern 7135 after filtering
Processed 14247 trips for Pattern 7135. There was 28 trip(s) with errors.
Time taken for stop times for 7135: 3.8349742690722146 minutes
Writing out/patterns/pid_941_stop.parquet and out/patterns/pid_941_segment.parquet
Processing stop time for all trips of pattern 941
Originally 2852 trips
Trying to process 2134 trips for Pattern 941 after filtering
Processed 2125 trips for Pattern 941. There was 2 trip(s) with errors.
Time taken for stop times for 941: 0.5127740661303203 minutes
Writing out/patterns/pid_7901_stop.parquet and out/patterns/pid_7901_segment.parquet
Processing stop time for all trips of pattern 7901
Originally 1374 trips
Trying to process 1012 trips for Pattern 7901 after filtering
Processed 815 trips for Pattern 7901. There was 98 trip(s) with errors.
Time taken for stop times for 7901: 0.25294501384099327 minutes
Writing out/patterns/pid_6402_stop.parquet and out/patterns/pid_6402_segment.parquet
Processing stop time for all trips of pattern 6402
Originally 4618 trips
Trying to process 4589 trips for Pattern 6402 after filtering
Processed 4574 trips for Pattern 6402. There was 6 trip(s) with errors.
Time taken for stop times for 6402: 1.6288617690404257 minutes
Writing out/patterns/pid_6126_stop.parquet and out/patterns/pid_6126_segment.parquet
Processing stop time for all trips of pattern 6126
Originally 20743 trips
Trying to process 20484 trips for Pattern 6126 after filtering
Processed 20307 trips for Pattern 6126. There was 87 trip(s) with errors.
Time taken for stop times for 6126: 9.260500264167785 minutes
Writing out/patterns/pid_10954_stop.parquet and out/patterns/pid_10954_segment.parquet
Processing stop time for all trips of pattern 10954
Originally 899 trips
Trying to process 893 trips for Pattern 10954 after filtering
Processed 891 trips for Pattern 10954. There was 1 trip(s) with errors.
Time taken for stop times for 10954: 0.30838036934534707 minutes
Writing out/patterns/pid_7871_stop.parquet and out/patterns/pid_7871_segment.parquet
Processing stop time for all trips of pattern 7871
Originally 8968 trips
Trying to process 8806 trips for Pattern 7871 after filtering
Processed 8781 trips for Pattern 7871. There was 9 trip(s) with errors.
Time taken for stop times for 7871: 3.360490846633911 minutes
Writing out/patterns/pid_4510_stop.parquet and out/patterns/pid_4510_segment.parquet
Processing stop time for all trips of pattern 4510
Originally 4380 trips
Trying to process 4349 trips for Pattern 4510 after filtering
Processed 4342 trips for Pattern 4510. There was 2 trip(s) with errors.
Time taken for stop times for 4510: 1.5720111807187398 minutes
Writing out/patterns/pid_5911_stop.parquet and out/patterns/pid_5911_segment.parquet
Processing stop time for all trips of pattern 5911
Originally 1935 trips
Trying to process 1904 trips for Pattern 5911 after filtering
Processed 1891 trips for Pattern 5911. There was 6 trip(s) with errors.
Time taken for stop times for 5911: 0.7939847509066263 minutes
Writing out/patterns/pid_4608_stop.parquet and out/patterns/pid_4608_segment.parquet
Processing stop time for all trips of pattern 4608
Originally 828 trips
Trying to process 750 trips for Pattern 4608 after filtering
Processed 668 trips for Pattern 4608. There was 1 trip(s) with errors.
Time taken for stop times for 4608: 0.2197385311126709 minutes
Writing out/patterns/pid_3466_stop.parquet and out/patterns/pid_3466_segment.parquet
Processing stop time for all trips of pattern 3466
Originally 184 trips
Trying to process 182 trips for Pattern 3466 after filtering
Processed 182 trips for Pattern 3466. There was 0 trip(s) with errors.
Time taken for stop times for 3466: 0.05197138388951619 minutes
Writing out/patterns/pid_1168_stop.parquet and out/patterns/pid_1168_segment.parquet
Processing stop time for all trips of pattern 1168
Originally 1607 trips
Trying to process 448 trips for Pattern 1168 after filtering
Processed 439 trips for Pattern 1168. There was 4 trip(s) with errors.
Time taken for stop times for 1168: 0.10671588579813639 minutes
Writing out/patterns/pid_8414_stop.parquet and out/patterns/pid_8414_segment.parquet
Processing stop time for all trips of pattern 8414
Originally 1341 trips
Trying to process 906 trips for Pattern 8414 after filtering
Processed 904 trips for Pattern 8414. There was 1 trip(s) with errors.
Time taken for stop times for 8414: 0.21852426528930663 minutes
Writing out/patterns/pid_7804_stop.parquet and out/patterns/pid_7804_segment.parquet
Processing stop time for all trips of pattern 7804
Originally 6284 trips
Trying to process 6246 trips for Pattern 7804 after filtering
Processed 6234 trips for Pattern 7804. There was 6 trip(s) with errors.
Time taken for stop times for 7804: 2.1509777148564657 minutes
Writing out/patterns/pid_6358_stop.parquet and out/patterns/pid_6358_segment.parquet
Processing stop time for all trips of pattern 6358
Originally 22729 trips
Trying to process 22530 trips for Pattern 6358 after filtering
Processed 22469 trips for Pattern 6358. There was 29 trip(s) with errors.
Time taken for stop times for 6358: 8.880558085441589 minutes
Writing out/patterns/pid_1938_stop.parquet and out/patterns/pid_1938_segment.parquet
Processing stop time for all trips of pattern 1938
Originally 2108 trips
Trying to process 2106 trips for Pattern 1938 after filtering
Processed 2106 trips for Pattern 1938. There was 0 trip(s) with errors.
Time taken for stop times for 1938: 0.7549445867538452 minutes
Writing out/patterns/pid_1192_stop.parquet and out/patterns/pid_1192_segment.parquet
Processing stop time for all trips of pattern 1192
Originally 21377 trips
Trying to process 21174 trips for Pattern 1192 after filtering
Processed 21109 trips for Pattern 1192. There was 26 trip(s) with errors.
Time taken for stop times for 1192: 9.904782950878143 minutes
Writing out/patterns/pid_5616_stop.parquet and out/patterns/pid_5616_segment.parquet
Processing stop time for all trips of pattern 5616
Originally 279 trips
Trying to process 277 trips for Pattern 5616 after filtering
Processed 277 trips for Pattern 5616. There was 0 trip(s) with errors.
Time taken for stop times for 5616: 0.1376366337140401 minutes
Writing out/patterns/pid_8129_stop.parquet and out/patterns/pid_8129_segment.parquet
Processing stop time for all trips of pattern 8129
Originally 155 trips
Trying to process 153 trips for Pattern 8129 after filtering
Processed 151 trips for Pattern 8129. There was 1 trip(s) with errors.
Time taken for stop times for 8129: 0.06733428637186686 minutes
Writing out/patterns/pid_1682_stop.parquet and out/patterns/pid_1682_segment.parquet
Processing stop time for all trips of pattern 1682
Originally 16747 trips
Trying to process 16684 trips for Pattern 1682 after filtering
Processed 16652 trips for Pattern 1682. There was 12 trip(s) with errors.
Time taken for stop times for 1682: 7.082221531867981 minutes
Writing out/patterns/pid_6414_stop.parquet and out/patterns/pid_6414_segment.parquet
Processing stop time for all trips of pattern 6414
Originally 19658 trips
Trying to process 19021 trips for Pattern 6414 after filtering
Processed 18703 trips for Pattern 6414. There was 152 trip(s) with errors.
Time taken for stop times for 6414: 10.359044003486634 minutes
Writing out/patterns/pid_1017_stop.parquet and out/patterns/pid_1017_segment.parquet
Processing stop time for all trips of pattern 1017
Originally 248 trips
Trying to process 243 trips for Pattern 1017 after filtering
Processed 238 trips for Pattern 1017. There was 2 trip(s) with errors.
Time taken for stop times for 1017: 0.1480404535929362 minutes
Writing out/patterns/pid_8109_stop.parquet and out/patterns/pid_8109_segment.parquet
Processing stop time for all trips of pattern 8109
Originally 213 trips
Trying to process 212 trips for Pattern 8109 after filtering
Processed 212 trips for Pattern 8109. There was 0 trip(s) with errors.
Time taken for stop times for 8109: 0.06342583100001017 minutes
Writing out/patterns/pid_1523_stop.parquet and out/patterns/pid_1523_segment.parquet
Processing stop time for all trips of pattern 1523
Originally 1836 trips
Trying to process 1825 trips for Pattern 1523 after filtering
Processed 1825 trips for Pattern 1523. There was 0 trip(s) with errors.
Time taken for stop times for 1523: 0.7374158024787902 minutes
Writing out/patterns/pid_7109_stop.parquet and out/patterns/pid_7109_segment.parquet
Processing stop time for all trips of pattern 7109
Originally 696 trips
Trying to process 691 trips for Pattern 7109 after filtering
Processed 689 trips for Pattern 7109. There was 1 trip(s) with errors.
Time taken for stop times for 7109: 0.3806459148724874 minutes
Writing out/patterns/pid_754_stop.parquet and out/patterns/pid_754_segment.parquet
Processing stop time for all trips of pattern 754
Originally 29348 trips
Trying to process 28091 trips for Pattern 754 after filtering
Processed 27805 trips for Pattern 754. There was 136 trip(s) with errors.
Time taken for stop times for 754: 9.945120469729106 minutes
Writing out/patterns/pid_1111_stop.parquet and out/patterns/pid_1111_segment.parquet
Processing stop time for all trips of pattern 1111
Originally 228 trips
Trying to process 218 trips for Pattern 1111 after filtering
Processed 217 trips for Pattern 1111. There was 0 trip(s) with errors.
Time taken for stop times for 1111: 0.06493488550186158 minutes
Writing out/patterns/pid_3208_stop.parquet and out/patterns/pid_3208_segment.parquet
Processing stop time for all trips of pattern 3208
Originally 166 trips
Trying to process 165 trips for Pattern 3208 after filtering
Processed 165 trips for Pattern 3208. There was 0 trip(s) with errors.
Time taken for stop times for 3208: 0.05067290067672729 minutes
Writing out/patterns/pid_7100_stop.parquet and out/patterns/pid_7100_segment.parquet
Processing stop time for all trips of pattern 7100
Originally 2812 trips
Trying to process 2785 trips for Pattern 7100 after filtering
Processed 2774 trips for Pattern 7100. There was 5 trip(s) with errors.
Time taken for stop times for 7100: 0.9840136329332988 minutes
Writing out/patterns/pid_1597_stop.parquet and out/patterns/pid_1597_segment.parquet
Processing stop time for all trips of pattern 1597
Originally 2548 trips
Trying to process 2453 trips for Pattern 1597 after filtering
Processed 2441 trips for Pattern 1597. There was 5 trip(s) with errors.
Time taken for stop times for 1597: 0.9705342332522074 minutes
Writing out/patterns/pid_10911_stop.parquet and out/patterns/pid_10911_segment.parquet
Processing stop time for all trips of pattern 10911
Originally 531 trips
Trying to process 518 trips for Pattern 10911 after filtering
Processed 517 trips for Pattern 10911. There was 0 trip(s) with errors.
Time taken for stop times for 10911: 0.2006689508756002 minutes
Writing out/patterns/pid_9364_stop.parquet and out/patterns/pid_9364_segment.parquet
Processing stop time for all trips of pattern 9364
Originally 862 trips
Trying to process 855 trips for Pattern 9364 after filtering
Processed 849 trips for Pattern 9364. There was 2 trip(s) with errors.
Time taken for stop times for 9364: 0.41136254866917926 minutes
Writing out/patterns/pid_4579_stop.parquet and out/patterns/pid_4579_segment.parquet
Processing stop time for all trips of pattern 4579
Originally 4943 trips
Trying to process 4815 trips for Pattern 4579 after filtering
Processed 4795 trips for Pattern 4579. There was 9 trip(s) with errors.
Time taken for stop times for 4579: 2.2003898143768312 minutes
Writing out/patterns/pid_4107_stop.parquet and out/patterns/pid_4107_segment.parquet
Processing stop time for all trips of pattern 4107
Originally 31015 trips
Trying to process 30425 trips for Pattern 4107 after filtering
Processed 30205 trips for Pattern 4107. There was 101 trip(s) with errors.
Time taken for stop times for 4107: 8.81552593310674 minutes
Writing out/patterns/pid_8179_stop.parquet and out/patterns/pid_8179_segment.parquet
Processing stop time for all trips of pattern 8179
Originally 15122 trips
Trying to process 15004 trips for Pattern 8179 after filtering
Processed 14983 trips for Pattern 8179. There was 9 trip(s) with errors.
Time taken for stop times for 8179: 4.599999896685282 minutes
Writing out/patterns/pid_14132_stop.parquet and out/patterns/pid_14132_segment.parquet
Processing stop time for all trips of pattern 14132
Originally 13186 trips
Trying to process 13049 trips for Pattern 14132 after filtering
Processed 13020 trips for Pattern 14132. There was 12 trip(s) with errors.
Time taken for stop times for 14132: 5.365466364224752 minutes
Writing out/patterns/pid_5057_stop.parquet and out/patterns/pid_5057_segment.parquet
Processing stop time for all trips of pattern 5057
Originally 1333 trips
Trying to process 1278 trips for Pattern 5057 after filtering
Processed 1274 trips for Pattern 5057. There was 2 trip(s) with errors.
Time taken for stop times for 5057: 0.5312011361122131 minutes
Writing out/patterns/pid_1465_stop.parquet and out/patterns/pid_1465_segment.parquet
Processing stop time for all trips of pattern 1465
Originally 1777 trips
Trying to process 1753 trips for Pattern 1465 after filtering
Processed 1744 trips for Pattern 1465. There was 3 trip(s) with errors.
Time taken for stop times for 1465: 0.7198397000630696 minutes
Writing out/patterns/pid_1449_stop.parquet and out/patterns/pid_1449_segment.parquet
Processing stop time for all trips of pattern 1449
Originally 347 trips
Trying to process 180 trips for Pattern 1449 after filtering
Processed 173 trips for Pattern 1449. There was 3 trip(s) with errors.
Time taken for stop times for 1449: 0.05662786563237508 minutes
Writing out/patterns/pid_10934_stop.parquet and out/patterns/pid_10934_segment.parquet
Processing stop time for all trips of pattern 10934
Originally 28246 trips
Trying to process 27628 trips for Pattern 10934 after filtering
/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/Interpolation.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trip_df.loc[:, "data_time"] = pd.to_datetime(trip_df.data_time)
Processed 27481 trips for Pattern 10934. There was 52 trip(s) with errors.
Time taken for stop times for 10934: 9.430787603060404 minutes
Writing out/patterns/pid_949_stop.parquet and out/patterns/pid_949_segment.parquet
Processing stop time for all trips of pattern 949
Originally 4709 trips
Trying to process 4428 trips for Pattern 949 after filtering
Processed 4421 trips for Pattern 949. There was 3 trip(s) with errors.
Time taken for stop times for 949: 1.411542014280955 minutes
Writing out/patterns/pid_9808_stop.parquet and out/patterns/pid_9808_segment.parquet
Processing stop time for all trips of pattern 9808
Originally 394 trips
Trying to process 387 trips for Pattern 9808 after filtering
Processed 385 trips for Pattern 9808. There was 1 trip(s) with errors.
Time taken for stop times for 9808: 0.14806315104166667 minutes
Writing out/patterns/pid_7101_stop.parquet and out/patterns/pid_7101_segment.parquet
Processing stop time for all trips of pattern 7101
Originally 4280 trips
Trying to process 4225 trips for Pattern 7101 after filtering
Processed 4210 trips for Pattern 7101. There was 5 trip(s) with errors.
Time taken for stop times for 7101: 1.8115048686663309 minutes
Writing out/patterns/pid_1009_stop.parquet and out/patterns/pid_1009_segment.parquet
Processing stop time for all trips of pattern 1009
Originally 151 trips
Trying to process 145 trips for Pattern 1009 after filtering
Processed 144 trips for Pattern 1009. There was 0 trip(s) with errors.
Time taken for stop times for 1009: 0.038133148352305093 minutes
Writing out/patterns/pid_1464_stop.parquet and out/patterns/pid_1464_segment.parquet
Processing stop time for all trips of pattern 1464
Originally 3624 trips
Trying to process 3585 trips for Pattern 1464 after filtering
Processed 3567 trips for Pattern 1464. There was 6 trip(s) with errors.
Time taken for stop times for 1464: 1.4558072050412496 minutes
Writing out/patterns/pid_1651_stop.parquet and out/patterns/pid_1651_segment.parquet
Processing stop time for all trips of pattern 1651
Originally 3419 trips
Trying to process 3361 trips for Pattern 1651 after filtering
Processed 3351 trips for Pattern 1651. There was 5 trip(s) with errors.
Time taken for stop times for 1651: 1.3623440504074096 minutes
Writing out/patterns/pid_5857_stop.parquet and out/patterns/pid_5857_segment.parquet
Processing stop time for all trips of pattern 5857
Originally 13660 trips
Trying to process 13513 trips for Pattern 5857 after filtering
Processed 13448 trips for Pattern 5857. There was 31 trip(s) with errors.
Time taken for stop times for 5857: 6.901774962743123 minutes
Writing out/patterns/pid_8465_stop.parquet and out/patterns/pid_8465_segment.parquet
Processing stop time for all trips of pattern 8465
Originally 246 trips
Trying to process 245 trips for Pattern 8465 after filtering
Processed 245 trips for Pattern 8465. There was 0 trip(s) with errors.
Time taken for stop times for 8465: 0.1093248963356018 minutes
Writing out/patterns/pid_8118_stop.parquet and out/patterns/pid_8118_segment.parquet
Processing stop time for all trips of pattern 8118
Originally 151 trips
Trying to process 146 trips for Pattern 8118 after filtering
Processed 146 trips for Pattern 8118. There was 0 trip(s) with errors.
Time taken for stop times for 8118: 0.039128565788269044 minutes
Writing out/patterns/pid_19390_stop.parquet and out/patterns/pid_19390_segment.parquet
Processing stop time for all trips of pattern 19390
Originally 186 trips
Trying to process 101 trips for Pattern 19390 after filtering
Processed 99 trips for Pattern 19390. There was 1 trip(s) with errors.
Time taken for stop times for 19390: 0.023816184202829997 minutes
Writing out/patterns/pid_2922_stop.parquet and out/patterns/pid_2922_segment.parquet
Processing stop time for all trips of pattern 2922
Originally 283 trips
Trying to process 283 trips for Pattern 2922 after filtering
Processed 283 trips for Pattern 2922. There was 0 trip(s) with errors.
Time taken for stop times for 2922: 0.09948537747065227 minutes
Writing out/patterns/pid_7559_stop.parquet and out/patterns/pid_7559_segment.parquet
Processing stop time for all trips of pattern 7559
Originally 2378 trips
Trying to process 2369 trips for Pattern 7559 after filtering
Processed 2367 trips for Pattern 7559. There was 1 trip(s) with errors.
Time taken for stop times for 7559: 1.3994452357292175 minutes
Writing out/patterns/pid_10902_stop.parquet and out/patterns/pid_10902_segment.parquet
Processing stop time for all trips of pattern 10902
Originally 32070 trips
Trying to process 31559 trips for Pattern 10902 after filtering
Processed 31497 trips for Pattern 10902. There was 23 trip(s) with errors.
Time taken for stop times for 10902: 20.511196800072987 minutes
Writing out/patterns/pid_1291_stop.parquet and out/patterns/pid_1291_segment.parquet
Processing stop time for all trips of pattern 1291
Originally 614 trips
Trying to process 602 trips for Pattern 1291 after filtering
Processed 598 trips for Pattern 1291. There was 1 trip(s) with errors.
Time taken for stop times for 1291: 0.24446893135706585 minutes
Writing out/patterns/pid_1748_stop.parquet and out/patterns/pid_1748_segment.parquet
Processing stop time for all trips of pattern 1748
Originally 146 trips
Trying to process 139 trips for Pattern 1748 after filtering
Processed 139 trips for Pattern 1748. There was 0 trip(s) with errors.
Time taken for stop times for 1748: 0.042394765218098956 minutes
Writing out/patterns/pid_4511_stop.parquet and out/patterns/pid_4511_segment.parquet
Processing stop time for all trips of pattern 4511
Originally 2585 trips
Trying to process 2554 trips for Pattern 4511 after filtering
Processed 2542 trips for Pattern 4511. There was 6 trip(s) with errors.
Time taken for stop times for 4511: 0.8299706856409709 minutes
Do not have pattern 11677 available currently
Writing out/patterns/pid_10931_stop.parquet and out/patterns/pid_10931_segment.parquet
Processing stop time for all trips of pattern 10931
Originally 7168 trips
Trying to process 7112 trips for Pattern 10931 after filtering
Processed 7095 trips for Pattern 10931. There was 8 trip(s) with errors.
Time taken for stop times for 10931: 2.674705215295156 minutes
Writing out/patterns/pid_7904_stop.parquet and out/patterns/pid_7904_segment.parquet
Processing stop time for all trips of pattern 7904
Originally 150 trips
Trying to process 150 trips for Pattern 7904 after filtering
Processed 148 trips for Pattern 7904. There was 1 trip(s) with errors.
Time taken for stop times for 7904: 0.06415803035100301 minutes
Writing out/patterns/pid_8233_stop.parquet and out/patterns/pid_8233_segment.parquet
Processing stop time for all trips of pattern 8233
Originally 664 trips
Trying to process 654 trips for Pattern 8233 after filtering
Processed 652 trips for Pattern 8233. There was 1 trip(s) with errors.
Time taken for stop times for 8233: 0.25643900235493977 minutes
Writing out/patterns/pid_5211_stop.parquet and out/patterns/pid_5211_segment.parquet
Processing stop time for all trips of pattern 5211
Originally 3835 trips
Trying to process 3767 trips for Pattern 5211 after filtering
Processed 3734 trips for Pattern 5211. There was 12 trip(s) with errors.
Time taken for stop times for 5211: 2.455093185106913 minutes
Writing out/patterns/pid_3879_stop.parquet and out/patterns/pid_3879_segment.parquet
Processing stop time for all trips of pattern 3879
Originally 995 trips
Trying to process 956 trips for Pattern 3879 after filtering
Processed 952 trips for Pattern 3879. There was 1 trip(s) with errors.
Time taken for stop times for 3879: 0.2651658018430074 minutes
Writing out/patterns/pid_6655_stop.parquet and out/patterns/pid_6655_segment.parquet
Processing stop time for all trips of pattern 6655
Originally 3162 trips
Trying to process 3114 trips for Pattern 6655 after filtering
Processed 3110 trips for Pattern 6655. There was 1 trip(s) with errors.
Time taken for stop times for 6655: 0.9001810868581136 minutes
Writing out/patterns/pid_7028_stop.parquet and out/patterns/pid_7028_segment.parquet
Processing stop time for all trips of pattern 7028
Originally 1308 trips
Trying to process 1296 trips for Pattern 7028 after filtering
Processed 1290 trips for Pattern 7028. There was 3 trip(s) with errors.
Time taken for stop times for 7028: 0.5457540512084961 minutes
Writing out/patterns/pid_714_stop.parquet and out/patterns/pid_714_segment.parquet
Processing stop time for all trips of pattern 714
Originally 249 trips
Trying to process 232 trips for Pattern 714 after filtering
Processed 228 trips for Pattern 714. There was 2 trip(s) with errors.
Time taken for stop times for 714: 0.06631948153177897 minutes
Writing out/patterns/pid_720_stop.parquet and out/patterns/pid_720_segment.parquet
Processing stop time for all trips of pattern 720
Originally 159 trips
Trying to process 157 trips for Pattern 720 after filtering
Processed 157 trips for Pattern 720. There was 0 trip(s) with errors.
Time taken for stop times for 720: 0.0446459690729777 minutes
Writing out/patterns/pid_1681_stop.parquet and out/patterns/pid_1681_segment.parquet
Processing stop time for all trips of pattern 1681
Originally 16826 trips
Trying to process 16763 trips for Pattern 1681 after filtering
Processed 16743 trips for Pattern 1681. There was 7 trip(s) with errors.
Time taken for stop times for 1681: 5.648598062992096 minutes
Writing out/patterns/pid_21484_stop.parquet and out/patterns/pid_21484_segment.parquet
Processing stop time for all trips of pattern 21484
Originally 71 trips
Trying to process 71 trips for Pattern 21484 after filtering
Processed 71 trips for Pattern 21484. There was 0 trip(s) with errors.
Time taken for stop times for 21484: 0.03886260191599528 minutes
Writing out/patterns/pid_5569_stop.parquet and out/patterns/pid_5569_segment.parquet
Processing stop time for all trips of pattern 5569
Originally 11853 trips
Trying to process 11761 trips for Pattern 5569 after filtering
Processed 11748 trips for Pattern 5569. There was 5 trip(s) with errors.
Time taken for stop times for 5569: 5.865852149327596 minutes
Writing out/patterns/pid_3467_stop.parquet and out/patterns/pid_3467_segment.parquet
Processing stop time for all trips of pattern 3467
Originally 326 trips
Trying to process 322 trips for Pattern 3467 after filtering
Processed 322 trips for Pattern 3467. There was 0 trip(s) with errors.
Time taken for stop times for 3467: 0.09651690324147542 minutes
Writing out/patterns/pid_1664_stop.parquet and out/patterns/pid_1664_segment.parquet
Processing stop time for all trips of pattern 1664
Originally 1473 trips
Trying to process 1433 trips for Pattern 1664 after filtering
Processed 1425 trips for Pattern 1664. There was 4 trip(s) with errors.
Time taken for stop times for 1664: 0.4118027130762736 minutes
Writing out/patterns/pid_6658_stop.parquet and out/patterns/pid_6658_segment.parquet
Processing stop time for all trips of pattern 6658
Originally 1304 trips
Trying to process 1287 trips for Pattern 6658 after filtering
Processed 1282 trips for Pattern 6658. There was 2 trip(s) with errors.
Time taken for stop times for 6658: 0.6956817348798116 minutes
Writing out/patterns/pid_6364_stop.parquet and out/patterns/pid_6364_segment.parquet
Processing stop time for all trips of pattern 6364
Originally 2434 trips
Trying to process 2376 trips for Pattern 6364 after filtering
Processed 2347 trips for Pattern 6364. There was 10 trip(s) with errors.
Time taken for stop times for 6364: 0.5866483648618063 minutes
Writing out/patterns/pid_7555_stop.parquet and out/patterns/pid_7555_segment.parquet
Processing stop time for all trips of pattern 7555
Originally 14048 trips
Trying to process 13928 trips for Pattern 7555 after filtering
Processed 13907 trips for Pattern 7555. There was 7 trip(s) with errors.
Time taken for stop times for 7555: 7.029939651489258 minutes
Writing out/patterns/pid_4108_stop.parquet and out/patterns/pid_4108_segment.parquet
Processing stop time for all trips of pattern 4108
Originally 31531 trips
Trying to process 30872 trips for Pattern 4108 after filtering
Processed 30778 trips for Pattern 4108. There was 39 trip(s) with errors.
Time taken for stop times for 4108: 9.092043395837148 minutes
Writing out/patterns/pid_4522_stop.parquet and out/patterns/pid_4522_segment.parquet
Processing stop time for all trips of pattern 4522
Originally 2360 trips
Trying to process 2342 trips for Pattern 4522 after filtering
Processed 2326 trips for Pattern 4522. There was 6 trip(s) with errors.
Time taken for stop times for 4522: 0.8166613141695659 minutes
Writing out/patterns/pid_3519_stop.parquet and out/patterns/pid_3519_segment.parquet
Processing stop time for all trips of pattern 3519
Originally 304 trips
Trying to process 303 trips for Pattern 3519 after filtering
Processed 303 trips for Pattern 3519. There was 0 trip(s) with errors.
Time taken for stop times for 3519: 0.11409856875737508 minutes
Writing out/patterns/pid_750_stop.parquet and out/patterns/pid_750_segment.parquet
Processing stop time for all trips of pattern 750
Originally 1626 trips
Trying to process 1547 trips for Pattern 750 after filtering
Processed 1537 trips for Pattern 750. There was 2 trip(s) with errors.
Time taken for stop times for 750: 0.4790084481239319 minutes
Writing out/patterns/pid_7348_stop.parquet and out/patterns/pid_7348_segment.parquet
Processing stop time for all trips of pattern 7348
Originally 177 trips
Trying to process 173 trips for Pattern 7348 after filtering
Processed 173 trips for Pattern 7348. There was 0 trip(s) with errors.
Time taken for stop times for 7348: 0.05903221766153972 minutes
Writing out/patterns/pid_4135_stop.parquet and out/patterns/pid_4135_segment.parquet
Processing stop time for all trips of pattern 4135
Originally 27935 trips
Trying to process 27389 trips for Pattern 4135 after filtering
Processed 27295 trips for Pattern 4135. There was 42 trip(s) with errors.
Time taken for stop times for 4135: 10.801533647378285 minutes
Writing out/patterns/pid_6351_stop.parquet and out/patterns/pid_6351_segment.parquet
Processing stop time for all trips of pattern 6351
Originally 6639 trips
Trying to process 6588 trips for Pattern 6351 after filtering
Processed 6568 trips for Pattern 6351. There was 9 trip(s) with errors.
Time taken for stop times for 6351: 2.054240636030833 minutes
Writing out/patterns/pid_2404_stop.parquet and out/patterns/pid_2404_segment.parquet
Processing stop time for all trips of pattern 2404
Originally 9831 trips
Trying to process 9768 trips for Pattern 2404 after filtering
Processed 9736 trips for Pattern 2404. There was 14 trip(s) with errors.
Time taken for stop times for 2404: 4.515604217847188 minutes
Writing out/patterns/pid_2945_stop.parquet and out/patterns/pid_2945_segment.parquet
Processing stop time for all trips of pattern 2945
Originally 20775 trips
Trying to process 20478 trips for Pattern 2945 after filtering
Processed 20424 trips for Pattern 2945. There was 21 trip(s) with errors.
Time taken for stop times for 2945: 8.427324151992797 minutes
Writing out/patterns/pid_1054_stop.parquet and out/patterns/pid_1054_segment.parquet
Processing stop time for all trips of pattern 1054
Originally 257 trips
Trying to process 256 trips for Pattern 1054 after filtering
Processed 255 trips for Pattern 1054. There was 0 trip(s) with errors.
Time taken for stop times for 1054: 0.12181098461151123 minutes
Writing out/patterns/pid_10943_stop.parquet and out/patterns/pid_10943_segment.parquet
Processing stop time for all trips of pattern 10943
Originally 29627 trips
Trying to process 29137 trips for Pattern 10943 after filtering
Processed 28991 trips for Pattern 10943. There was 58 trip(s) with errors.
Time taken for stop times for 10943: 12.09385568300883 minutes
Writing out/patterns/pid_4369_stop.parquet and out/patterns/pid_4369_segment.parquet
Processing stop time for all trips of pattern 4369
Originally 16158 trips
Trying to process 16031 trips for Pattern 4369 after filtering
Processed 15986 trips for Pattern 4369. There was 21 trip(s) with errors.
Time taken for stop times for 4369: 6.569337399800618 minutes
Writing out/patterns/pid_4491_stop.parquet and out/patterns/pid_4491_segment.parquet
Processing stop time for all trips of pattern 4491
Originally 6017 trips
Trying to process 5850 trips for Pattern 4491 after filtering
Processed 5775 trips for Pattern 4491. There was 28 trip(s) with errors.
Time taken for stop times for 4491: 1.8463635802268983 minutes
Writing out/patterns/pid_308_stop.parquet and out/patterns/pid_308_segment.parquet
Processing stop time for all trips of pattern 308
Originally 29 trips
Trying to process 24 trips for Pattern 308 after filtering
Processed 21 trips for Pattern 308. There was 1 trip(s) with errors.
Time taken for stop times for 308: 0.010565149784088134 minutes
Writing out/patterns/pid_4288_stop.parquet and out/patterns/pid_4288_segment.parquet
Processing stop time for all trips of pattern 4288
Originally 22631 trips
Trying to process 21776 trips for Pattern 4288 after filtering
Processed 21526 trips for Pattern 4288. There was 102 trip(s) with errors.
Time taken for stop times for 4288: 12.186695416768393 minutes
Writing out/patterns/pid_9368_stop.parquet and out/patterns/pid_9368_segment.parquet
Processing stop time for all trips of pattern 9368
Originally 32001 trips
Trying to process 31336 trips for Pattern 9368 after filtering
Processed 31127 trips for Pattern 9368. There was 92 trip(s) with errors.
Time taken for stop times for 9368: 18.1674698472023 minutes
Do not have pattern 8385 available currently
Writing out/patterns/pid_3498_stop.parquet and out/patterns/pid_3498_segment.parquet
Processing stop time for all trips of pattern 3498
Originally 7427 trips
Trying to process 7372 trips for Pattern 3498 after filtering
Processed 7359 trips for Pattern 3498. There was 4 trip(s) with errors.
Time taken for stop times for 3498: 4.899502329031626 minutes
Writing out/patterns/pid_593_stop.parquet and out/patterns/pid_593_segment.parquet
Processing stop time for all trips of pattern 593
Originally 3469 trips
Trying to process 3277 trips for Pattern 593 after filtering
Processed 3206 trips for Pattern 593. There was 25 trip(s) with errors.
Time taken for stop times for 593: 1.9506483674049377 minutes
Writing out/patterns/pid_5613_stop.parquet and out/patterns/pid_5613_segment.parquet
Processing stop time for all trips of pattern 5613
Originally 1036 trips
Trying to process 1027 trips for Pattern 5613 after filtering
Processed 1027 trips for Pattern 5613. There was 0 trip(s) with errors.
Time taken for stop times for 5613: 0.38143670161565146 minutes
Writing out/patterns/pid_2095_stop.parquet and out/patterns/pid_2095_segment.parquet
Processing stop time for all trips of pattern 2095
Originally 1321 trips
Trying to process 1310 trips for Pattern 2095 after filtering
Processed 1310 trips for Pattern 2095. There was 0 trip(s) with errors.
Time taken for stop times for 2095: 0.49372203747431437 minutes
Do not have pattern 5568 available currently
Writing out/patterns/pid_3937_stop.parquet and out/patterns/pid_3937_segment.parquet
Processing stop time for all trips of pattern 3937
Originally 153 trips
Trying to process 153 trips for Pattern 3937 after filtering
Processed 153 trips for Pattern 3937. There was 0 trip(s) with errors.
Time taken for stop times for 3937: 0.05668025016784668 minutes
Writing out/patterns/pid_7451_stop.parquet and out/patterns/pid_7451_segment.parquet
Processing stop time for all trips of pattern 7451
Originally 19903 trips
Trying to process 18925 trips for Pattern 7451 after filtering
Processed 18711 trips for Pattern 7451. There was 96 trip(s) with errors.
Time taken for stop times for 7451: 8.735135666529338 minutes
Writing out/patterns/pid_4571_stop.parquet and out/patterns/pid_4571_segment.parquet
Processing stop time for all trips of pattern 4571
Originally 16451 trips
Trying to process 16235 trips for Pattern 4571 after filtering
Processed 16222 trips for Pattern 4571. There was 5 trip(s) with errors.
Time taken for stop times for 4571: 7.236257934570313 minutes
Do not have pattern 8466 available currently
Writing out/patterns/pid_9379_stop.parquet and out/patterns/pid_9379_segment.parquet
Processing stop time for all trips of pattern 9379
Originally 146 trips
Trying to process 139 trips for Pattern 9379 after filtering
Processed 139 trips for Pattern 9379. There was 0 trip(s) with errors.
Time taken for stop times for 9379: 0.07763826847076416 minutes
Writing out/patterns/pid_8114_stop.parquet and out/patterns/pid_8114_segment.parquet
Processing stop time for all trips of pattern 8114
Originally 1201 trips
Trying to process 1194 trips for Pattern 8114 after filtering
Processed 1190 trips for Pattern 8114. There was 1 trip(s) with errors.
Time taken for stop times for 8114: 0.4489670515060425 minutes
Writing out/patterns/pid_1512_stop.parquet and out/patterns/pid_1512_segment.parquet
Processing stop time for all trips of pattern 1512
Originally 155 trips
Trying to process 148 trips for Pattern 1512 after filtering
Processed 148 trips for Pattern 1512. There was 0 trip(s) with errors.
Time taken for stop times for 1512: 0.049294932683308916 minutes
Writing out/patterns/pid_6980_stop.parquet and out/patterns/pid_6980_segment.parquet
Processing stop time for all trips of pattern 6980
Originally 1575 trips
Trying to process 1556 trips for Pattern 6980 after filtering
Processed 1544 trips for Pattern 6980. There was 6 trip(s) with errors.
Time taken for stop times for 6980: 0.4567621151606242 minutes
Writing out/patterns/pid_3090_stop.parquet and out/patterns/pid_3090_segment.parquet
Processing stop time for all trips of pattern 3090
Originally 1950 trips
Trying to process 1778 trips for Pattern 3090 after filtering
Processed 1667 trips for Pattern 3090. There was 48 trip(s) with errors.
Time taken for stop times for 3090: 0.7685213843981425 minutes
Writing out/patterns/pid_4603_stop.parquet and out/patterns/pid_4603_segment.parquet
Processing stop time for all trips of pattern 4603
Originally 140 trips
Trying to process 138 trips for Pattern 4603 after filtering
Processed 137 trips for Pattern 4603. There was 0 trip(s) with errors.
Time taken for stop times for 4603: 0.052163898944854736 minutes
Writing out/patterns/pid_1344_stop.parquet and out/patterns/pid_1344_segment.parquet
Processing stop time for all trips of pattern 1344
Originally 2546 trips
Trying to process 2261 trips for Pattern 1344 after filtering
Processed 1963 trips for Pattern 1344. There was 115 trip(s) with errors.
Time taken for stop times for 1344: 0.8474534630775452 minutes
Writing out/patterns/pid_905_stop.parquet and out/patterns/pid_905_segment.parquet
Processing stop time for all trips of pattern 905
Originally 31173 trips
Trying to process 30516 trips for Pattern 905 after filtering
Processed 30310 trips for Pattern 905. There was 94 trip(s) with errors.
Time taken for stop times for 905: 9.657391599814098 minutes
Writing out/patterns/pid_1350_stop.parquet and out/patterns/pid_1350_segment.parquet
Processing stop time for all trips of pattern 1350
Originally 24206 trips
Trying to process 23876 trips for Pattern 1350 after filtering
Processed 23809 trips for Pattern 1350. There was 27 trip(s) with errors.
Time taken for stop times for 1350: 13.096567805608114 minutes
Writing out/patterns/pid_3934_stop.parquet and out/patterns/pid_3934_segment.parquet
Processing stop time for all trips of pattern 3934
Originally 212 trips
Trying to process 207 trips for Pattern 3934 after filtering
Processed 207 trips for Pattern 3934. There was 0 trip(s) with errors.
Time taken for stop times for 3934: 0.08320818344751994 minutes
Do not have pattern 6787 available currently
Writing out/patterns/pid_4371_stop.parquet and out/patterns/pid_4371_segment.parquet
Processing stop time for all trips of pattern 4371
Originally 861 trips
Trying to process 853 trips for Pattern 4371 after filtering
Processed 844 trips for Pattern 4371. There was 4 trip(s) with errors.
Time taken for stop times for 4371: 0.3209657351175944 minutes
Writing out/patterns/pid_1145_stop.parquet and out/patterns/pid_1145_segment.parquet
Processing stop time for all trips of pattern 1145
Originally 3085 trips
Trying to process 1569 trips for Pattern 1145 after filtering
Processed 1553 trips for Pattern 1145. There was 8 trip(s) with errors.
Time taken for stop times for 1145: 0.3715316335360209 minutes
Writing out/patterns/pid_6362_stop.parquet and out/patterns/pid_6362_segment.parquet
Processing stop time for all trips of pattern 6362
Originally 1675 trips
Trying to process 1458 trips for Pattern 6362 after filtering
Processed 1362 trips for Pattern 6362. There was 43 trip(s) with errors.
Time taken for stop times for 6362: 0.33080015182495115 minutes
Writing out/patterns/pid_6982_stop.parquet and out/patterns/pid_6982_segment.parquet
Processing stop time for all trips of pattern 6982
Originally 7455 trips
Trying to process 5989 trips for Pattern 6982 after filtering
/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/Interpolation.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trip_df.loc[:, "data_time"] = pd.to_datetime(trip_df.data_time)
Processed 5946 trips for Pattern 6982. There was 3 trip(s) with errors.
Time taken for stop times for 6982: 1.5135504484176636 minutes
Writing out/patterns/pid_8085_stop.parquet and out/patterns/pid_8085_segment.parquet
Processing stop time for all trips of pattern 8085
Originally 6160 trips
Trying to process 6134 trips for Pattern 8085 after filtering
Processed 6133 trips for Pattern 8085. There was 0 trip(s) with errors.
Time taken for stop times for 8085: 2.0875145276387532 minutes
Writing out/patterns/pid_8380_stop.parquet and out/patterns/pid_8380_segment.parquet
Processing stop time for all trips of pattern 8380
Originally 89 trips
Trying to process 86 trips for Pattern 8380 after filtering
Processed 86 trips for Pattern 8380. There was 0 trip(s) with errors.
Time taken for stop times for 8380: 0.02473855415980021 minutes
Writing out/patterns/pid_4554_stop.parquet and out/patterns/pid_4554_segment.parquet
Processing stop time for all trips of pattern 4554
Originally 2136 trips
Trying to process 2126 trips for Pattern 4554 after filtering
Processed 2120 trips for Pattern 4554. There was 3 trip(s) with errors.
Time taken for stop times for 4554: 0.8351744453112284 minutes
Writing out/patterns/pid_1300_stop.parquet and out/patterns/pid_1300_segment.parquet
Processing stop time for all trips of pattern 1300
Originally 5280 trips
Trying to process 5211 trips for Pattern 1300 after filtering
Processed 5192 trips for Pattern 1300. There was 8 trip(s) with errors.
Time taken for stop times for 1300: 2.009528601169586 minutes
Writing out/patterns/pid_1516_stop.parquet and out/patterns/pid_1516_segment.parquet
Processing stop time for all trips of pattern 1516
Originally 1121 trips
Trying to process 1069 trips for Pattern 1516 after filtering
Processed 1063 trips for Pattern 1516. There was 3 trip(s) with errors.
Time taken for stop times for 1516: 0.30350981950759887 minutes
Writing out/patterns/pid_7556_stop.parquet and out/patterns/pid_7556_segment.parquet
Processing stop time for all trips of pattern 7556
Originally 14004 trips
Trying to process 13883 trips for Pattern 7556 after filtering
Processed 13849 trips for Pattern 7556. There was 15 trip(s) with errors.
Time taken for stop times for 7556: 5.530129750569661 minutes
Writing out/patterns/pid_7864_stop.parquet and out/patterns/pid_7864_segment.parquet
Processing stop time for all trips of pattern 7864
Originally 156 trips
Trying to process 149 trips for Pattern 7864 after filtering
Processed 128 trips for Pattern 7864. There was 6 trip(s) with errors.
Time taken for stop times for 7864: 0.034420883655548094 minutes
Writing out/patterns/pid_8101_stop.parquet and out/patterns/pid_8101_segment.parquet
Processing stop time for all trips of pattern 8101
Originally 265 trips
Trying to process 261 trips for Pattern 8101 after filtering
Processed 259 trips for Pattern 8101. There was 1 trip(s) with errors.
Time taken for stop times for 8101: 0.14795138041178385 minutes
Writing out/patterns/pid_6586_stop.parquet and out/patterns/pid_6586_segment.parquet
Processing stop time for all trips of pattern 6586
Originally 736 trips
Trying to process 726 trips for Pattern 6586 after filtering
Processed 725 trips for Pattern 6586. There was 0 trip(s) with errors.
Time taken for stop times for 6586: 0.2916814684867859 minutes
Writing out/patterns/pid_14110_stop.parquet and out/patterns/pid_14110_segment.parquet
Processing stop time for all trips of pattern 14110
Originally 17289 trips
Trying to process 17026 trips for Pattern 14110 after filtering
Processed 16986 trips for Pattern 14110. There was 16 trip(s) with errors.
Time taken for stop times for 14110: 4.551982231934866 minutes
Writing out/patterns/pid_951_stop.parquet and out/patterns/pid_951_segment.parquet
Processing stop time for all trips of pattern 951
Originally 477 trips
Trying to process 475 trips for Pattern 951 after filtering
Processed 473 trips for Pattern 951. There was 1 trip(s) with errors.
Time taken for stop times for 951: 0.1579970637957255 minutes
Writing out/patterns/pid_5603_stop.parquet and out/patterns/pid_5603_segment.parquet
Processing stop time for all trips of pattern 5603
Originally 2998 trips
Trying to process 2938 trips for Pattern 5603 after filtering
Processed 2938 trips for Pattern 5603. There was 0 trip(s) with errors.
Time taken for stop times for 5603: 1.017604366938273 minutes
Writing out/patterns/pid_3493_stop.parquet and out/patterns/pid_3493_segment.parquet
Processing stop time for all trips of pattern 3493
Originally 1332 trips
Trying to process 756 trips for Pattern 3493 after filtering
Processed 754 trips for Pattern 3493. There was 0 trip(s) with errors.
Time taken for stop times for 3493: 0.18762221733729045 minutes
Writing out/patterns/pid_3501_stop.parquet and out/patterns/pid_3501_segment.parquet
Processing stop time for all trips of pattern 3501
Originally 8111 trips
Trying to process 7982 trips for Pattern 3501 after filtering
Processed 7894 trips for Pattern 3501. There was 39 trip(s) with errors.
Time taken for stop times for 3501: 4.253810167312622 minutes
Writing out/patterns/pid_947_stop.parquet and out/patterns/pid_947_segment.parquet
Processing stop time for all trips of pattern 947
Originally 3843 trips
Trying to process 3771 trips for Pattern 947 after filtering
Processed 3755 trips for Pattern 947. There was 7 trip(s) with errors.
Time taken for stop times for 947: 2.075407079855601 minutes
Writing out/patterns/pid_1661_stop.parquet and out/patterns/pid_1661_segment.parquet
Processing stop time for all trips of pattern 1661
Originally 219 trips
Trying to process 217 trips for Pattern 1661 after filtering
Processed 217 trips for Pattern 1661. There was 0 trip(s) with errors.
Time taken for stop times for 1661: 0.08814956744511922 minutes
Writing out/patterns/pid_1798_stop.parquet and out/patterns/pid_1798_segment.parquet
Processing stop time for all trips of pattern 1798
Originally 1590 trips
Trying to process 377 trips for Pattern 1798 after filtering
Processed 359 trips for Pattern 1798. There was 7 trip(s) with errors.
Time taken for stop times for 1798: 0.0851066549619039 minutes
Writing out/patterns/pid_1977_stop.parquet and out/patterns/pid_1977_segment.parquet
Processing stop time for all trips of pattern 1977
Originally 258 trips
Trying to process 167 trips for Pattern 1977 after filtering
Processed 133 trips for Pattern 1977. There was 13 trip(s) with errors.
Time taken for stop times for 1977: 0.05702614784240723 minutes
Writing out/patterns/pid_9718_stop.parquet and out/patterns/pid_9718_segment.parquet
Processing stop time for all trips of pattern 9718
Originally 202 trips
Trying to process 201 trips for Pattern 9718 after filtering
Processed 199 trips for Pattern 9718. There was 1 trip(s) with errors.
Time taken for stop times for 9718: 0.11313949823379517 minutes
Writing out/patterns/pid_4605_stop.parquet and out/patterns/pid_4605_segment.parquet
Processing stop time for all trips of pattern 4605
Originally 23640 trips
Trying to process 22753 trips for Pattern 4605 after filtering
Processed 22016 trips for Pattern 4605. There was 346 trip(s) with errors.
Time taken for stop times for 4605: 14.81848068634669 minutes
Writing out/patterns/pid_1151_stop.parquet and out/patterns/pid_1151_segment.parquet
Processing stop time for all trips of pattern 1151
Originally 20626 trips
Trying to process 20335 trips for Pattern 1151 after filtering
Processed 20216 trips for Pattern 1151. There was 56 trip(s) with errors.
Time taken for stop times for 1151: 9.972670300801594 minutes
Writing out/patterns/pid_7770_stop.parquet and out/patterns/pid_7770_segment.parquet
Processing stop time for all trips of pattern 7770
Originally 422 trips
Trying to process 415 trips for Pattern 7770 after filtering
Processed 411 trips for Pattern 7770. There was 1 trip(s) with errors.
Time taken for stop times for 7770: 0.1491144299507141 minutes
Writing out/patterns/pid_704_stop.parquet and out/patterns/pid_704_segment.parquet
Processing stop time for all trips of pattern 704
Originally 175 trips
Trying to process 173 trips for Pattern 704 after filtering
Processed 167 trips for Pattern 704. There was 3 trip(s) with errors.
Time taken for stop times for 704: 0.049601399898529054 minutes
Writing out/patterns/pid_14106_stop.parquet and out/patterns/pid_14106_segment.parquet
Processing stop time for all trips of pattern 14106
Originally 614 trips
Trying to process 601 trips for Pattern 14106 after filtering
Processed 599 trips for Pattern 14106. There was 1 trip(s) with errors.
Time taken for stop times for 14106: 0.34600496689478555 minutes
Writing out/patterns/pid_13253_stop.parquet and out/patterns/pid_13253_segment.parquet
Processing stop time for all trips of pattern 13253
Originally 307 trips
Trying to process 278 trips for Pattern 13253 after filtering
Processed 277 trips for Pattern 13253. There was 0 trip(s) with errors.
Time taken for stop times for 13253: 0.06561378240585328 minutes
Writing out/patterns/pid_10951_stop.parquet and out/patterns/pid_10951_segment.parquet
Processing stop time for all trips of pattern 10951
Originally 257 trips
Trying to process 256 trips for Pattern 10951 after filtering
Processed 256 trips for Pattern 10951. There was 0 trip(s) with errors.
Time taken for stop times for 10951: 0.12371561527252198 minutes
Writing out/patterns/pid_7104_stop.parquet and out/patterns/pid_7104_segment.parquet
Processing stop time for all trips of pattern 7104
Originally 15944 trips
Trying to process 15832 trips for Pattern 7104 after filtering
Processed 15798 trips for Pattern 7104. There was 13 trip(s) with errors.
Time taken for stop times for 7104: 6.713406682014465 minutes
Writing out/patterns/pid_16549_stop.parquet and out/patterns/pid_16549_segment.parquet
Processing stop time for all trips of pattern 16549
Originally 20980 trips
Trying to process 20537 trips for Pattern 16549 after filtering
Processed 20397 trips for Pattern 16549. There was 53 trip(s) with errors.
Time taken for stop times for 16549: 9.484110196431478 minutes
Writing out/patterns/pid_2289_stop.parquet and out/patterns/pid_2289_segment.parquet
Processing stop time for all trips of pattern 2289
Originally 25036 trips
Trying to process 24758 trips for Pattern 2289 after filtering
Processed 24674 trips for Pattern 2289. There was 36 trip(s) with errors.
Time taken for stop times for 2289: 8.933585433165232 minutes
Writing out/patterns/pid_956_stop.parquet and out/patterns/pid_956_segment.parquet
Processing stop time for all trips of pattern 956
Originally 5362 trips
Trying to process 5029 trips for Pattern 956 after filtering
Processed 5022 trips for Pattern 956. There was 2 trip(s) with errors.
Time taken for stop times for 956: 1.3233107328414917 minutes
Writing out/patterns/pid_9372_stop.parquet and out/patterns/pid_9372_segment.parquet
Processing stop time for all trips of pattern 9372
Originally 24208 trips
Trying to process 23247 trips for Pattern 9372 after filtering
Processed 22643 trips for Pattern 9372. There was 258 trip(s) with errors.
Time taken for stop times for 9372: 12.215794849395753 minutes
Writing out/patterns/pid_10923_stop.parquet and out/patterns/pid_10923_segment.parquet
Processing stop time for all trips of pattern 10923
Originally 470 trips
Trying to process 454 trips for Pattern 10923 after filtering
Processed 444 trips for Pattern 10923. There was 5 trip(s) with errors.
Time taken for stop times for 10923: 0.2038470188776652 minutes
Writing out/patterns/pid_8111_stop.parquet and out/patterns/pid_8111_segment.parquet
Processing stop time for all trips of pattern 8111
Originally 514 trips
Trying to process 512 trips for Pattern 8111 after filtering
Processed 512 trips for Pattern 8111. There was 0 trip(s) with errors.
Time taken for stop times for 8111: 0.17778329849243163 minutes
Do not have pattern 7861 available currently
Writing out/patterns/pid_4339_stop.parquet and out/patterns/pid_4339_segment.parquet
Processing stop time for all trips of pattern 4339
Originally 363 trips
Trying to process 358 trips for Pattern 4339 after filtering
Processed 358 trips for Pattern 4339. There was 0 trip(s) with errors.
Time taken for stop times for 4339: 0.1395198345184326 minutes
Writing out/patterns/pid_4680_stop.parquet and out/patterns/pid_4680_segment.parquet
Processing stop time for all trips of pattern 4680
Originally 2053 trips
Trying to process 1766 trips for Pattern 4680 after filtering
Processed 1728 trips for Pattern 4680. There was 18 trip(s) with errors.
Time taken for stop times for 4680: 1.0251586159070334 minutes
Writing out/patterns/pid_4362_stop.parquet and out/patterns/pid_4362_segment.parquet
Processing stop time for all trips of pattern 4362
Originally 937 trips
Trying to process 932 trips for Pattern 4362 after filtering
Processed 932 trips for Pattern 4362. There was 0 trip(s) with errors.
Time taken for stop times for 4362: 0.29428648153940834 minutes
Writing out/patterns/pid_1329_stop.parquet and out/patterns/pid_1329_segment.parquet
Processing stop time for all trips of pattern 1329
Originally 164 trips
Trying to process 160 trips for Pattern 1329 after filtering
Processed 160 trips for Pattern 1329. There was 0 trip(s) with errors.
Time taken for stop times for 1329: 0.04767036437988281 minutes
Writing out/patterns/pid_4618_stop.parquet and out/patterns/pid_4618_segment.parquet
Processing stop time for all trips of pattern 4618
Originally 1485 trips
Trying to process 1360 trips for Pattern 4618 after filtering
Processed 1357 trips for Pattern 4618. There was 1 trip(s) with errors.
Time taken for stop times for 4618: 0.4605357845624288 minutes
Do not have pattern 6596 available currently
Writing out/patterns/pid_8125_stop.parquet and out/patterns/pid_8125_segment.parquet
Processing stop time for all trips of pattern 8125
Originally 6673 trips
Trying to process 6601 trips for Pattern 8125 after filtering
Processed 6590 trips for Pattern 8125. There was 4 trip(s) with errors.
Time taken for stop times for 8125: 1.9709585785865784 minutes
Writing out/patterns/pid_7453_stop.parquet and out/patterns/pid_7453_segment.parquet
Processing stop time for all trips of pattern 7453
Originally 175 trips
Trying to process 172 trips for Pattern 7453 after filtering
Processed 170 trips for Pattern 7453. There was 1 trip(s) with errors.
Time taken for stop times for 7453: 0.054844399293263756 minutes
Writing out/patterns/pid_1970_stop.parquet and out/patterns/pid_1970_segment.parquet
Processing stop time for all trips of pattern 1970
Originally 21081 trips
Trying to process 20048 trips for Pattern 1970 after filtering
Processed 19360 trips for Pattern 1970. There was 304 trip(s) with errors.
Time taken for stop times for 1970: 9.144545829296112 minutes
Writing out/patterns/pid_14119_stop.parquet and out/patterns/pid_14119_segment.parquet
Processing stop time for all trips of pattern 14119
Originally 21835 trips
Trying to process 21349 trips for Pattern 14119 after filtering
Processed 21169 trips for Pattern 14119. There was 68 trip(s) with errors.
Time taken for stop times for 14119: 13.285687148571014 minutes
Writing out/patterns/pid_4853_stop.parquet and out/patterns/pid_4853_segment.parquet
Processing stop time for all trips of pattern 4853
Originally 145 trips
Trying to process 142 trips for Pattern 4853 after filtering
Processed 142 trips for Pattern 4853. There was 0 trip(s) with errors.
Time taken for stop times for 4853: 0.05778831640879313 minutes
Writing out/patterns/pid_5736_stop.parquet and out/patterns/pid_5736_segment.parquet
Processing stop time for all trips of pattern 5736
Originally 230 trips
Trying to process 226 trips for Pattern 5736 after filtering
Processed 226 trips for Pattern 5736. There was 0 trip(s) with errors.
Time taken for stop times for 5736: 0.0727200984954834 minutes
Writing out/patterns/pid_15955_stop.parquet and out/patterns/pid_15955_segment.parquet
Processing stop time for all trips of pattern 15955
Originally 2941 trips
Trying to process 2918 trips for Pattern 15955 after filtering
Processed 2915 trips for Pattern 15955. There was 0 trip(s) with errors.
Time taken for stop times for 15955: 1.1382473985354105 minutes
Do not have pattern 19375 available currently
Writing out/patterns/pid_10950_stop.parquet and out/patterns/pid_10950_segment.parquet
Processing stop time for all trips of pattern 10950
Originally 8420 trips
Trying to process 8230 trips for Pattern 10950 after filtering
Processed 8109 trips for Pattern 10950. There was 50 trip(s) with errors.
Time taken for stop times for 10950: 2.1827283978462217 minutes
Writing out/patterns/pid_6485_stop.parquet and out/patterns/pid_6485_segment.parquet
Processing stop time for all trips of pattern 6485
Originally 284 trips
Trying to process 281 trips for Pattern 6485 after filtering
Processed 281 trips for Pattern 6485. There was 0 trip(s) with errors.
Time taken for stop times for 6485: 0.09782508611679078 minutes
Writing out/patterns/pid_2944_stop.parquet and out/patterns/pid_2944_segment.parquet
Processing stop time for all trips of pattern 2944
Originally 18023 trips
Trying to process 17884 trips for Pattern 2944 after filtering
Processed 17860 trips for Pattern 2944. There was 7 trip(s) with errors.
Time taken for stop times for 2944: 7.089272169272105 minutes
Writing out/patterns/pid_18416_stop.parquet and out/patterns/pid_18416_segment.parquet
Processing stop time for all trips of pattern 18416
Originally 446 trips
Trying to process 438 trips for Pattern 18416 after filtering
Processed 438 trips for Pattern 18416. There was 0 trip(s) with errors.
Time taken for stop times for 18416: 0.18910449743270874 minutes
Writing out/patterns/pid_8121_stop.parquet and out/patterns/pid_8121_segment.parquet
Processing stop time for all trips of pattern 8121
Originally 7806 trips
Trying to process 7673 trips for Pattern 8121 after filtering
Processed 7615 trips for Pattern 8121. There was 23 trip(s) with errors.
Time taken for stop times for 8121: 4.407838769753774 minutes
Writing out/patterns/pid_4570_stop.parquet and out/patterns/pid_4570_segment.parquet
Processing stop time for all trips of pattern 4570
Originally 5302 trips
Trying to process 5248 trips for Pattern 4570 after filtering
Processed 5239 trips for Pattern 4570. There was 3 trip(s) with errors.
Time taken for stop times for 4570: 2.2772159536679584 minutes
Writing out/patterns/pid_3213_stop.parquet and out/patterns/pid_3213_segment.parquet
Processing stop time for all trips of pattern 3213
Originally 2245 trips
Trying to process 2221 trips for Pattern 3213 after filtering
Processed 2193 trips for Pattern 3213. There was 14 trip(s) with errors.
Time taken for stop times for 3213: 0.6659039815266927 minutes
Writing out/patterns/pid_15951_stop.parquet and out/patterns/pid_15951_segment.parquet
Processing stop time for all trips of pattern 15951
Originally 319 trips
Trying to process 318 trips for Pattern 15951 after filtering
Processed 318 trips for Pattern 15951. There was 0 trip(s) with errors.
Time taken for stop times for 15951: 0.11704681317011516 minutes
Writing out/patterns/pid_4276_stop.parquet and out/patterns/pid_4276_segment.parquet
Processing stop time for all trips of pattern 4276
Originally 20059 trips
Trying to process 18930 trips for Pattern 4276 after filtering
Processed 18508 trips for Pattern 4276. There was 178 trip(s) with errors.
Time taken for stop times for 4276: 8.588805468877156 minutes
Writing out/patterns/pid_2156_stop.parquet and out/patterns/pid_2156_segment.parquet
Processing stop time for all trips of pattern 2156
Originally 9609 trips
Trying to process 9332 trips for Pattern 2156 after filtering
Processed 9278 trips for Pattern 2156. There was 23 trip(s) with errors.
Time taken for stop times for 2156: 3.5220460017522175 minutes
Writing out/patterns/pid_5425_stop.parquet and out/patterns/pid_5425_segment.parquet
Processing stop time for all trips of pattern 5425
Originally 27605 trips
Trying to process 27004 trips for Pattern 5425 after filtering
Processed 26893 trips for Pattern 5425. There was 50 trip(s) with errors.
Time taken for stop times for 5425: 15.315611330668132 minutes
Writing out/patterns/pid_3929_stop.parquet and out/patterns/pid_3929_segment.parquet
Processing stop time for all trips of pattern 3929
Originally 914 trips
Trying to process 907 trips for Pattern 3929 after filtering
Processed 896 trips for Pattern 3929. There was 4 trip(s) with errors.
Time taken for stop times for 3929: 0.34846498171488444 minutes
Writing out/patterns/pid_18413_stop.parquet and out/patterns/pid_18413_segment.parquet
Processing stop time for all trips of pattern 18413
Originally 740 trips
Trying to process 733 trips for Pattern 18413 after filtering
Processed 733 trips for Pattern 18413. There was 0 trip(s) with errors.
Time taken for stop times for 18413: 0.322818652788798 minutes
Writing out/patterns/pid_9717_stop.parquet and out/patterns/pid_9717_segment.parquet
Processing stop time for all trips of pattern 9717
Originally 39 trips
Trying to process 39 trips for Pattern 9717 after filtering
Processed 39 trips for Pattern 9717. There was 0 trip(s) with errors.
Time taken for stop times for 9717: 0.021330002943674722 minutes
Writing out/patterns/pid_2946_stop.parquet and out/patterns/pid_2946_segment.parquet
Processing stop time for all trips of pattern 2946
Originally 21852 trips
Trying to process 21142 trips for Pattern 2946 after filtering
Processed 20653 trips for Pattern 2946. There was 233 trip(s) with errors.
Time taken for stop times for 2946: 8.344301080703735 minutes
Writing out/patterns/pid_3449_stop.parquet and out/patterns/pid_3449_segment.parquet
Processing stop time for all trips of pattern 3449
Originally 554 trips
Trying to process 542 trips for Pattern 3449 after filtering
Processed 540 trips for Pattern 3449. There was 1 trip(s) with errors.
Time taken for stop times for 3449: 0.1592776854832967 minutes
Writing out/patterns/pid_4561_stop.parquet and out/patterns/pid_4561_segment.parquet
Processing stop time for all trips of pattern 4561
Originally 1909 trips
Trying to process 1885 trips for Pattern 4561 after filtering
Processed 1881 trips for Pattern 4561. There was 2 trip(s) with errors.
Time taken for stop times for 4561: 0.7763609170913697 minutes
Writing out/patterns/pid_4663_stop.parquet and out/patterns/pid_4663_segment.parquet
Processing stop time for all trips of pattern 4663
Originally 4147 trips
Trying to process 4068 trips for Pattern 4663 after filtering
Processed 4048 trips for Pattern 4663. There was 9 trip(s) with errors.
Time taken for stop times for 4663: 1.2321560462315877 minutes
Writing out/patterns/pid_4115_stop.parquet and out/patterns/pid_4115_segment.parquet
Processing stop time for all trips of pattern 4115
Originally 2104 trips
Trying to process 2077 trips for Pattern 4115 after filtering
Processed 2073 trips for Pattern 4115. There was 2 trip(s) with errors.
Time taken for stop times for 4115: 0.7855063160260518 minutes
Writing out/patterns/pid_4556_stop.parquet and out/patterns/pid_4556_segment.parquet
Processing stop time for all trips of pattern 4556
Originally 1061 trips
Trying to process 1046 trips for Pattern 4556 after filtering
Processed 1046 trips for Pattern 4556. There was 0 trip(s) with errors.
Time taken for stop times for 4556: 0.5187593301137289 minutes
Writing out/patterns/pid_4504_stop.parquet and out/patterns/pid_4504_segment.parquet
Processing stop time for all trips of pattern 4504
Originally 203 trips
Trying to process 200 trips for Pattern 4504 after filtering
Processed 198 trips for Pattern 4504. There was 1 trip(s) with errors.
Time taken for stop times for 4504: 0.06480225324630737 minutes
Writing out/patterns/pid_16550_stop.parquet and out/patterns/pid_16550_segment.parquet
Processing stop time for all trips of pattern 16550
Originally 33058 trips
Trying to process 32299 trips for Pattern 16550 after filtering
Processed 32201 trips for Pattern 16550. There was 34 trip(s) with errors.
Time taken for stop times for 16550: 12.113108650843303 minutes
Writing out/patterns/pid_727_stop.parquet and out/patterns/pid_727_segment.parquet
Processing stop time for all trips of pattern 727
Originally 153 trips
Trying to process 153 trips for Pattern 727 after filtering
Processed 153 trips for Pattern 727. There was 0 trip(s) with errors.
Time taken for stop times for 727: 0.05871461629867554 minutes
Writing out/patterns/pid_10925_stop.parquet and out/patterns/pid_10925_segment.parquet
Processing stop time for all trips of pattern 10925
Originally 106 trips
Trying to process 103 trips for Pattern 10925 after filtering
Processed 103 trips for Pattern 10925. There was 0 trip(s) with errors.
Time taken for stop times for 10925: 0.045766234397888184 minutes
Writing out/patterns/pid_6546_stop.parquet and out/patterns/pid_6546_segment.parquet
Processing stop time for all trips of pattern 6546
Originally 9915 trips
Trying to process 9735 trips for Pattern 6546 after filtering
Processed 9676 trips for Pattern 6546. There was 28 trip(s) with errors.
Time taken for stop times for 6546: 4.402610365549723 minutes
Writing out/patterns/pid_1235_stop.parquet and out/patterns/pid_1235_segment.parquet
Processing stop time for all trips of pattern 1235
Originally 4642 trips
Trying to process 4489 trips for Pattern 1235 after filtering
Processed 4478 trips for Pattern 1235. There was 5 trip(s) with errors.
Time taken for stop times for 1235: 1.3527848680814107 minutes
Do not have pattern 7126 available currently
Writing out/patterns/pid_6412_stop.parquet and out/patterns/pid_6412_segment.parquet
Processing stop time for all trips of pattern 6412
Originally 863 trips
Trying to process 850 trips for Pattern 6412 after filtering
Processed 847 trips for Pattern 6412. There was 1 trip(s) with errors.
Time taken for stop times for 6412: 0.41072843472162884 minutes
Writing out/patterns/pid_10905_stop.parquet and out/patterns/pid_10905_segment.parquet
Processing stop time for all trips of pattern 10905
Originally 10034 trips
Trying to process 9991 trips for Pattern 10905 after filtering
Processed 9982 trips for Pattern 10905. There was 4 trip(s) with errors.
Time taken for stop times for 10905: 3.78199858268102 minutes
Writing out/patterns/pid_7389_stop.parquet and out/patterns/pid_7389_segment.parquet
Processing stop time for all trips of pattern 7389
Originally 306 trips
Trying to process 304 trips for Pattern 7389 after filtering
Processed 304 trips for Pattern 7389. There was 0 trip(s) with errors.
Time taken for stop times for 7389: 0.08739305337270101 minutes
Writing out/patterns/pid_12430_stop.parquet and out/patterns/pid_12430_segment.parquet
Processing stop time for all trips of pattern 12430
Originally 801 trips
Trying to process 796 trips for Pattern 12430 after filtering
Processed 796 trips for Pattern 12430. There was 0 trip(s) with errors.
Time taken for stop times for 12430: 0.3465235312779745 minutes
Writing out/patterns/pid_5678_stop.parquet and out/patterns/pid_5678_segment.parquet
Processing stop time for all trips of pattern 5678
Originally 358 trips
Trying to process 355 trips for Pattern 5678 after filtering
Processed 355 trips for Pattern 5678. There was 0 trip(s) with errors.
Time taken for stop times for 5678: 0.12107093334197998 minutes
Writing out/patterns/pid_1457_stop.parquet and out/patterns/pid_1457_segment.parquet
Processing stop time for all trips of pattern 1457
Originally 452 trips
Trying to process 420 trips for Pattern 1457 after filtering
Processed 416 trips for Pattern 1457. There was 2 trip(s) with errors.
Time taken for stop times for 1457: 0.10820018450419108 minutes
Writing out/patterns/pid_1454_stop.parquet and out/patterns/pid_1454_segment.parquet
Processing stop time for all trips of pattern 1454
Originally 2317 trips
Trying to process 2272 trips for Pattern 1454 after filtering
Processed 2259 trips for Pattern 1454. There was 1 trip(s) with errors.
Time taken for stop times for 1454: 1.1670849204063416 minutes
Writing out/patterns/pid_8384_stop.parquet and out/patterns/pid_8384_segment.parquet
Processing stop time for all trips of pattern 8384
Originally 2193 trips
Trying to process 2187 trips for Pattern 8384 after filtering
Processed 2184 trips for Pattern 8384. There was 1 trip(s) with errors.
Time taken for stop times for 8384: 0.932616118590037 minutes
Writing out/patterns/pid_4658_stop.parquet and out/patterns/pid_4658_segment.parquet
Processing stop time for all trips of pattern 4658
Originally 8162 trips
Trying to process 8010 trips for Pattern 4658 after filtering
Processed 7994 trips for Pattern 4658. There was 5 trip(s) with errors.
Time taken for stop times for 4658: 3.403784183661143 minutes
Writing out/patterns/pid_9653_stop.parquet and out/patterns/pid_9653_segment.parquet
Processing stop time for all trips of pattern 9653
Originally 104 trips
Trying to process 102 trips for Pattern 9653 after filtering
Processed 100 trips for Pattern 9653. There was 1 trip(s) with errors.
Time taken for stop times for 9653: 0.048150813579559325 minutes
Writing out/patterns/pid_5355_stop.parquet and out/patterns/pid_5355_segment.parquet
Processing stop time for all trips of pattern 5355
Originally 203 trips
Trying to process 201 trips for Pattern 5355 after filtering
Processed 199 trips for Pattern 5355. There was 1 trip(s) with errors.
Time taken for stop times for 5355: 0.07036097844441731 minutes
Writing out/patterns/pid_4598_stop.parquet and out/patterns/pid_4598_segment.parquet
Processing stop time for all trips of pattern 4598
Originally 2447 trips
Trying to process 2407 trips for Pattern 4598 after filtering
Processed 2385 trips for Pattern 4598. There was 11 trip(s) with errors.
Time taken for stop times for 4598: 0.7301348487536112 minutes
Writing out/patterns/pid_10907_stop.parquet and out/patterns/pid_10907_segment.parquet
Processing stop time for all trips of pattern 10907
Originally 214 trips
Trying to process 212 trips for Pattern 10907 after filtering
Processed 210 trips for Pattern 10907. There was 1 trip(s) with errors.
Time taken for stop times for 10907: 0.08406696319580079 minutes
Writing out/patterns/pid_729_stop.parquet and out/patterns/pid_729_segment.parquet
Processing stop time for all trips of pattern 729
Originally 6867 trips
Trying to process 6790 trips for Pattern 729 after filtering
Processed 6766 trips for Pattern 729. There was 7 trip(s) with errors.
Time taken for stop times for 729: 2.2197817007700604 minutes
Writing out/patterns/pid_1228_stop.parquet and out/patterns/pid_1228_segment.parquet
Processing stop time for all trips of pattern 1228
Originally 31330 trips
Trying to process 31082 trips for Pattern 1228 after filtering
Processed 31032 trips for Pattern 1228. There was 20 trip(s) with errors.
Time taken for stop times for 1228: 14.327526048819225 minutes
Writing out/patterns/pid_14131_stop.parquet and out/patterns/pid_14131_segment.parquet
Processing stop time for all trips of pattern 14131
Originally 671 trips
Trying to process 646 trips for Pattern 14131 after filtering
Processed 639 trips for Pattern 14131. There was 3 trip(s) with errors.
Time taken for stop times for 14131: 0.2331044356028239 minutes
Writing out/patterns/pid_4589_stop.parquet and out/patterns/pid_4589_segment.parquet
Processing stop time for all trips of pattern 4589
Originally 1125 trips
Trying to process 1117 trips for Pattern 4589 after filtering
Processed 1115 trips for Pattern 4589. There was 1 trip(s) with errors.
Time taken for stop times for 4589: 0.3384292205174764 minutes
Writing out/patterns/pid_5674_stop.parquet and out/patterns/pid_5674_segment.parquet
Processing stop time for all trips of pattern 5674
Originally 184 trips
Trying to process 184 trips for Pattern 5674 after filtering
Processed 184 trips for Pattern 5674. There was 0 trip(s) with errors.
Time taken for stop times for 5674: 0.0639009674390157 minutes
Writing out/patterns/pid_3190_stop.parquet and out/patterns/pid_3190_segment.parquet
Processing stop time for all trips of pattern 3190
Originally 1685 trips
Trying to process 1677 trips for Pattern 3190 after filtering
Processed 1667 trips for Pattern 3190. There was 5 trip(s) with errors.
Time taken for stop times for 3190: 0.5670247515042622 minutes
Writing out/patterns/pid_4521_stop.parquet and out/patterns/pid_4521_segment.parquet
Processing stop time for all trips of pattern 4521
Originally 1596 trips
Trying to process 1587 trips for Pattern 4521 after filtering
Processed 1586 trips for Pattern 4521. There was 0 trip(s) with errors.
Time taken for stop times for 4521: 0.5550702015558878 minutes
Writing out/patterns/pid_12433_stop.parquet and out/patterns/pid_12433_segment.parquet
Processing stop time for all trips of pattern 12433
Originally 4078 trips
Trying to process 4034 trips for Pattern 12433 after filtering
Processed 4013 trips for Pattern 12433. There was 9 trip(s) with errors.
Time taken for stop times for 12433: 2.50005126396815 minutes
Writing out/patterns/pid_8181_stop.parquet and out/patterns/pid_8181_segment.parquet
Processing stop time for all trips of pattern 8181
Originally 51 trips
Trying to process 11 trips for Pattern 8181 after filtering
Processed 11 trips for Pattern 8181. There was 0 trip(s) with errors.
Time taken for stop times for 8181: 0.0036825338999430337 minutes
Writing out/patterns/pid_5704_stop.parquet and out/patterns/pid_5704_segment.parquet
Processing stop time for all trips of pattern 5704
Originally 374 trips
Trying to process 370 trips for Pattern 5704 after filtering
Processed 370 trips for Pattern 5704. There was 0 trip(s) with errors.
Time taken for stop times for 5704: 0.188523264726003 minutes
Writing out/patterns/pid_3207_stop.parquet and out/patterns/pid_3207_segment.parquet
Processing stop time for all trips of pattern 3207
Originally 323 trips
Trying to process 192 trips for Pattern 3207 after filtering
Processed -118 trips for Pattern 3207. There was 154 trip(s) with errors.
Time taken for stop times for 3207: 0.03482946554819743 minutes
Writing out/patterns/pid_8415_stop.parquet and out/patterns/pid_8415_segment.parquet
Processing stop time for all trips of pattern 8415
Originally 1432 trips
Trying to process 713 trips for Pattern 8415 after filtering
Processed 708 trips for Pattern 8415. There was 2 trip(s) with errors.
Time taken for stop times for 8415: 0.16898049910863241 minutes
Writing out/patterns/pid_15952_stop.parquet and out/patterns/pid_15952_segment.parquet
Processing stop time for all trips of pattern 15952
Originally 33 trips
Trying to process 32 trips for Pattern 15952 after filtering
Processed 32 trips for Pattern 15952. There was 0 trip(s) with errors.
Time taken for stop times for 15952: 0.011298867066701253 minutes
Writing out/patterns/pid_4274_stop.parquet and out/patterns/pid_4274_segment.parquet
Processing stop time for all trips of pattern 4274
Originally 24 trips
Trying to process 24 trips for Pattern 4274 after filtering
Processed 24 trips for Pattern 4274. There was 0 trip(s) with errors.
Time taken for stop times for 4274: 0.009426430861155192 minutes
Writing out/patterns/pid_3932_stop.parquet and out/patterns/pid_3932_segment.parquet
Processing stop time for all trips of pattern 3932
Originally 27133 trips
Trying to process 26529 trips for Pattern 3932 after filtering
Processed 26401 trips for Pattern 3932. There was 51 trip(s) with errors.
Time taken for stop times for 3932: 12.585225864251454 minutes
Do not have pattern 13246 available currently
Writing out/patterns/pid_2169_stop.parquet and out/patterns/pid_2169_segment.parquet
Processing stop time for all trips of pattern 2169
Originally 20172 trips
Trying to process 20070 trips for Pattern 2169 after filtering
Processed 20043 trips for Pattern 2169. There was 11 trip(s) with errors.
Time taken for stop times for 2169: 10.515460550785065 minutes
Writing out/patterns/pid_4595_stop.parquet and out/patterns/pid_4595_segment.parquet
Processing stop time for all trips of pattern 4595
Originally 15390 trips
Trying to process 15111 trips for Pattern 4595 after filtering
Processed 15004 trips for Pattern 4595. There was 45 trip(s) with errors.
Time taken for stop times for 4595: 5.986352717876434 minutes
Writing out/patterns/pid_14133_stop.parquet and out/patterns/pid_14133_segment.parquet
Processing stop time for all trips of pattern 14133
Originally 489 trips
Trying to process 474 trips for Pattern 14133 after filtering
Processed 430 trips for Pattern 14133. There was 22 trip(s) with errors.
Time taken for stop times for 14133: 0.14483075141906737 minutes
Writing out/patterns/pid_6777_stop.parquet and out/patterns/pid_6777_segment.parquet
Processing stop time for all trips of pattern 6777
Originally 250 trips
Trying to process 248 trips for Pattern 6777 after filtering
Processed 247 trips for Pattern 6777. There was 0 trip(s) with errors.
Time taken for stop times for 6777: 0.10348869959513346 minutes
Do not have pattern 8462 available currently
Writing out/patterns/pid_6522_stop.parquet and out/patterns/pid_6522_segment.parquet
Processing stop time for all trips of pattern 6522
Originally 127 trips
Trying to process 126 trips for Pattern 6522 after filtering
Processed 126 trips for Pattern 6522. There was 0 trip(s) with errors.
Time taken for stop times for 6522: 0.04539964993794759 minutes
Writing out/patterns/pid_19388_stop.parquet and out/patterns/pid_19388_segment.parquet
Processing stop time for all trips of pattern 19388
Originally 385 trips
Trying to process 360 trips for Pattern 19388 after filtering
Processed 332 trips for Pattern 19388. There was 14 trip(s) with errors.
Time taken for stop times for 19388: 0.10087675253550211 minutes
Writing out/patterns/pid_19378_stop.parquet and out/patterns/pid_19378_segment.parquet
Processing stop time for all trips of pattern 19378
Originally 24128 trips
Trying to process 23862 trips for Pattern 19378 after filtering
Processed 23773 trips for Pattern 19378. There was 38 trip(s) with errors.
Time taken for stop times for 19378: 14.092328997453054 minutes
Writing out/patterns/pid_4597_stop.parquet and out/patterns/pid_4597_segment.parquet
Processing stop time for all trips of pattern 4597
Originally 15449 trips
Trying to process 15171 trips for Pattern 4597 after filtering
Processed 14998 trips for Pattern 4597. There was 75 trip(s) with errors.
Time taken for stop times for 4597: 4.730099968115488 minutes
Writing out/patterns/pid_15950_stop.parquet and out/patterns/pid_15950_segment.parquet
Processing stop time for all trips of pattern 15950
Originally 145 trips
Trying to process 144 trips for Pattern 15950 after filtering
Processed 144 trips for Pattern 15950. There was 0 trip(s) with errors.
Time taken for stop times for 15950: 0.06709126631418864 minutes
Writing out/patterns/pid_1299_stop.parquet and out/patterns/pid_1299_segment.parquet
Processing stop time for all trips of pattern 1299
Originally 2203 trips
Trying to process 1992 trips for Pattern 1299 after filtering
Processed 1976 trips for Pattern 1299. There was 4 trip(s) with errors.
Time taken for stop times for 1299: 0.5165175199508667 minutes
Writing out/patterns/pid_1070_stop.parquet and out/patterns/pid_1070_segment.parquet
Processing stop time for all trips of pattern 1070
Originally 163 trips
Trying to process 144 trips for Pattern 1070 after filtering
Processed 144 trips for Pattern 1070. There was 0 trip(s) with errors.
Time taken for stop times for 1070: 0.039871466159820554 minutes
Writing out/patterns/pid_6652_stop.parquet and out/patterns/pid_6652_segment.parquet
Processing stop time for all trips of pattern 6652
Originally 1982 trips
Trying to process 1953 trips for Pattern 6652 after filtering
Processed 1953 trips for Pattern 6652. There was 0 trip(s) with errors.
Time taken for stop times for 6652: 0.618633234500885 minutes
Writing out/patterns/pid_7014_stop.parquet and out/patterns/pid_7014_segment.parquet
Processing stop time for all trips of pattern 7014
Originally 17956 trips
Trying to process 17835 trips for Pattern 7014 after filtering
Processed 17807 trips for Pattern 7014. There was 13 trip(s) with errors.
Time taken for stop times for 7014: 7.807380779584249 minutes
Writing out/patterns/pid_3464_stop.parquet and out/patterns/pid_3464_segment.parquet
Processing stop time for all trips of pattern 3464
Originally 170 trips
Trying to process 170 trips for Pattern 3464 after filtering
Processed 170 trips for Pattern 3464. There was 0 trip(s) with errors.
Time taken for stop times for 3464: 0.06110179821650187 minutes
Writing out/patterns/pid_1706_stop.parquet and out/patterns/pid_1706_segment.parquet
Processing stop time for all trips of pattern 1706
Originally 2973 trips
Trying to process 2698 trips for Pattern 1706 after filtering
Processed 2673 trips for Pattern 1706. There was 10 trip(s) with errors.
Time taken for stop times for 1706: 0.7170080502827962 minutes
Writing out/patterns/pid_904_stop.parquet and out/patterns/pid_904_segment.parquet
Processing stop time for all trips of pattern 904
Originally 30878 trips
Trying to process 30270 trips for Pattern 904 after filtering
Processed 30164 trips for Pattern 904. There was 40 trip(s) with errors.
Time taken for stop times for 904: 9.864525465170543 minutes
Writing out/patterns/pid_1190_stop.parquet and out/patterns/pid_1190_segment.parquet
Processing stop time for all trips of pattern 1190
Originally 228 trips
Trying to process 219 trips for Pattern 1190 after filtering
Processed 210 trips for Pattern 1190. There was 2 trip(s) with errors.
Time taken for stop times for 1190: 0.09602930148442586 minutes
Writing out/patterns/pid_8420_stop.parquet and out/patterns/pid_8420_segment.parquet
Processing stop time for all trips of pattern 8420
Originally 26736 trips
Trying to process 26300 trips for Pattern 8420 after filtering
Processed 26219 trips for Pattern 8420. There was 37 trip(s) with errors.
Time taken for stop times for 8420: 13.035109468301137 minutes
Writing out/patterns/pid_19397_stop.parquet and out/patterns/pid_19397_segment.parquet
Processing stop time for all trips of pattern 19397
Originally 329 trips
Trying to process 327 trips for Pattern 19397 after filtering
Processed 326 trips for Pattern 19397. There was 0 trip(s) with errors.
Time taken for stop times for 19397: 0.12441040277481079 minutes
Writing out/patterns/pid_4557_stop.parquet and out/patterns/pid_4557_segment.parquet
Processing stop time for all trips of pattern 4557
Originally 20901 trips
Trying to process 20598 trips for Pattern 4557 after filtering
Processed 20549 trips for Pattern 4557. There was 19 trip(s) with errors.
Time taken for stop times for 4557: 12.694411786397298 minutes
Writing out/patterns/pid_978_stop.parquet and out/patterns/pid_978_segment.parquet
Processing stop time for all trips of pattern 978
Originally 316 trips
Trying to process 307 trips for Pattern 978 after filtering
Processed 305 trips for Pattern 978. There was 0 trip(s) with errors.
Time taken for stop times for 978: 0.08679304917653402 minutes
Writing out/patterns/pid_3215_stop.parquet and out/patterns/pid_3215_segment.parquet
Processing stop time for all trips of pattern 3215
Originally 14964 trips
Trying to process 14521 trips for Pattern 3215 after filtering
Processed 14448 trips for Pattern 3215. There was 25 trip(s) with errors.
Time taken for stop times for 3215: 4.438258715470632 minutes
Writing out/patterns/pid_3751_stop.parquet and out/patterns/pid_3751_segment.parquet
Processing stop time for all trips of pattern 3751
Originally 155 trips
Trying to process 151 trips for Pattern 3751 after filtering
Processed 149 trips for Pattern 3751. There was 1 trip(s) with errors.
Time taken for stop times for 3751: 0.04525827964146932 minutes
Writing out/patterns/pid_8119_stop.parquet and out/patterns/pid_8119_segment.parquet
Processing stop time for all trips of pattern 8119
Originally 7707 trips
Trying to process 7459 trips for Pattern 8119 after filtering
Processed 7400 trips for Pattern 8119. There was 22 trip(s) with errors.
Time taken for stop times for 8119: 4.455751001834869 minutes
Writing out/patterns/pid_4519_stop.parquet and out/patterns/pid_4519_segment.parquet
Processing stop time for all trips of pattern 4519
Originally 2757 trips
Trying to process 2736 trips for Pattern 4519 after filtering
Processed 2728 trips for Pattern 4519. There was 4 trip(s) with errors.
Time taken for stop times for 4519: 0.9072128812472026 minutes
Writing out/patterns/pid_3881_stop.parquet and out/patterns/pid_3881_segment.parquet
Processing stop time for all trips of pattern 3881
Originally 851 trips
Trying to process 800 trips for Pattern 3881 after filtering
Processed 799 trips for Pattern 3881. There was 0 trip(s) with errors.
Time taken for stop times for 3881: 0.2648613135019938 minutes
Writing out/patterns/pid_4337_stop.parquet and out/patterns/pid_4337_segment.parquet
Processing stop time for all trips of pattern 4337
Originally 300 trips
Trying to process 299 trips for Pattern 4337 after filtering
Processed 297 trips for Pattern 4337. There was 1 trip(s) with errors.
Time taken for stop times for 4337: 0.11593446731567383 minutes
Writing out/patterns/pid_5570_stop.parquet and out/patterns/pid_5570_segment.parquet
Processing stop time for all trips of pattern 5570
Originally 17717 trips
Trying to process 17562 trips for Pattern 5570 after filtering
Processed 17543 trips for Pattern 5570. There was 8 trip(s) with errors.
Time taken for stop times for 5570: 11.844908618927002 minutes
Writing out/patterns/pid_6984_stop.parquet and out/patterns/pid_6984_segment.parquet
Processing stop time for all trips of pattern 6984
Originally 162 trips
Trying to process 162 trips for Pattern 6984 after filtering
Processed 162 trips for Pattern 6984. There was 0 trip(s) with errors.
Time taken for stop times for 6984: 0.07430176734924317 minutes
Writing out/patterns/pid_19367_stop.parquet and out/patterns/pid_19367_segment.parquet
Processing stop time for all trips of pattern 19367
Originally 495 trips
Trying to process 491 trips for Pattern 19367 after filtering
Processed 484 trips for Pattern 19367. There was 3 trip(s) with errors.
Time taken for stop times for 19367: 0.1565640648206075 minutes
Writing out/patterns/pid_19383_stop.parquet and out/patterns/pid_19383_segment.parquet
Processing stop time for all trips of pattern 19383
Originally 6208 trips
Trying to process 6165 trips for Pattern 19383 after filtering
Processed 6156 trips for Pattern 19383. There was 3 trip(s) with errors.
Time taken for stop times for 19383: 3.703487920761108 minutes
Writing out/patterns/pid_6538_stop.parquet and out/patterns/pid_6538_segment.parquet
Processing stop time for all trips of pattern 6538
Originally 147 trips
Trying to process 147 trips for Pattern 6538 after filtering
Processed 147 trips for Pattern 6538. There was 0 trip(s) with errors.
Time taken for stop times for 6538: 0.06124961773554484 minutes
Writing out/patterns/pid_7911_stop.parquet and out/patterns/pid_7911_segment.parquet
Processing stop time for all trips of pattern 7911
Originally 2962 trips
Trying to process 2938 trips for Pattern 7911 after filtering
Processed 2930 trips for Pattern 7911. There was 3 trip(s) with errors.
Time taken for stop times for 7911: 1.2729057629903158 minutes
Writing out/patterns/pid_8468_stop.parquet and out/patterns/pid_8468_segment.parquet
Processing stop time for all trips of pattern 8468
Originally 859 trips
Trying to process 851 trips for Pattern 8468 after filtering
Processed 851 trips for Pattern 8468. There was 0 trip(s) with errors.
Time taken for stop times for 8468: 0.28692883253097534 minutes
Do not have pattern 8730 available currently
Writing out/patterns/pid_7108_stop.parquet and out/patterns/pid_7108_segment.parquet
Processing stop time for all trips of pattern 7108
Originally 839 trips
Trying to process 830 trips for Pattern 7108 after filtering
Processed 828 trips for Pattern 7108. There was 1 trip(s) with errors.
Time taken for stop times for 7108: 0.4000943342844645 minutes
Writing out/patterns/pid_9370_stop.parquet and out/patterns/pid_9370_segment.parquet
Processing stop time for all trips of pattern 9370
Originally 709 trips
Trying to process 700 trips for Pattern 9370 after filtering
Processed 692 trips for Pattern 9370. There was 4 trip(s) with errors.
Time taken for stop times for 9370: 0.3505156834920247 minutes
Do not have pattern 17466 available currently
Writing out/patterns/pid_7102_stop.parquet and out/patterns/pid_7102_segment.parquet
Processing stop time for all trips of pattern 7102
Originally 632 trips
Trying to process 627 trips for Pattern 7102 after filtering
Processed 625 trips for Pattern 7102. There was 1 trip(s) with errors.
Time taken for stop times for 7102: 0.28607680002848307 minutes
Writing out/patterns/pid_8142_stop.parquet and out/patterns/pid_8142_segment.parquet
Processing stop time for all trips of pattern 8142
Originally 1356 trips
Trying to process 1346 trips for Pattern 8142 after filtering
Processed 1344 trips for Pattern 8142. There was 1 trip(s) with errors.
Time taken for stop times for 8142: 1.030260717868805 minutes
Writing out/patterns/pid_7130_stop.parquet and out/patterns/pid_7130_segment.parquet
Processing stop time for all trips of pattern 7130
Originally 167 trips
Trying to process 157 trips for Pattern 7130 after filtering
Processed 156 trips for Pattern 7130. There was 0 trip(s) with errors.
Time taken for stop times for 7130: 0.052088216940561934 minutes
Do not have pattern 4244 available currently
Writing out/patterns/pid_14117_stop.parquet and out/patterns/pid_14117_segment.parquet
Processing stop time for all trips of pattern 14117
Originally 24587 trips
Trying to process 24381 trips for Pattern 14117 after filtering
/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/Interpolation.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trip_df.loc[:, "data_time"] = pd.to_datetime(trip_df.data_time)
Processed 24345 trips for Pattern 14117. There was 14 trip(s) with errors.
Time taken for stop times for 14117: 9.95144366423289 minutes
Writing out/patterns/pid_2104_stop.parquet and out/patterns/pid_2104_segment.parquet
Processing stop time for all trips of pattern 2104
Originally 5586 trips
Trying to process 5513 trips for Pattern 2104 after filtering
Processed 5482 trips for Pattern 2104. There was 15 trip(s) with errors.
Time taken for stop times for 2104: 2.1369154175122578 minutes
Do not have pattern 1043 available currently
Writing out/patterns/pid_706_stop.parquet and out/patterns/pid_706_segment.parquet
Processing stop time for all trips of pattern 706
Originally 157 trips
Trying to process 153 trips for Pattern 706 after filtering
Processed 144 trips for Pattern 706. There was 4 trip(s) with errors.
Time taken for stop times for 706: 0.045246434211730954 minutes
Writing out/patterns/pid_4588_stop.parquet and out/patterns/pid_4588_segment.parquet
Processing stop time for all trips of pattern 4588
Originally 26804 trips
Trying to process 26520 trips for Pattern 4588 after filtering
Processed 26493 trips for Pattern 4588. There was 8 trip(s) with errors.
Time taken for stop times for 4588: 10.326784296830494 minutes
Writing out/patterns/pid_7103_stop.parquet and out/patterns/pid_7103_segment.parquet
Processing stop time for all trips of pattern 7103
Originally 15806 trips
Trying to process 15683 trips for Pattern 7103 after filtering
Processed 15628 trips for Pattern 7103. There was 21 trip(s) with errors.
Time taken for stop times for 7103: 8.614980014165242 minutes
Writing out/patterns/pid_1679_stop.parquet and out/patterns/pid_1679_segment.parquet
Processing stop time for all trips of pattern 1679
Originally 5774 trips
Trying to process 5733 trips for Pattern 1679 after filtering
Processed 5719 trips for Pattern 1679. There was 7 trip(s) with errors.
Time taken for stop times for 1679: 3.008294399579366 minutes
Writing out/patterns/pid_19369_stop.parquet and out/patterns/pid_19369_segment.parquet
Processing stop time for all trips of pattern 19369
Originally 5227 trips
Trying to process 5065 trips for Pattern 19369 after filtering
Processed 5040 trips for Pattern 19369. There was 9 trip(s) with errors.
Time taken for stop times for 19369: 2.46574068069458 minutes
Writing out/patterns/pid_4277_stop.parquet and out/patterns/pid_4277_segment.parquet
Processing stop time for all trips of pattern 4277
Originally 3922 trips
Trying to process 3834 trips for Pattern 4277 after filtering
Processed 3785 trips for Pattern 4277. There was 21 trip(s) with errors.
Time taken for stop times for 4277: 1.7776953140894571 minutes
Writing out/patterns/pid_5671_stop.parquet and out/patterns/pid_5671_segment.parquet
Processing stop time for all trips of pattern 5671
Originally 492 trips
Trying to process 488 trips for Pattern 5671 after filtering
Processed 488 trips for Pattern 5671. There was 0 trip(s) with errors.
Time taken for stop times for 5671: 0.19461461305618286 minutes
Do not have pattern 4529 available currently
Writing out/patterns/pid_1450_stop.parquet and out/patterns/pid_1450_segment.parquet
Processing stop time for all trips of pattern 1450
Originally 650 trips
Trying to process 645 trips for Pattern 1450 after filtering
Processed 642 trips for Pattern 1450. There was 1 trip(s) with errors.
Time taken for stop times for 1450: 0.2369392991065979 minutes
Writing out/patterns/pid_14105_stop.parquet and out/patterns/pid_14105_segment.parquet
Processing stop time for all trips of pattern 14105
Originally 94 trips
Trying to process 90 trips for Pattern 14105 after filtering
Processed 90 trips for Pattern 14105. There was 0 trip(s) with errors.
Time taken for stop times for 14105: 0.03353158632914225 minutes
Writing out/patterns/pid_1286_stop.parquet and out/patterns/pid_1286_segment.parquet
Processing stop time for all trips of pattern 1286
Originally 14980 trips
Trying to process 14801 trips for Pattern 1286 after filtering
Processed 14747 trips for Pattern 1286. There was 20 trip(s) with errors.
Time taken for stop times for 1286: 5.560709663232168 minutes
Writing out/patterns/pid_1144_stop.parquet and out/patterns/pid_1144_segment.parquet
Processing stop time for all trips of pattern 1144
Originally 2547 trips
Trying to process 1172 trips for Pattern 1144 after filtering
Processed 1152 trips for Pattern 1144. There was 10 trip(s) with errors.
Time taken for stop times for 1144: 0.2928363045056661 minutes
Writing out/patterns/pid_7012_stop.parquet and out/patterns/pid_7012_segment.parquet
Processing stop time for all trips of pattern 7012
Originally 2241 trips
Trying to process 2219 trips for Pattern 7012 after filtering
Processed 2218 trips for Pattern 7012. There was 0 trip(s) with errors.
Time taken for stop times for 7012: 0.6810872316360473 minutes
Writing out/patterns/pid_2673_stop.parquet and out/patterns/pid_2673_segment.parquet
Processing stop time for all trips of pattern 2673
Originally 2116 trips
Trying to process 2097 trips for Pattern 2673 after filtering
Processed 2085 trips for Pattern 2673. There was 5 trip(s) with errors.
Time taken for stop times for 2673: 0.7835386673609416 minutes
Writing out/patterns/pid_1117_stop.parquet and out/patterns/pid_1117_segment.parquet
Processing stop time for all trips of pattern 1117
Originally 5399 trips
Trying to process 5316 trips for Pattern 1117 after filtering
Processed 5279 trips for Pattern 1117. There was 14 trip(s) with errors.
Time taken for stop times for 1117: 2.935214948654175 minutes
Writing out/patterns/pid_16548_stop.parquet and out/patterns/pid_16548_segment.parquet
Processing stop time for all trips of pattern 16548
Originally 20557 trips
Trying to process 20244 trips for Pattern 16548 after filtering
Processed 20178 trips for Pattern 16548. There was 29 trip(s) with errors.
Time taken for stop times for 16548: 7.317401083310445 minutes
Writing out/patterns/pid_5020_stop.parquet and out/patterns/pid_5020_segment.parquet
Processing stop time for all trips of pattern 5020
Originally 2117 trips
Trying to process 2097 trips for Pattern 5020 after filtering
Processed 2090 trips for Pattern 5020. There was 3 trip(s) with errors.
Time taken for stop times for 5020: 0.8512767990430196 minutes
Writing out/patterns/pid_2162_stop.parquet and out/patterns/pid_2162_segment.parquet
Processing stop time for all trips of pattern 2162
Originally 9154 trips
Trying to process 8753 trips for Pattern 2162 after filtering
Processed 8479 trips for Pattern 2162. There was 136 trip(s) with errors.
Time taken for stop times for 2162: 3.569109567006429 minutes
Writing out/patterns/pid_1656_stop.parquet and out/patterns/pid_1656_segment.parquet
Processing stop time for all trips of pattern 1656
Originally 3959 trips
Trying to process 3656 trips for Pattern 1656 after filtering
Processed 3636 trips for Pattern 1656. There was 9 trip(s) with errors.
Time taken for stop times for 1656: 2.0073548038800557 minutes
Writing out/patterns/pid_10937_stop.parquet and out/patterns/pid_10937_segment.parquet
Processing stop time for all trips of pattern 10937
Originally 390 trips
Trying to process 385 trips for Pattern 10937 after filtering
Processed 385 trips for Pattern 10937. There was 0 trip(s) with errors.
Time taken for stop times for 10937: 0.12295843362808227 minutes
Writing out/patterns/pid_1293_stop.parquet and out/patterns/pid_1293_segment.parquet
Processing stop time for all trips of pattern 1293
Originally 4982 trips
Trying to process 4933 trips for Pattern 1293 after filtering
Processed 4921 trips for Pattern 1293. There was 5 trip(s) with errors.
Time taken for stop times for 1293: 2.5114739378293356 minutes
Writing out/patterns/pid_3634_stop.parquet and out/patterns/pid_3634_segment.parquet
Processing stop time for all trips of pattern 3634
Originally 7867 trips
Trying to process 6201 trips for Pattern 3634 after filtering
Processed 6177 trips for Pattern 3634. There was 8 trip(s) with errors.
Time taken for stop times for 3634: 1.5744930028915405 minutes
Writing out/patterns/pid_6398_stop.parquet and out/patterns/pid_6398_segment.parquet
Processing stop time for all trips of pattern 6398
Originally 1050 trips
Trying to process 1027 trips for Pattern 6398 after filtering
Processed 1023 trips for Pattern 6398. There was 0 trip(s) with errors.
Time taken for stop times for 6398: 0.32015851736068723 minutes
Writing out/patterns/pid_7097_stop.parquet and out/patterns/pid_7097_segment.parquet
Processing stop time for all trips of pattern 7097
Originally 166 trips
Trying to process 166 trips for Pattern 7097 after filtering
Processed 166 trips for Pattern 7097. There was 0 trip(s) with errors.
Time taken for stop times for 7097: 0.0861349860827128 minutes
Writing out/patterns/pid_7005_stop.parquet and out/patterns/pid_7005_segment.parquet
Processing stop time for all trips of pattern 7005
Originally 1252 trips
Trying to process 1247 trips for Pattern 7005 after filtering
Processed 1240 trips for Pattern 7005. There was 3 trip(s) with errors.
Time taken for stop times for 7005: 0.5107677141825359 minutes
Writing out/patterns/pid_5194_stop.parquet and out/patterns/pid_5194_segment.parquet
Processing stop time for all trips of pattern 5194
Originally 923 trips
Trying to process 919 trips for Pattern 5194 after filtering
Processed 919 trips for Pattern 5194. There was 0 trip(s) with errors.
Time taken for stop times for 5194: 0.43293344577153525 minutes
Writing out/patterns/pid_14128_stop.parquet and out/patterns/pid_14128_segment.parquet
Processing stop time for all trips of pattern 14128
Originally 2130 trips
Trying to process 120 trips for Pattern 14128 after filtering
Processed 104 trips for Pattern 14128. There was 4 trip(s) with errors.
Time taken for stop times for 14128: 0.03348986705144246 minutes
Writing out/patterns/pid_4531_stop.parquet and out/patterns/pid_4531_segment.parquet
Processing stop time for all trips of pattern 4531
Originally 282 trips
Trying to process 275 trips for Pattern 4531 after filtering
Processed 273 trips for Pattern 4531. There was 1 trip(s) with errors.
Time taken for stop times for 4531: 0.09172048171361287 minutes
Writing out/patterns/pid_8422_stop.parquet and out/patterns/pid_8422_segment.parquet
Processing stop time for all trips of pattern 8422
Originally 26028 trips
Trying to process 25751 trips for Pattern 8422 after filtering
/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/Interpolation.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trip_df.loc[:, "data_time"] = pd.to_datetime(trip_df.data_time)
Processed 25725 trips for Pattern 8422. There was 10 trip(s) with errors.
Time taken for stop times for 8422: 9.98631078004837 minutes
Writing out/patterns/pid_7387_stop.parquet and out/patterns/pid_7387_segment.parquet
Processing stop time for all trips of pattern 7387
Originally 61 trips
Trying to process 58 trips for Pattern 7387 after filtering
Processed 57 trips for Pattern 7387. There was 0 trip(s) with errors.
Time taken for stop times for 7387: 0.02610593239466349 minutes
Writing out/patterns/pid_14104_stop.parquet and out/patterns/pid_14104_segment.parquet
Processing stop time for all trips of pattern 14104
Originally 417 trips
Trying to process 414 trips for Pattern 14104 after filtering
Processed 408 trips for Pattern 14104. There was 3 trip(s) with errors.
Time taken for stop times for 14104: 0.239850648244222 minutes
Writing out/patterns/pid_2107_stop.parquet and out/patterns/pid_2107_segment.parquet
Processing stop time for all trips of pattern 2107
Originally 5325 trips
Trying to process 5255 trips for Pattern 2107 after filtering
Processed 5232 trips for Pattern 2107. There was 10 trip(s) with errors.
Time taken for stop times for 2107: 1.6882439335187276 minutes
Writing out/patterns/pid_6125_stop.parquet and out/patterns/pid_6125_segment.parquet
Processing stop time for all trips of pattern 6125
Originally 620 trips
Trying to process 568 trips for Pattern 6125 after filtering
Processed 558 trips for Pattern 6125. There was 5 trip(s) with errors.
Time taken for stop times for 6125: 0.21371163527170817 minutes
Writing out/patterns/pid_3189_stop.parquet and out/patterns/pid_3189_segment.parquet
Processing stop time for all trips of pattern 3189
Originally 717 trips
Trying to process 717 trips for Pattern 3189 after filtering
Processed 716 trips for Pattern 3189. There was 0 trip(s) with errors.
Time taken for stop times for 3189: 0.25677146911621096 minutes
Writing out/patterns/pid_6509_stop.parquet and out/patterns/pid_6509_segment.parquet
Processing stop time for all trips of pattern 6509
Originally 44 trips
Trying to process 42 trips for Pattern 6509 after filtering
Processed 42 trips for Pattern 6509. There was 0 trip(s) with errors.
Time taken for stop times for 6509: 0.013281933466593425 minutes
Writing out/patterns/pid_5905_stop.parquet and out/patterns/pid_5905_segment.parquet
Processing stop time for all trips of pattern 5905
Originally 25597 trips
Trying to process 25148 trips for Pattern 5905 after filtering
Processed 25051 trips for Pattern 5905. There was 38 trip(s) with errors.
Time taken for stop times for 5905: 11.451653480529785 minutes
Writing out/patterns/pid_749_stop.parquet and out/patterns/pid_749_segment.parquet
Processing stop time for all trips of pattern 749
Originally 18428 trips
Trying to process 18238 trips for Pattern 749 after filtering
Processed 18154 trips for Pattern 749. There was 33 trip(s) with errors.
Time taken for stop times for 749: 7.65634503364563 minutes
Writing out/patterns/pid_4555_stop.parquet and out/patterns/pid_4555_segment.parquet
Processing stop time for all trips of pattern 4555
Originally 20036 trips
Trying to process 19587 trips for Pattern 4555 after filtering
/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/Interpolation.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trip_df.loc[:, "data_time"] = pd.to_datetime(trip_df.data_time)
Processed 19483 trips for Pattern 4555. There was 47 trip(s) with errors.
Time taken for stop times for 4555: 8.89144725004832 minutes
Writing out/patterns/pid_7455_stop.parquet and out/patterns/pid_7455_segment.parquet
Processing stop time for all trips of pattern 7455
Originally 221 trips
Trying to process 220 trips for Pattern 7455 after filtering
Processed 220 trips for Pattern 7455. There was 0 trip(s) with errors.
Time taken for stop times for 7455: 0.06321706771850585 minutes
Writing out/patterns/pid_23278_stop.parquet and out/patterns/pid_23278_segment.parquet
Processing stop time for all trips of pattern 23278
Originally 4 trips
Trying to process 4 trips for Pattern 23278 after filtering
Processed 4 trips for Pattern 23278. There was 0 trip(s) with errors.
Time taken for stop times for 23278: 0.002095230420430501 minutes
Writing out/patterns/pid_14107_stop.parquet and out/patterns/pid_14107_segment.parquet
Processing stop time for all trips of pattern 14107
Originally 1183 trips
Trying to process 1158 trips for Pattern 14107 after filtering
Processed 1154 trips for Pattern 14107. There was 0 trip(s) with errors.
Time taken for stop times for 14107: 0.3335816343625387 minutes
Writing out/patterns/pid_8417_stop.parquet and out/patterns/pid_8417_segment.parquet
Processing stop time for all trips of pattern 8417
Originally 31 trips
Trying to process 31 trips for Pattern 8417 after filtering
Processed 31 trips for Pattern 8417. There was 0 trip(s) with errors.
Time taken for stop times for 8417: 0.009416683514912923 minutes
Writing out/patterns/pid_10877_stop.parquet and out/patterns/pid_10877_segment.parquet
Processing stop time for all trips of pattern 10877
Originally 84 trips
Trying to process 83 trips for Pattern 10877 after filtering
Processed 83 trips for Pattern 10877. There was 0 trip(s) with errors.
Time taken for stop times for 10877: 0.03365491628646851 minutes
Writing out/patterns/pid_10944_stop.parquet and out/patterns/pid_10944_segment.parquet
Processing stop time for all trips of pattern 10944
Originally 31277 trips
Trying to process 30391 trips for Pattern 10944 after filtering
/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/Interpolation.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trip_df.loc[:, "data_time"] = pd.to_datetime(trip_df.data_time)
Processed 30234 trips for Pattern 10944. There was 74 trip(s) with errors.
Time taken for stop times for 10944: 13.89962211449941 minutes
Writing out/patterns/pid_10913_stop.parquet and out/patterns/pid_10913_segment.parquet
Processing stop time for all trips of pattern 10913
Originally 24591 trips
Trying to process 24371 trips for Pattern 10913 after filtering
/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/Interpolation.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trip_df.loc[:, "data_time"] = pd.to_datetime(trip_df.data_time)
Processed 24332 trips for Pattern 10913. There was 16 trip(s) with errors.
Time taken for stop times for 10913: 11.001397101084391 minutes
Writing out/patterns/pid_9344_stop.parquet and out/patterns/pid_9344_segment.parquet
Processing stop time for all trips of pattern 9344
Originally 474 trips
Trying to process 468 trips for Pattern 9344 after filtering
Processed 468 trips for Pattern 9344. There was 0 trip(s) with errors.
Time taken for stop times for 9344: 0.15651891628901163 minutes
Writing out/patterns/pid_7128_stop.parquet and out/patterns/pid_7128_segment.parquet
Processing stop time for all trips of pattern 7128
Originally 13629 trips
Trying to process 13465 trips for Pattern 7128 after filtering
Processed 13413 trips for Pattern 7128. There was 23 trip(s) with errors.
Time taken for stop times for 7128: 5.043729031085968 minutes
Writing out/patterns/pid_1315_stop.parquet and out/patterns/pid_1315_segment.parquet
Processing stop time for all trips of pattern 1315
Originally 132 trips
Trying to process 130 trips for Pattern 1315 after filtering
Processed 126 trips for Pattern 1315. There was 2 trip(s) with errors.
Time taken for stop times for 1315: 0.043724767367045086 minutes
Writing out/patterns/pid_7184_stop.parquet and out/patterns/pid_7184_segment.parquet
Processing stop time for all trips of pattern 7184
Originally 305 trips
Trying to process 298 trips for Pattern 7184 after filtering
Processed 285 trips for Pattern 7184. There was 5 trip(s) with errors.
Time taken for stop times for 7184: 0.1190002997716268 minutes
Writing out/patterns/pid_730_stop.parquet and out/patterns/pid_730_segment.parquet
Processing stop time for all trips of pattern 730
Originally 772 trips
Trying to process 767 trips for Pattern 730 after filtering
Processed 763 trips for Pattern 730. There was 2 trip(s) with errors.
Time taken for stop times for 730: 0.29527374903361003 minutes
Writing out/patterns/pid_6354_stop.parquet and out/patterns/pid_6354_segment.parquet
Processing stop time for all trips of pattern 6354
Originally 22011 trips
Trying to process 21486 trips for Pattern 6354 after filtering
Processed 21370 trips for Pattern 6354. There was 56 trip(s) with errors.
Time taken for stop times for 6354: 8.646917247772217 minutes
Writing out/patterns/pid_8088_stop.parquet and out/patterns/pid_8088_segment.parquet
Processing stop time for all trips of pattern 8088
Originally 694 trips
Trying to process 683 trips for Pattern 8088 after filtering
Processed 674 trips for Pattern 8088. There was 4 trip(s) with errors.
Time taken for stop times for 8088: 0.18669408162434895 minutes
Writing out/patterns/pid_7805_stop.parquet and out/patterns/pid_7805_segment.parquet
Processing stop time for all trips of pattern 7805
Originally 6570 trips
Trying to process 6510 trips for Pattern 7805 after filtering
Processed 6493 trips for Pattern 7805. There was 8 trip(s) with errors.
Time taken for stop times for 7805: 2.2946001013120014 minutes
Writing out/patterns/pid_5376_stop.parquet and out/patterns/pid_5376_segment.parquet
Processing stop time for all trips of pattern 5376
Originally 153 trips
Trying to process 153 trips for Pattern 5376 after filtering
Processed 153 trips for Pattern 5376. There was 0 trip(s) with errors.
Time taken for stop times for 5376: 0.0418777346611023 minutes
Writing out/patterns/pid_1180_stop.parquet and out/patterns/pid_1180_segment.parquet
Processing stop time for all trips of pattern 1180
Originally 36531 trips
Trying to process 35712 trips for Pattern 1180 after filtering
Processed 35315 trips for Pattern 1180. There was 176 trip(s) with errors.
Time taken for stop times for 1180: 19.450525299708048 minutes
Writing out/patterns/pid_4524_stop.parquet and out/patterns/pid_4524_segment.parquet
Processing stop time for all trips of pattern 4524
Originally 24064 trips
Trying to process 23685 trips for Pattern 4524 after filtering
Processed 23442 trips for Pattern 4524. There was 101 trip(s) with errors.
Time taken for stop times for 4524: 10.34306146701177 minutes
Writing out/patterns/pid_1275_stop.parquet and out/patterns/pid_1275_segment.parquet
Processing stop time for all trips of pattern 1275
Originally 4692 trips
Trying to process 4669 trips for Pattern 1275 after filtering
Processed 4656 trips for Pattern 1275. There was 6 trip(s) with errors.
Time taken for stop times for 1275: 2.1335410833358766 minutes
Writing out/patterns/pid_9803_stop.parquet and out/patterns/pid_9803_segment.parquet
Processing stop time for all trips of pattern 9803
Originally 128 trips
Trying to process 125 trips for Pattern 9803 after filtering
Processed 125 trips for Pattern 9803. There was 0 trip(s) with errors.
Time taken for stop times for 9803: 0.05759666760762532 minutes
Writing out/patterns/pid_4509_stop.parquet and out/patterns/pid_4509_segment.parquet
Processing stop time for all trips of pattern 4509
Originally 5013 trips
Trying to process 4988 trips for Pattern 4509 after filtering
Processed 4978 trips for Pattern 4509. There was 4 trip(s) with errors.
Time taken for stop times for 4509: 1.6976510485013325 minutes
Writing out/patterns/pid_3470_stop.parquet and out/patterns/pid_3470_segment.parquet
Processing stop time for all trips of pattern 3470
Originally 2166 trips
Trying to process 2145 trips for Pattern 3470 after filtering
Processed 2136 trips for Pattern 3470. There was 4 trip(s) with errors.
Time taken for stop times for 3470: 0.7773407697677612 minutes
Writing out/patterns/pid_10903_stop.parquet and out/patterns/pid_10903_segment.parquet
Processing stop time for all trips of pattern 10903
Originally 1165 trips
Trying to process 1158 trips for Pattern 10903 after filtering
Processed 1155 trips for Pattern 10903. There was 1 trip(s) with errors.
Time taken for stop times for 10903: 0.4877328117688497 minutes
Writing out/patterns/pid_5675_stop.parquet and out/patterns/pid_5675_segment.parquet
Processing stop time for all trips of pattern 5675
Originally 11237 trips
Trying to process 11071 trips for Pattern 5675 after filtering
Processed 11031 trips for Pattern 5675. There was 17 trip(s) with errors.
Time taken for stop times for 5675: 3.47263126373291 minutes
Writing out/patterns/pid_10928_stop.parquet and out/patterns/pid_10928_segment.parquet
Processing stop time for all trips of pattern 10928
Originally 2074 trips
Trying to process 2058 trips for Pattern 10928 after filtering
Processed 2053 trips for Pattern 10928. There was 2 trip(s) with errors.
Time taken for stop times for 10928: 0.9145932356516521 minutes
Writing out/patterns/pid_5029_stop.parquet and out/patterns/pid_5029_segment.parquet
Processing stop time for all trips of pattern 5029
Originally 255 trips
Trying to process 250 trips for Pattern 5029 after filtering
Processed 250 trips for Pattern 5029. There was 0 trip(s) with errors.
Time taken for stop times for 5029: 0.08927503426869711 minutes
Writing out/patterns/pid_18414_stop.parquet and out/patterns/pid_18414_segment.parquet
Processing stop time for all trips of pattern 18414
Originally 30381 trips
Trying to process 29790 trips for Pattern 18414 after filtering
Processed 29594 trips for Pattern 18414. There was 90 trip(s) with errors.
Time taken for stop times for 18414: 16.141342906157174 minutes
Writing out/patterns/pid_10916_stop.parquet and out/patterns/pid_10916_segment.parquet
Processing stop time for all trips of pattern 10916
Originally 29498 trips
Trying to process 28891 trips for Pattern 10916 after filtering
Processed 28645 trips for Pattern 10916. There was 106 trip(s) with errors.
Time taken for stop times for 10916: 10.988266789913178 minutes
Writing out/patterns/pid_1341_stop.parquet and out/patterns/pid_1341_segment.parquet
Processing stop time for all trips of pattern 1341
Originally 1618 trips
Trying to process 1612 trips for Pattern 1341 after filtering
Processed 1612 trips for Pattern 1341. There was 0 trip(s) with errors.
Time taken for stop times for 1341: 0.8550366640090943 minutes
Writing out/patterns/pid_8120_stop.parquet and out/patterns/pid_8120_segment.parquet
Processing stop time for all trips of pattern 8120
Originally 1750 trips
Trying to process 1697 trips for Pattern 8120 after filtering
Processed 1686 trips for Pattern 8120. There was 4 trip(s) with errors.
Time taken for stop times for 8120: 1.008590583006541 minutes
Writing out/patterns/pid_5566_stop.parquet and out/patterns/pid_5566_segment.parquet
Processing stop time for all trips of pattern 5566
Originally 1546 trips
Trying to process 1483 trips for Pattern 5566 after filtering
Processed 1466 trips for Pattern 5566. There was 3 trip(s) with errors.
Time taken for stop times for 5566: 0.42196414470672605 minutes
Writing out/patterns/pid_1175_stop.parquet and out/patterns/pid_1175_segment.parquet
Processing stop time for all trips of pattern 1175
Originally 313 trips
Trying to process 311 trips for Pattern 1175 after filtering
Processed 309 trips for Pattern 1175. There was 1 trip(s) with errors.
Time taken for stop times for 1175: 0.10168893337249756 minutes
Writing out/patterns/pid_8469_stop.parquet and out/patterns/pid_8469_segment.parquet
Processing stop time for all trips of pattern 8469
Originally 3937 trips
Trying to process 3869 trips for Pattern 8469 after filtering
Processed 3847 trips for Pattern 8469. There was 10 trip(s) with errors.
Time taken for stop times for 8469: 1.5292849977811178 minutes
Writing out/patterns/pid_4357_stop.parquet and out/patterns/pid_4357_segment.parquet
Processing stop time for all trips of pattern 4357
Originally 2558 trips
Trying to process 2521 trips for Pattern 4357 after filtering
Processed 2519 trips for Pattern 4357. There was 0 trip(s) with errors.
Time taken for stop times for 4357: 1.0821645657221477 minutes
Writing out/patterns/pid_4609_stop.parquet and out/patterns/pid_4609_segment.parquet
Processing stop time for all trips of pattern 4609
Originally 2055 trips
Trying to process 1811 trips for Pattern 4609 after filtering
Processed 1721 trips for Pattern 4609. There was 36 trip(s) with errors.
Time taken for stop times for 4609: 0.6977551658948262 minutes
Writing out/patterns/pid_1541_stop.parquet and out/patterns/pid_1541_segment.parquet
Processing stop time for all trips of pattern 1541
Originally 24510 trips
Trying to process 23166 trips for Pattern 1541 after filtering
Processed 22931 trips for Pattern 1541. There was 113 trip(s) with errors.
Time taken for stop times for 1541: 14.398952082792919 minutes
Writing out/patterns/pid_752_stop.parquet and out/patterns/pid_752_segment.parquet
Processing stop time for all trips of pattern 752
Originally 18790 trips
Trying to process 18599 trips for Pattern 752 after filtering
Processed 18560 trips for Pattern 752. There was 14 trip(s) with errors.
Time taken for stop times for 752: 6.367997968196869 minutes
Writing out/patterns/pid_6657_stop.parquet and out/patterns/pid_6657_segment.parquet
Processing stop time for all trips of pattern 6657
Originally 18112 trips
Trying to process 17716 trips for Pattern 6657 after filtering
Processed 17616 trips for Pattern 6657. There was 44 trip(s) with errors.
Time taken for stop times for 6657: 10.360311118761699 minutes
Writing out/patterns/pid_5426_stop.parquet and out/patterns/pid_5426_segment.parquet
Processing stop time for all trips of pattern 5426
Originally 1859 trips
Trying to process 1834 trips for Pattern 5426 after filtering
Processed 1828 trips for Pattern 5426. There was 3 trip(s) with errors.
Time taken for stop times for 5426: 0.7027034044265748 minutes
Writing out/patterns/pid_3469_stop.parquet and out/patterns/pid_3469_segment.parquet
Processing stop time for all trips of pattern 3469
Originally 637 trips
Trying to process 630 trips for Pattern 3469 after filtering
Processed 623 trips for Pattern 3469. There was 3 trip(s) with errors.
Time taken for stop times for 3469: 0.18588189681371053 minutes
Writing out/patterns/pid_1971_stop.parquet and out/patterns/pid_1971_segment.parquet
Processing stop time for all trips of pattern 1971
Originally 20823 trips
Trying to process 20251 trips for Pattern 1971 after filtering
Processed 20168 trips for Pattern 1971. There was 33 trip(s) with errors.
Time taken for stop times for 1971: 8.960117395718893 minutes
Writing out/patterns/pid_954_stop.parquet and out/patterns/pid_954_segment.parquet
Processing stop time for all trips of pattern 954
Originally 19817 trips
Trying to process 19358 trips for Pattern 954 after filtering
Processed 19303 trips for Pattern 954. There was 18 trip(s) with errors.
Time taken for stop times for 954: 10.813534466425578 minutes
Writing out/patterns/pid_8084_stop.parquet and out/patterns/pid_8084_segment.parquet
Processing stop time for all trips of pattern 8084
Originally 1209 trips
Trying to process 1207 trips for Pattern 8084 after filtering
Processed 1207 trips for Pattern 8084. There was 0 trip(s) with errors.
Time taken for stop times for 8084: 0.40692540009816486 minutes
Writing out/patterns/pid_1184_stop.parquet and out/patterns/pid_1184_segment.parquet
Processing stop time for all trips of pattern 1184
Originally 787 trips
Trying to process 157 trips for Pattern 1184 after filtering
Processed 150 trips for Pattern 1184. There was 3 trip(s) with errors.
Time taken for stop times for 1184: 0.03491894801457723 minutes
Writing out/patterns/pid_1609_stop.parquet and out/patterns/pid_1609_segment.parquet
Processing stop time for all trips of pattern 1609
Originally 162 trips
Trying to process 161 trips for Pattern 1609 after filtering
Processed 161 trips for Pattern 1609. There was 0 trip(s) with errors.
Time taken for stop times for 1609: 0.06713121732076009 minutes
Writing out/patterns/pid_18412_stop.parquet and out/patterns/pid_18412_segment.parquet
Processing stop time for all trips of pattern 18412
Originally 170 trips
Trying to process 163 trips for Pattern 18412 after filtering
Processed 161 trips for Pattern 18412. There was 0 trip(s) with errors.
Time taken for stop times for 18412: 0.06956949631373087 minutes
Do not have pattern 20423 available currently
Writing out/patterns/pid_1277_stop.parquet and out/patterns/pid_1277_segment.parquet
Processing stop time for all trips of pattern 1277
Originally 15499 trips
Trying to process 15260 trips for Pattern 1277 after filtering
Processed 15188 trips for Pattern 1277. There was 25 trip(s) with errors.
Time taken for stop times for 1277: 5.840846165021261 minutes
Writing out/patterns/pid_4341_stop.parquet and out/patterns/pid_4341_segment.parquet
Processing stop time for all trips of pattern 4341
Originally 340 trips
Trying to process 339 trips for Pattern 4341 after filtering
Processed 336 trips for Pattern 4341. There was 1 trip(s) with errors.
Time taken for stop times for 4341: 0.18706804911295574 minutes
Writing out/patterns/pid_5364_stop.parquet and out/patterns/pid_5364_segment.parquet
Processing stop time for all trips of pattern 5364
Originally 350 trips
Trying to process 348 trips for Pattern 5364 after filtering
Processed 348 trips for Pattern 5364. There was 0 trip(s) with errors.
Time taken for stop times for 5364: 0.15397274891535442 minutes
Writing out/patterns/pid_1231_stop.parquet and out/patterns/pid_1231_segment.parquet
Processing stop time for all trips of pattern 1231
Originally 1686 trips
Trying to process 1678 trips for Pattern 1231 after filtering
Processed 1674 trips for Pattern 1231. There was 1 trip(s) with errors.
Time taken for stop times for 1231: 0.6195279677708944 minutes
Writing out/patterns/pid_6492_stop.parquet and out/patterns/pid_6492_segment.parquet
Processing stop time for all trips of pattern 6492
Originally 88 trips
Trying to process 83 trips for Pattern 6492 after filtering
Processed 81 trips for Pattern 6492. There was 1 trip(s) with errors.
Time taken for stop times for 6492: 0.025339233875274658 minutes
Writing out/patterns/pid_4530_stop.parquet and out/patterns/pid_4530_segment.parquet
Processing stop time for all trips of pattern 4530
Originally 144 trips
Trying to process 143 trips for Pattern 4530 after filtering
Processed 143 trips for Pattern 4530. There was 0 trip(s) with errors.
Time taken for stop times for 4530: 0.07285132010777791 minutes
Writing out/patterns/pid_5901_stop.parquet and out/patterns/pid_5901_segment.parquet
Processing stop time for all trips of pattern 5901
Originally 109 trips
Trying to process 109 trips for Pattern 5901 after filtering
Processed 107 trips for Pattern 5901. There was 1 trip(s) with errors.
Time taken for stop times for 5901: 0.043931384881337486 minutes
Writing out/patterns/pid_4355_stop.parquet and out/patterns/pid_4355_segment.parquet
Processing stop time for all trips of pattern 4355
Originally 1918 trips
Trying to process 1903 trips for Pattern 4355 after filtering
Processed 1893 trips for Pattern 4355. There was 5 trip(s) with errors.
Time taken for stop times for 4355: 0.6432153662045796 minutes
Writing out/patterns/pid_8146_stop.parquet and out/patterns/pid_8146_segment.parquet
Processing stop time for all trips of pattern 8146
Originally 2752 trips
Trying to process 2606 trips for Pattern 8146 after filtering
Processed 2602 trips for Pattern 8146. There was 1 trip(s) with errors.
Time taken for stop times for 8146: 0.7526835997899374 minutes
Writing out/patterns/pid_7742_stop.parquet and out/patterns/pid_7742_segment.parquet
Processing stop time for all trips of pattern 7742
Originally 3501 trips
Trying to process 3365 trips for Pattern 7742 after filtering
Processed 3328 trips for Pattern 7742. There was 18 trip(s) with errors.
Time taken for stop times for 7742: 0.9108617345492045 minutes
Writing out/patterns/pid_712_stop.parquet and out/patterns/pid_712_segment.parquet
Processing stop time for all trips of pattern 712
Originally 2381 trips
Trying to process 2353 trips for Pattern 712 after filtering
Processed 2342 trips for Pattern 712. There was 5 trip(s) with errors.
Time taken for stop times for 712: 1.1928343335787455 minutes
Writing out/patterns/pid_8087_stop.parquet and out/patterns/pid_8087_segment.parquet
Processing stop time for all trips of pattern 8087
Originally 1432 trips
Trying to process 1398 trips for Pattern 8087 after filtering
Processed 1187 trips for Pattern 8087. There was 104 trip(s) with errors.
Time taken for stop times for 8087: 0.33171645005544026 minutes
Writing out/patterns/pid_7806_stop.parquet and out/patterns/pid_7806_segment.parquet
Processing stop time for all trips of pattern 7806
Originally 181 trips
Trying to process 178 trips for Pattern 7806 after filtering
Processed 177 trips for Pattern 7806. There was 0 trip(s) with errors.
Time taken for stop times for 7806: 0.06380664904912313 minutes
Writing out/patterns/pid_5530_stop.parquet and out/patterns/pid_5530_segment.parquet
Processing stop time for all trips of pattern 5530
Originally 4218 trips
Trying to process 4172 trips for Pattern 5530 after filtering
Processed 4157 trips for Pattern 5530. There was 7 trip(s) with errors.
Time taken for stop times for 5530: 1.7023919383684794 minutes
Writing out/patterns/pid_3527_stop.parquet and out/patterns/pid_3527_segment.parquet
Processing stop time for all trips of pattern 3527
Originally 10340 trips
Trying to process 10278 trips for Pattern 3527 after filtering
Processed 10264 trips for Pattern 3527. There was 6 trip(s) with errors.
Time taken for stop times for 3527: 4.48646358648936 minutes
Writing out/patterns/pid_4335_stop.parquet and out/patterns/pid_4335_segment.parquet
Processing stop time for all trips of pattern 4335
Originally 243 trips
Trying to process 233 trips for Pattern 4335 after filtering
Processed 233 trips for Pattern 4335. There was 0 trip(s) with errors.
Time taken for stop times for 4335: 0.071619979540507 minutes
Writing out/patterns/pid_1526_stop.parquet and out/patterns/pid_1526_segment.parquet
Processing stop time for all trips of pattern 1526
Originally 2923 trips
Trying to process 2903 trips for Pattern 1526 after filtering
Processed 2887 trips for Pattern 1526. There was 7 trip(s) with errors.
Time taken for stop times for 1526: 1.0803784807523091 minutes
Writing out/patterns/pid_3916_stop.parquet and out/patterns/pid_3916_segment.parquet
Processing stop time for all trips of pattern 3916
Originally 15308 trips
Trying to process 15199 trips for Pattern 3916 after filtering
Processed 15161 trips for Pattern 3916. There was 17 trip(s) with errors.
Time taken for stop times for 3916: 5.7653609156608585 minutes
Writing out/patterns/pid_3953_stop.parquet and out/patterns/pid_3953_segment.parquet
Processing stop time for all trips of pattern 3953
Originally 205 trips
Trying to process 204 trips for Pattern 3953 after filtering
Processed 204 trips for Pattern 3953. There was 0 trip(s) with errors.
Time taken for stop times for 3953: 0.10442983309427897 minutes
Writing out/patterns/pid_8115_stop.parquet and out/patterns/pid_8115_segment.parquet
Processing stop time for all trips of pattern 8115
Originally 413 trips
Trying to process 406 trips for Pattern 8115 after filtering
Processed 396 trips for Pattern 8115. There was 5 trip(s) with errors.
Time taken for stop times for 8115: 0.11939678192138672 minutes
Writing out/patterns/pid_3474_stop.parquet and out/patterns/pid_3474_segment.parquet
Processing stop time for all trips of pattern 3474
Originally 174 trips
Trying to process 172 trips for Pattern 3474 after filtering
Processed 172 trips for Pattern 3474. There was 0 trip(s) with errors.
Time taken for stop times for 3474: 0.049727686246236164 minutes
Writing out/patterns/pid_17465_stop.parquet and out/patterns/pid_17465_segment.parquet
Processing stop time for all trips of pattern 17465
Originally 88 trips
Trying to process 87 trips for Pattern 17465 after filtering
Processed 87 trips for Pattern 17465. There was 0 trip(s) with errors.
Time taken for stop times for 17465: 0.031082963943481444 minutes
Writing out/patterns/pid_7112_stop.parquet and out/patterns/pid_7112_segment.parquet
Processing stop time for all trips of pattern 7112
Originally 256 trips
Trying to process 253 trips for Pattern 7112 after filtering
Processed 253 trips for Pattern 7112. There was 0 trip(s) with errors.
Time taken for stop times for 7112: 0.11860001881917318 minutes
Writing out/patterns/pid_3463_stop.parquet and out/patterns/pid_3463_segment.parquet
Processing stop time for all trips of pattern 3463
Originally 1093 trips
Trying to process 1091 trips for Pattern 3463 after filtering
Processed 1088 trips for Pattern 3463. There was 1 trip(s) with errors.
Time taken for stop times for 3463: 0.3808091998100281 minutes
Writing out/patterns/pid_710_stop.parquet and out/patterns/pid_710_segment.parquet
Processing stop time for all trips of pattern 710
Originally 172 trips
Trying to process 167 trips for Pattern 710 after filtering
Processed 164 trips for Pattern 710. There was 1 trip(s) with errors.
Time taken for stop times for 710: 0.05404380162556966 minutes
Writing out/patterns/pid_6395_stop.parquet and out/patterns/pid_6395_segment.parquet
Processing stop time for all trips of pattern 6395
Originally 5006 trips
Trying to process 4968 trips for Pattern 6395 after filtering
Processed 4944 trips for Pattern 6395. There was 9 trip(s) with errors.
Time taken for stop times for 6395: 2.053623346487681 minutes
Writing out/patterns/pid_14103_stop.parquet and out/patterns/pid_14103_segment.parquet
Processing stop time for all trips of pattern 14103
Originally 16427 trips
Trying to process 16191 trips for Pattern 14103 after filtering
Processed 16104 trips for Pattern 14103. There was 41 trip(s) with errors.
Time taken for stop times for 14103: 4.282463351885478 minutes
Writing out/patterns/pid_6547_stop.parquet and out/patterns/pid_6547_segment.parquet
Processing stop time for all trips of pattern 6547
Originally 10266 trips
Trying to process 9957 trips for Pattern 6547 after filtering
Processed 9916 trips for Pattern 6547. There was 19 trip(s) with errors.
Time taken for stop times for 6547: 3.675558880964915 minutes
Writing out/patterns/pid_19363_stop.parquet and out/patterns/pid_19363_segment.parquet
Processing stop time for all trips of pattern 19363
Originally 141 trips
Trying to process 134 trips for Pattern 19363 after filtering
Processed 126 trips for Pattern 19363. There was 4 trip(s) with errors.
Time taken for stop times for 19363: 0.03978753089904785 minutes
Writing out/patterns/pid_8089_stop.parquet and out/patterns/pid_8089_segment.parquet
Processing stop time for all trips of pattern 8089
Originally 1646 trips
Trying to process 1461 trips for Pattern 8089 after filtering
Processed 901 trips for Pattern 8089. There was 275 trip(s) with errors.
Time taken for stop times for 8089: 0.4039482355117798 minutes
Writing out/patterns/pid_1342_stop.parquet and out/patterns/pid_1342_segment.parquet
Processing stop time for all trips of pattern 1342
Originally 1908 trips
Trying to process 1898 trips for Pattern 1342 after filtering
Processed 1895 trips for Pattern 1342. There was 1 trip(s) with errors.
Time taken for stop times for 1342: 1.076461919148763 minutes
Writing out/patterns/pid_1799_stop.parquet and out/patterns/pid_1799_segment.parquet
Processing stop time for all trips of pattern 1799
Originally 1824 trips
Trying to process 282 trips for Pattern 1799 after filtering
Processed 275 trips for Pattern 1799. There was 3 trip(s) with errors.
Time taken for stop times for 1799: 0.06852923234303793 minutes
Writing out/patterns/pid_7172_stop.parquet and out/patterns/pid_7172_segment.parquet
Processing stop time for all trips of pattern 7172
Originally 1965 trips
Trying to process 1953 trips for Pattern 7172 after filtering
Processed 1953 trips for Pattern 7172. There was 0 trip(s) with errors.
Time taken for stop times for 7172: 0.6915046811103821 minutes
Writing out/patterns/pid_7210_stop.parquet and out/patterns/pid_7210_segment.parquet
Processing stop time for all trips of pattern 7210
Originally 726 trips
Trying to process 721 trips for Pattern 7210 after filtering
Processed 719 trips for Pattern 7210. There was 1 trip(s) with errors.
Time taken for stop times for 7210: 0.3052845478057861 minutes
Writing out/patterns/pid_2063_stop.parquet and out/patterns/pid_2063_segment.parquet
Processing stop time for all trips of pattern 2063
Originally 12405 trips
Trying to process 12241 trips for Pattern 2063 after filtering
Processed 12201 trips for Pattern 2063. There was 15 trip(s) with errors.
Time taken for stop times for 2063: 5.169632871945699 minutes
Writing out/patterns/pid_8381_stop.parquet and out/patterns/pid_8381_segment.parquet
Processing stop time for all trips of pattern 8381
Originally 227 trips
Trying to process 226 trips for Pattern 8381 after filtering
Processed 226 trips for Pattern 8381. There was 0 trip(s) with errors.
Time taken for stop times for 8381: 0.11263535022735596 minutes
Writing out/patterns/pid_7127_stop.parquet and out/patterns/pid_7127_segment.parquet
Processing stop time for all trips of pattern 7127
Originally 8867 trips
Trying to process 8741 trips for Pattern 7127 after filtering
Processed 8723 trips for Pattern 7127. There was 6 trip(s) with errors.
Time taken for stop times for 7127: 3.060771969954173 minutes
Writing out/patterns/pid_19376_stop.parquet and out/patterns/pid_19376_segment.parquet
Processing stop time for all trips of pattern 19376
Originally 144 trips
Trying to process 123 trips for Pattern 19376 after filtering
Processed 113 trips for Pattern 19376. There was 4 trip(s) with errors.
Time taken for stop times for 19376: 0.040514814853668216 minutes
Do not have pattern 7386 available currently
Writing out/patterns/pid_10924_stop.parquet and out/patterns/pid_10924_segment.parquet
Processing stop time for all trips of pattern 10924
Originally 2383 trips
Trying to process 2290 trips for Pattern 10924 after filtering
Processed 2276 trips for Pattern 10924. There was 7 trip(s) with errors.
Time taken for stop times for 10924: 0.9727357983589172 minutes
Writing out/patterns/pid_4582_stop.parquet and out/patterns/pid_4582_segment.parquet
Processing stop time for all trips of pattern 4582
Originally 435 trips
Trying to process 431 trips for Pattern 4582 after filtering
Processed 400 trips for Pattern 4582. There was 0 trip(s) with errors.
Time taken for stop times for 4582: 0.12692861557006835 minutes
Writing out/patterns/pid_10941_stop.parquet and out/patterns/pid_10941_segment.parquet
Processing stop time for all trips of pattern 10941
Originally 9379 trips
Trying to process 8788 trips for Pattern 10941 after filtering
Processed 8709 trips for Pattern 10941. There was 38 trip(s) with errors.
Time taken for stop times for 10941: 3.5116240342458087 minutes
Writing out/patterns/pid_19381_stop.parquet and out/patterns/pid_19381_segment.parquet
Processing stop time for all trips of pattern 19381
Originally 57 trips
Trying to process 57 trips for Pattern 19381 after filtering
Processed 57 trips for Pattern 19381. There was 0 trip(s) with errors.
Time taken for stop times for 19381: 0.034155233701070147 minutes
Writing out/patterns/pid_14118_stop.parquet and out/patterns/pid_14118_segment.parquet
Processing stop time for all trips of pattern 14118
Originally 21806 trips
Trying to process 21314 trips for Pattern 14118 after filtering
Processed 21174 trips for Pattern 14118. There was 56 trip(s) with errors.
Time taken for stop times for 14118: 13.646463100115458 minutes
Writing out/patterns/pid_6357_stop.parquet and out/patterns/pid_6357_segment.parquet
Processing stop time for all trips of pattern 6357
Originally 22620 trips
Trying to process 22279 trips for Pattern 6357 after filtering
Processed 22165 trips for Pattern 6357. There was 52 trip(s) with errors.
Time taken for stop times for 6357: 9.098934320608775 minutes
Writing out/patterns/pid_1707_stop.parquet and out/patterns/pid_1707_segment.parquet
Processing stop time for all trips of pattern 1707
Originally 470 trips
Trying to process 465 trips for Pattern 1707 after filtering
Processed 465 trips for Pattern 1707. There was 0 trip(s) with errors.
Time taken for stop times for 1707: 0.20661695003509523 minutes
Writing out/patterns/pid_6137_stop.parquet and out/patterns/pid_6137_segment.parquet
Processing stop time for all trips of pattern 6137
Originally 10756 trips
Trying to process 10620 trips for Pattern 6137 after filtering
Processed 10524 trips for Pattern 6137. There was 44 trip(s) with errors.
Time taken for stop times for 6137: 3.5151410659154254 minutes
Writing out/patterns/pid_4726_stop.parquet and out/patterns/pid_4726_segment.parquet
Processing stop time for all trips of pattern 4726
Originally 3377 trips
Trying to process 3341 trips for Pattern 4726 after filtering
Processed 3331 trips for Pattern 4726. There was 4 trip(s) with errors.
Time taken for stop times for 4726: 1.248327084382375 minutes
Writing out/patterns/pid_4720_stop.parquet and out/patterns/pid_4720_segment.parquet
Processing stop time for all trips of pattern 4720
Originally 3263 trips
Trying to process 3245 trips for Pattern 4720 after filtering
Processed 3237 trips for Pattern 4720. There was 2 trip(s) with errors.
Time taken for stop times for 4720: 1.2303340355555217 minutes
Do not have pattern 6659 available currently
Writing out/patterns/pid_8148_stop.parquet and out/patterns/pid_8148_segment.parquet
Processing stop time for all trips of pattern 8148
Originally 1400 trips
Trying to process 1390 trips for Pattern 8148 after filtering
Processed 1386 trips for Pattern 8148. There was 2 trip(s) with errors.
Time taken for stop times for 8148: 1.0815134525299073 minutes
Writing out/patterns/pid_6489_stop.parquet and out/patterns/pid_6489_segment.parquet
Processing stop time for all trips of pattern 6489
Originally 974 trips
Trying to process 971 trips for Pattern 6489 after filtering
Processed 969 trips for Pattern 6489. There was 1 trip(s) with errors.
Time taken for stop times for 6489: 0.3900436798731486 minutes
Writing out/patterns/pid_5424_stop.parquet and out/patterns/pid_5424_segment.parquet
Processing stop time for all trips of pattern 5424
Originally 27380 trips
Trying to process 26591 trips for Pattern 5424 after filtering
Processed 26274 trips for Pattern 5424. There was 152 trip(s) with errors.
Time taken for stop times for 5424: 11.879397197564442 minutes
Writing out/patterns/pid_5531_stop.parquet and out/patterns/pid_5531_segment.parquet
Processing stop time for all trips of pattern 5531
Originally 2551 trips
Trying to process 2516 trips for Pattern 5531 after filtering
Processed 2492 trips for Pattern 5531. There was 10 trip(s) with errors.
Time taken for stop times for 5531: 1.017051601409912 minutes
Writing out/patterns/pid_7007_stop.parquet and out/patterns/pid_7007_segment.parquet
Processing stop time for all trips of pattern 7007
Originally 158 trips
Trying to process 158 trips for Pattern 7007 after filtering
Processed 158 trips for Pattern 7007. There was 0 trip(s) with errors.
Time taken for stop times for 7007: 0.053592832883199056 minutes
Writing out/patterns/pid_19379_stop.parquet and out/patterns/pid_19379_segment.parquet
Processing stop time for all trips of pattern 19379
Originally 22750 trips
Trying to process 22238 trips for Pattern 19379 after filtering
Processed 22041 trips for Pattern 19379. There was 95 trip(s) with errors.
Time taken for stop times for 19379: 13.440054166316987 minutes
Writing out/patterns/pid_14130_stop.parquet and out/patterns/pid_14130_segment.parquet
Processing stop time for all trips of pattern 14130
Originally 13872 trips
Trying to process 13740 trips for Pattern 14130 after filtering
Processed 13689 trips for Pattern 14130. There was 20 trip(s) with errors.
Time taken for stop times for 14130: 6.373389585812887 minutes
Writing out/patterns/pid_8123_stop.parquet and out/patterns/pid_8123_segment.parquet
Processing stop time for all trips of pattern 8123
Originally 4321 trips
Trying to process 4238 trips for Pattern 8123 after filtering
Processed 4093 trips for Pattern 8123. There was 71 trip(s) with errors.
Time taken for stop times for 8123: 2.2745675007502237 minutes
Writing out/patterns/pid_2402_stop.parquet and out/patterns/pid_2402_segment.parquet
Processing stop time for all trips of pattern 2402
Originally 9243 trips
Trying to process 9177 trips for Pattern 2402 after filtering
Processed 9133 trips for Pattern 2402. There was 20 trip(s) with errors.
Time taken for stop times for 2402: 3.272583782672882 minutes
Writing out/patterns/pid_5572_stop.parquet and out/patterns/pid_5572_segment.parquet
Processing stop time for all trips of pattern 5572
Originally 1560 trips
Trying to process 1532 trips for Pattern 5572 after filtering
Processed 1520 trips for Pattern 5572. There was 4 trip(s) with errors.
Time taken for stop times for 5572: 0.5140814304351806 minutes
Writing out/patterns/pid_6411_stop.parquet and out/patterns/pid_6411_segment.parquet
Processing stop time for all trips of pattern 6411
Originally 1941 trips
Trying to process 1836 trips for Pattern 6411 after filtering
Processed 1810 trips for Pattern 6411. There was 12 trip(s) with errors.
Time taken for stop times for 6411: 0.8105791171391805 minutes
Writing out/patterns/pid_5058_stop.parquet and out/patterns/pid_5058_segment.parquet
Processing stop time for all trips of pattern 5058
Originally 1 trips
Trying to process 1 trips for Pattern 5058 after filtering
Processed 1 trips for Pattern 5058. There was 0 trip(s) with errors.
Time taken for stop times for 5058: 0.0013059178988138835 minutes
Writing out/patterns/pid_751_stop.parquet and out/patterns/pid_751_segment.parquet
Processing stop time for all trips of pattern 751
Originally 1219 trips
Trying to process 1188 trips for Pattern 751 after filtering
Processed 1186 trips for Pattern 751. There was 1 trip(s) with errors.
Time taken for stop times for 751: 0.33099567095438637 minutes
Writing out/patterns/pid_6779_stop.parquet and out/patterns/pid_6779_segment.parquet
Processing stop time for all trips of pattern 6779
Originally 3759 trips
Trying to process 3738 trips for Pattern 6779 after filtering
Processed 3725 trips for Pattern 6779. There was 4 trip(s) with errors.
Time taken for stop times for 6779: 1.6643728017807007 minutes
Writing out/patterns/pid_8113_stop.parquet and out/patterns/pid_8113_segment.parquet
Processing stop time for all trips of pattern 8113
Originally 475 trips
Trying to process 471 trips for Pattern 8113 after filtering
Processed 471 trips for Pattern 8113. There was 0 trip(s) with errors.
Time taken for stop times for 8113: 0.17702188491821289 minutes
Writing out/patterns/pid_3910_stop.parquet and out/patterns/pid_3910_segment.parquet
Processing stop time for all trips of pattern 3910
Originally 1704 trips
Trying to process 1699 trips for Pattern 3910 after filtering
Processed 1699 trips for Pattern 3910. There was 0 trip(s) with errors.
Time taken for stop times for 3910: 0.9966869831085206 minutes
Writing out/patterns/pid_700_stop.parquet and out/patterns/pid_700_segment.parquet
Processing stop time for all trips of pattern 700
Originally 916 trips
Trying to process 853 trips for Pattern 700 after filtering
Processed 853 trips for Pattern 700. There was 0 trip(s) with errors.
Time taken for stop times for 700: 0.2373360315958659 minutes
Writing out/patterns/pid_8106_stop.parquet and out/patterns/pid_8106_segment.parquet
Processing stop time for all trips of pattern 8106
Originally 8408 trips
Trying to process 8334 trips for Pattern 8106 after filtering
Processed 8312 trips for Pattern 8106. There was 11 trip(s) with errors.
Time taken for stop times for 8106: 3.106964699427287 minutes
Writing out/patterns/pid_6773_stop.parquet and out/patterns/pid_6773_segment.parquet
Processing stop time for all trips of pattern 6773
Originally 1110 trips
Trying to process 1025 trips for Pattern 6773 after filtering
Processed 1007 trips for Pattern 6773. There was 7 trip(s) with errors.
Time taken for stop times for 6773: 0.2846776803334554 minutes
Writing out/patterns/pid_4245_stop.parquet and out/patterns/pid_4245_segment.parquet
Processing stop time for all trips of pattern 4245
Originally 382 trips
Trying to process 379 trips for Pattern 4245 after filtering
Processed 376 trips for Pattern 4245. There was 1 trip(s) with errors.
Time taken for stop times for 4245: 0.13033103148142497 minutes
Writing out/patterns/pid_6508_stop.parquet and out/patterns/pid_6508_segment.parquet
Processing stop time for all trips of pattern 6508
Originally 40 trips
Trying to process 40 trips for Pattern 6508 after filtering
Processed 40 trips for Pattern 6508. There was 0 trip(s) with errors.
Time taken for stop times for 6508: 0.016642598311106364 minutes
Writing out/patterns/pid_9371_stop.parquet and out/patterns/pid_9371_segment.parquet
Processing stop time for all trips of pattern 9371
Originally 875 trips
Trying to process 867 trips for Pattern 9371 after filtering
Processed 855 trips for Pattern 9371. There was 5 trip(s) with errors.
Time taken for stop times for 9371: 0.5238280018170675 minutes
Writing out/patterns/pid_8232_stop.parquet and out/patterns/pid_8232_segment.parquet
Processing stop time for all trips of pattern 8232
Originally 305 trips
Trying to process 117 trips for Pattern 8232 after filtering
Processed 97 trips for Pattern 8232. There was 1 trip(s) with errors.
Time taken for stop times for 8232: 0.025003317991892496 minutes
Writing out/patterns/pid_14109_stop.parquet and out/patterns/pid_14109_segment.parquet
Processing stop time for all trips of pattern 14109
Originally 54 trips
Trying to process 54 trips for Pattern 14109 after filtering
Processed 54 trips for Pattern 14109. There was 0 trip(s) with errors.
Time taken for stop times for 14109: 0.023521049817403158 minutes
Writing out/patterns/pid_6361_stop.parquet and out/patterns/pid_6361_segment.parquet
Processing stop time for all trips of pattern 6361
Originally 2733 trips
Trying to process 2679 trips for Pattern 6361 after filtering
Processed 2480 trips for Pattern 6361. There was 95 trip(s) with errors.
Time taken for stop times for 6361: 0.6109005848566691 minutes
Writing out/patterns/pid_3214_stop.parquet and out/patterns/pid_3214_segment.parquet
Processing stop time for all trips of pattern 3214
Originally 14513 trips
Trying to process 14343 trips for Pattern 3214 after filtering
Processed 14280 trips for Pattern 3214. There was 29 trip(s) with errors.
Time taken for stop times for 3214: 4.568849182128906 minutes
Writing out/patterns/pid_7458_stop.parquet and out/patterns/pid_7458_segment.parquet
Processing stop time for all trips of pattern 7458
Originally 18193 trips
Trying to process 18010 trips for Pattern 7458 after filtering
Processed 17985 trips for Pattern 7458. There was 8 trip(s) with errors.
Time taken for stop times for 7458: 8.434754753112793 minutes
Writing out/patterns/pid_7750_stop.parquet and out/patterns/pid_7750_segment.parquet
Processing stop time for all trips of pattern 7750
Originally 147 trips
Trying to process 139 trips for Pattern 7750 after filtering
Processed 130 trips for Pattern 7750. There was 1 trip(s) with errors.
Time taken for stop times for 7750: 0.03715363343556722 minutes
Writing out/patterns/pid_4287_stop.parquet and out/patterns/pid_4287_segment.parquet
Processing stop time for all trips of pattern 4287
Originally 23322 trips
Trying to process 22645 trips for Pattern 4287 after filtering
Processed 22504 trips for Pattern 4287. There was 64 trip(s) with errors.
Time taken for stop times for 4287: 12.068322046597798 minutes
"""

In [48]:
reg_patterns = r'Pattern\s([0-9]+)'
pattern = re.findall(reg_patterns, text)
pattern = set(pattern)
len(pattern)

len(p_trips)

840

In [42]:
import re
reg_time = r':\s([0-9]+.[0-9]+)'
reg_processed_trips = r'Processed\s([0-9]+)'
reg_error_trips = r'was\s([0-9]+)'
reg_og_trips = r'Originally\s([0-9]+)'


times = re.findall(reg_time, text)
p_trips = re.findall(reg_processed_trips, text)
errors = re.findall(reg_error_trips, text)
og_trips = re.findall(reg_og_trips, text)

total_time = sum([float(i) for i in times])
total_p_trips = sum([int(i) for i in p_trips])
total_og_trips = sum([int(i) for i in og_trips])
total_errors = sum([int(i) for i in errors])

print(f"There were {total_p_trips:,} processed trips out of {total_og_trips:,} 
      original trips ({round(total_p_trips/total_og_trips,3)*100}%) in {round(total_time/60,1)} 
      hours. There were {total_errors:,} trips with errors ({round(total_errors/total_p_trips,3)*100}% of processed trips).")



There were 4,932,234 processed trips out of 5,085,505 original trips (97.0%) in 59.6 hours. There were 13,314 trips with errors (0.3% of processed trips).
